## 載入資料

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#載入資料
import pandas as pd
import numpy as np
data = pd.read_csv('/content/drive/MyDrive/train.csv')
data.head(10)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
5,5,15771669,Genovese,588,Germany,Male,36.0,4,131778.58,1,1.0,0.0,136024.31,1
6,6,15692819,Ch'ang,593,France,Female,30.0,8,144772.69,1,1.0,0.0,29792.11,0
7,7,15669611,Chukwuebuka,678,Spain,Male,37.0,1,138476.41,1,1.0,0.0,106851.60,0
8,8,15691707,Manna,676,France,Male,43.0,4,0.00,2,1.0,0.0,142917.13,0
9,9,15591721,Cattaneo,583,Germany,Male,40.0,4,81274.33,1,1.0,1.0,170843.07,0


## 資料前處理

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
class AutoPreprocess:
    def __init__(self):
        self.scaler = {}
        self.fillna_value = {}
        self.onehotencode_value = {}
        self.field_names = []
        self.final_field_names = []

    def fit(self, data, field_names):
        self.__init__()
        self.field_names = field_names

        for fname in field_names:
            #自動補空值
            if (data[fname].dtype == object) or (data[fname].dtype == str): #字串型態欄位
                self.fillna_value[fname] = data[fname].mode()[0] #補眾數
                # self.fillna_value[fname] = 'np.nan'
                # self.fillna_value[fname] = np.nan # 維持空值
            elif data[fname].dtype == bool: #布林型態
                self.fillna_value[fname] = data[fname].mode()[0] #補眾數
            else: # 數字型態
                self.fillna_value[fname] = data[fname].median()  #補中位數

            #自動尺度轉換(scaling)
            if (data[fname].dtype == object) or (data[fname].dtype == str): #字串型態欄位
                pass #不用轉換
            elif data[fname].dtype == bool: #布林型態
                pass #不用轉換
            else: # 數字型態
                vc = data[fname].value_counts()
                if data[fname].isin([0, 1]).all(): #當數值只有0跟1
                    pass #不用轉換
                elif pd.api.types.is_integer_dtype(data[fname]) and data[fname].nunique() <= 10: #是否簡單的整數型類別且數量小於10
                    self.scaler[fname] = MinMaxScaler()
                    self.scaler[fname].fit(data[[fname]])
                else: #其他的數字型態
                    self.scaler[fname] = StandardScaler()
                    self.scaler[fname].fit(data[[fname]])


            #自動編碼
            if (data[fname].dtype == object) or (data[fname].dtype == str): #字串型態欄位, onehotencode
                field_value = data[fname].value_counts().index
                self.onehotencode_value[fname] = field_value
                for value in field_value:
                    fn = fname+"_"+value
                    # data[fn] = (data[fname] == value).astype('int8')
                    self.final_field_names.append(fn)
            elif data[fname].dtype == bool: #布林型態 轉成0跟1
                # data[fname] = data[fname].astype(int)
                self.final_field_names.append(fname)
            else: # 數字型態 不用重新編碼
                self.final_field_names.append(fname)

        return

    def transform(self, data):
        #如果輸入的data是dict，要先轉成dataframe
        if isinstance(data, dict):
            for fname in self.field_names:
                if fname in data:
                    data[fname] = [data[fname]]
                else:
                    data[fname] = [np.nan]
            data = pd.DataFrame(data)
        else: #將資料複製一份，不修改原本的資料
            data = data.copy()

        for fname in self.field_names:
            #自動補空值
            if data[fname].isnull().any(): #有空值
                # if fname in self.fillna_value:
                    data[fname] = data[fname].fillna(self.fillna_value[fname])


            #自動尺度轉換(scaling)
            if fname in self.scaler:
                data[fname] = self.scaler[fname].transform(data[[fname]])

            #自動編碼
            if (data[fname].dtype == object) or (data[fname].dtype == str): #字串型態欄位, onehotencode
                if fname in self.onehotencode_value:
                    field_value = self.onehotencode_value[fname]
                for value in field_value:
                    fn = fname+"_"+value
                    data[fn] = (data[fname] == value).astype('int8')
            elif data[fname].dtype == bool: #布林型態 轉成0跟1
                data[fname] = data[fname].astype(int)
            else: # 數字型態 不用重新編碼
                pass
        return data[self.final_field_names]

    def save(self, file_name):
        with open(file_name, "wb") as f:
            pickle.dump(self, f)

    @staticmethod
    def load(file_name):
        with open(file_name, "rb") as f:
            return pickle.load(f)

## 載入模型

In [ ]:
#load model
# ap = AutoPreprocess.load("/content/drive/MyDrive/preprocess_RFC.bin")
# f =  open("/content/drive/MyDrive/model_RFC.bin", "rb")
# model =  pickle.load(f)

In [ ]:
import joblib
model = joblib.load("/content/drive/MyDrive/model_RFC.bin")
preprocessor = joblib.load("/content/drive/MyDrive/preprocess_RFC.bin")


In [ ]:
data.head(10)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
5,5,15771669,Genovese,588,Germany,Male,36.0,4,131778.58,1,1.0,0.0,136024.31,1
6,6,15692819,Ch'ang,593,France,Female,30.0,8,144772.69,1,1.0,0.0,29792.11,0
7,7,15669611,Chukwuebuka,678,Spain,Male,37.0,1,138476.41,1,1.0,0.0,106851.60,0
8,8,15691707,Manna,676,France,Male,43.0,4,0.00,2,1.0,0.0,142917.13,0
9,9,15591721,Cattaneo,583,Germany,Male,40.0,4,81274.33,1,1.0,1.0,170843.07,0


## 介面設定

In [ ]:
data.columns

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
data["CustomerId"].value_counts()

,count
CustomerId,
15682355,121
15570194,99
15585835,98
15595588,91
15648067,90
...,...
15744577,1
15596817,1
15731021,1


In [ ]:
import pandas as pd

# 模擬已知模型格式
class DummyModel:
    def predict_proba(self, X):
        return np.array([[0.3, 0.7]])
model = DummyModel()

# 模擬前處理器
class DummyPreprocessor:
    def transform(self, X):
        return np.array([[0.5] * 10])
ap = DummyPreprocessor()

# data = pd.read_csv('/content/drive/MyDrive/train.csv')
df = data.copy()

VALID_CUSTOMER_IDS = df["CustomerId"].unique().tolist()
# 計算 Age、CreditScore、Balance 的眾數
mode_age = df["Age"].mode()[0]
mode_score = df["CreditScore"].mode()[0]
mode_balance = df["Balance"].mode()[0]

mode_age, mode_score, mode_balance

(np.float64(37.0), np.int64(850), np.float64(0.0))

In [ ]:
VALID_CUSTOMER_IDS = {15565701,15565706,15565714,15565759,15565779,15565796,15565806,15565807,15565878,15565879,15565891,15565996,15566028,15566030,15566091,15566111,15566139,15566156,15566187,15566211,15566251,15566253,15566269,15566289,15566292,15566295,15566312,15566372,15566378,15566380,15566389,15566467,15566491,15566494,15566495,15566531,15566543,15566563,15566594,15566633,15566660,15566689,15566708,15566735,15566740,15566787,15566790,15566843,15566891,15566892,15566894,15566896,15566958,15566988,15567013,15567063,15567078,15567114,15567147,15567246,15567328,15567333,15567335,15567339,15567367,15567383,15567391,15567398,15567399,15567422,15567431,15567437,15567442,15567446,15567452,15567486,15567506,15567557,15567630,15567725,15567729,15567778,15567802,15567806,15567807,15567832,15567834,15567839,15567843,15567845,15567855,15567860,15567862,15567893,15567897,15567919,15567960,15567980,15567984,15567991,15567993,15567996,15567998,15568006,15568025,15568032,15568044,15568046,15568088,15568104,15568106,15568112,15568114,15568120,15568146,15568162,15568163,15568164,15568238,15568240,15568326,15568328,15568359,15568360,15568367,15568387,15568429,15568449,15568469,15568487,15568495,15568498,15568506,15568519,15568562,15568573,15568574,15568595,15568682,15568742,15568748,15568819,15568834,15568849,15568869,15568876,15568878,15568885,15568904,15568912,15568915,15568953,15568956,15568963,15568982,15569050,15569078,15569079,15569095,15569098,15569120,15569178,15569187,15569209,15569222,15569247,15569248,15569249,15569264,15569266,15569274,15569364,15569394,15569410,15569423,15569430,15569432,15569438,15569446,15569451,15569452,15569487,15569503,15569526,15569571,15569572,15569586,15569590,15569595,15569624,15569626,15569627,15569635,15569641,15569654,15569657,15569666,15569670,15569678,15569682,15569706,15569759,15569783,15569807,15569867,15569878,15569892,15569917,15569945,15569976,15570002,15570004,15570018,15570030,15570043,15570051,15570060,15570067,15570069,15570073,15570076,15570078,15570080,15570083,15570086,15570087,15570090,15570091,15570092,15570097,15570115,15570134,15570141,15570142,15570145,15570146,15570147,15570182,15570185,15570190,15570192,15570194,15570205,15570266,15570268,15570271,15570289,15570299,15570309,15570326,15570349,15570360,15570374,15570379,15570414,15570417,15570418,15570485,15570508,15570533,15570601,15570629,15570631,15570681,15570769,15570835,15570855,15570857,15570858,15570859,15570895,15570908,15570912,15570914,15570919,15570931,15570932,15570943,15570947,15570970,15570972,15570974,15570988,15570990,15571002,15571053,15571059,15571063,15571065,15571081,15571086,15571121,15571135,15571139,15571144,15571148,15571193,15571201,15571221,15571244,15571251,15571254,15571261,15571269,15571277,15571280,15571281,15571282,15571284,15571286,15571287,15571289,15571302,15571303,15571305,15571415,15571417,15571430,15571433,15571450,15571465,15571476,15571480,15571489,15571519,15571525,15571529,15571530,15571542,15571543,15571549,15571550,15571551,15571552,15571555,15571569,15571571,15571573,15571574,15571579,15571581,15571586,15571589,15571591,15571604,15571607,15571629,15571638,15571646,15571654,15571655,15571656,15571657,15571666,15571668,15571674,15571686,15571689,15571692,15571742,15571747,15571748,15571749,15571751,15571752,15571756,15571766,15571769,15571771,15571778,15571816,15571817,15571818,15571834,15571835,15571840,15571843,15571844,15571848,15571867,15571869,15571870,15571871,15571873,15571883,15571912,15571914,15571915,15571917,15571926,15571927,15571928,15571930,15571940,15571941,15571943,15571944,15571948,15571952,15571955,15571958,15571973,15571995,15572021,15572022,15572027,15572028,15572030,15572033,15572038,15572040,15572048,15572050,15572051,15572058,15572071,15572072,15572073,15572082,15572093,15572098,15572114,15572145,15572158,15572174,15572182,15572216,15572222,15572235,15572265,15572274,15572278,15572283,15572286,15572289,15572291,15572293,15572294,15572328,15572351,15572354,15572356,15572359,15572360,15572361,15572363,15572365,15572366,15572374,15572379,15572390,15572394,15572398,15572406,15572408,15572412,15572414,15572415,15572416,15572421,15572428,15572429,15572430,15572431,15572435,15572446,15572454,15572456,15572460,15572461,15572466,15572467,15572478,15572498,15572515,15572518,15572547,15572574,15572576,15572601,15572626,15572628,15572631,15572635,15572656,15572657,15572677,15572683,15572701,15572705,15572706,15572708,15572726,15572727,15572728,15572729,15572735,15572736,15572738,15572744,15572748,15572751,15572756,15572758,15572761,15572762,15572766,15572767,15572769,15572773,15572776,15572777,15572778,15572779,15572781,15572792,15572794,15572796,15572797,15572798,15572801,15572835,15572847,15572901,15572906,15572948,15572955,15572956,15572977,15573026,15573045,15573046,15573055,15573059,15573073,15573074,15573076,15573077,15573086,15573112,15573152,15573171,15573242,15573278,15573280,15573284,15573309,15573318,15573319,15573322,15573330,15573345,15573347,15573348,15573359,15573365,15573452,15573454,15573456,15573520,15573521,15573599,15573625,15573628,15573629,15573665,15573682,15573722,15573723,15573725,15573730,15573741,15573747,15573748,15573769,15573773,15573776,15573778,15573788,15573792,15573794,15573795,15573798,15573825,15573826,15573851,15573852,15573854,15573856,15573858,15573860,15573865,15573868,15573869,15573871,15573876,15573877,15573881,15573888,15573893,15573926,15573928,15573942,15573949,15573959,15573967,15573969,15573971,15573972,15573976,15573978,15574004,15574012,15574038,15574067,15574068,15574071,15574072,15574073,15574091,15574115,15574119,15574122,15574137,15574142,15574167,15574169,15574192,15574206,15574213,15574237,15574283,15574296,15574305,15574324,15574331,15574352,15574369,15574372,15574480,15574514,15574523,15574526,15574528,15574543,15574544,15574551,15574554,15574556,15574558,15574576,15574577,15574578,15574582,15574583,15574584,15574592,15574608,15574692,15574743,15574748,15574760,15574761,15574765,15574776,15574778,15574782,15574783,15574787,15574790,15574792,15574794,15574795,15574796,15574828,15574830,15574842,15574847,15574848,15574849,15574851,15574853,15574858,15574859,15574860,15574861,15574865,15574866,15574867,15574868,15574872,15574876,15574877,15574878,15574879,15574880,15574882,15574886,15574892,15574895,15574896,15574898,15574910,15574916,15574917,15574928,15574943,15574947,15574951,15575002,15575024,15575025,15575029,15575060,15575067,15575079,15575119,15575126,15575146,15575147,15575185,15575214,15575243,15575247,15575252,15575335,15575339,15575360,15575391,15575399,15575409,15575410,15575417,15575428,15575430,15575438,15575445,15575469,15575478,15575486,15575490,15575492,15575498,15575510,15575581,15575605,15575623,15575676,15575679,15575691,15575694,15575748,15575759,15575858,15575883,15575935,15575945,15575955,15576000,15576022,15576024,15576027,15576029,15576039,15576044,15576045,15576055,15576061,15576065,15576069,15576077,15576078,15576079,15576084,15576085,15576086,15576091,15576094,15576095,15576096,15576111,15576116,15576124,15576126,15576131,15576133,15576156,15576162,15576185,15576196,15576200,15576216,15576219,15576256,15576263,15576269,15576296,15576304,15576313,15576352,15576360,15576368,15576417,15576425,15576469,15576492,15576494,15576515,15576516,15576517,15576520,15576522,15576526,15576545,15576548,15576550,15576602,15576615,15576623,15576624,15576635,15576641,15576644,15576660,15576671,15576674,15576676,15576680,15576682,15576683,15576685,15576687,15576696,15576706,15576713,15576714,15576717,15576720,15576723,15576724,15576725,15576726,15576729,15576743,15576745,15576760,15576761,15576765,15576770,15576774,15576779,15576780,15576794,15576822,15576824,15576826,15576845,15576850,15576876,15576895,15576914,15576915,15576917,15576921,15576928,15576929,15576935,15576936,15576939,15576945,15576946,15576951,15576967,15576990,15576992,15577018,15577025,15577064,15577065,15577078,15577079,15577081,15577103,15577104,15577107,15577146,15577170,15577178,15577274,15577288,15577313,15577358,15577359,15577398,15577402,15577423,15577478,15577510,15577512,15577514,15577515,15577517,15577522,15577523,15577535,15577538,15577551,15577565,15577576,15577583,15577586,15577587,15577590,15577591,15577593,15577594,15577596,15577597,15577598,15577599,15577657,15577658,15577683,15577695,15577700,15577751,15577771,15577779,15577806,15577831,15577905,15577913,15577914,15577970,15577978,15577985,15577988,15577999,15578006,15578045,15578047,15578048,15578049,15578065,15578071,15578073,15578075,15578088,15578096,15578098,15578141,15578156,15578186,15578211,15578224,15578238,15578251,15578280,15578297,15578307,15578369,15578374,15578375,15578435,15578462,15578475,15578515,15578528,15578543,15578546,15578560,15578580,15578603,15578671,15578715,15578718,15578722,15578725,15578726,15578728,15578733,15578736,15578737,15578738,15578739,15578744,15578745,15578746,15578747,15578760,15578761,15578769,15578775,15578783,15578787,15578788,15578798,15578799,15578809,15578818,15578825,15578835,15578838,15578845,15578854,15578855,15578858,15578865,15578866,15578868,15578876,15578878,15578882,15578883,15578884,15578885,15578886,15578888,15578892,15578893,15578896,15578905,15578907,15578908,15578914,15578915,15578916,15578918,15578922,15578928,15578930,15578933,15578935,15578943,15578944,15578947,15578948,15578955,15578958,15578977,15578980,15579040,15579062,15579109,15579112,15579130,15579131,15579133,15579166,15579174,15579183,15579189,15579207,15579208,15579211,15579212,15579223,15579230,15579262,15579266,15579287,15579334,15579345,15579387,15579428,15579433,15579435,15579514,15579515,15579516,15579522,15579523,15579526,15579544,15579546,15579548,15579550,15579557,15579559,15579560,15579561,15579566,15579583,15579611,15579614,15579616,15579617,15579622,15579639,15579647,15579660,15579706,15579714,15579715,15579777,15579781,15579783,15579787,15579808,15579809,15579820,15579825,15579826,15579863,15579866,15579869,15579876,15579891,15579892,15579894,15579913,15579914,15579915,15579938,15579961,15579968,15579969,15579987,15579994,15579996,15579997,15580043,15580068,15580134,15580141,15580142,15580144,15580145,15580146,15580147,15580148,15580149,15580166,15580177,15580194,15580203,15580213,15580227,15580249,15580252,15580269,15580362,15580366,15580462,15580487,15580492,15580493,15580560,15580579,15580602,15580623,15580629,15580682,15580683,15580684,15580701,15580719,15580805,15580845,15580852,15580869,15580870,15580872,15580912,15580914,15580918,15580935,15580953,15580956,15580988,15581036,15581089,15581102,15581197,15581198,15581218,15581220,15581226,15581229,15581230,15581235,15581236,15581237,15581240,15581248,15581249,15581251,15581252,15581253,15581254,15581257,15581261,15581272,15581274,15581276,15581277,15581279,15581280,15581282,15581284,15581286,15581289,15581290,15581292,15581295,15581299,15581316,15581320,15581323,15581325,15581326,15581332,15581333,15581334,15581340,15581348,15581364,15581370,15581379,15581388,15581409,15581440,15581450,15581457,15581474,15581479,15581480,15581505,15581519,15581520,15581524,15581525,15581526,15581528,15581529,15581530,15581533,15581536,15581539,15581548,15581549,15581550,15581551,15581552,15581553,15581554,15581579,15581589,15581606,15581610,15581620,15581626,15581628,15581629,15581630,15581633,15581634,15581641,15581642,15581647,15581648,15581652,15581654,15581661,15581674,15581677,15581689,15581690,15581699,15581700,15581702,15581703,15581733,15581736,15581740,15581748,15581757,15581768,15581776,15581802,15581811,15581822,15581840,15581870,15581871,15581872,15581884,15581892,15581893,15581940,15581968,15582016,15582033,15582066,15582073,15582090,15582098,15582116,15582129,15582149,15582154,15582180,15582229,15582246,15582259,15582275,15582276,15582291,15582310,15582321,15582325,15582335,15582336,15582349,15582351,15582355,15582358,15582361,15582365,15582373,15582381,15582390,15582392,15582404,15582454,15582456,15582474,15582492,15582501,15582511,15582515,15582517,15582518,15582519,15582521,15582522,15582524,15582525,15582528,15582530,15582541,15582548,15582576,15582578,15582591,15582616,15582704,15582705,15582707,15582709,15582711,15582712,15582714,15582715,15582717,15582718,15582719,15582728,15582741,15582749,15582751,15582756,15582757,15582761,15582762,15582763,15582767,15582777,15582796,15582797,15582835,15582841,15582842,15582847,15582848,15582849,15582851,15582857,15582861,15582865,15582867,15582892,15582897,15582903,15582907,15582909,15582910,15582911,15582912,15582914,15582917,15582919,15582924,15582925,15582926,15582933,15582934,15582937,15582940,15582941,15582942,15582944,15582945,15582948,15582949,15582950,15582951,15582954,15582955,15582956,15582957,15582960,15582964,15582967,15582977,15582979,15582993,15582996,15583024,15583025,15583026,15583028,15583029,15583033,15583040,15583041,15583046,15583048,15583049,15583051,15583053,15583063,15583065,15583067,15583071,15583073,15583076,15583079,15583081,15583088,15583090,15583092,15583093,15583096,15583097,15583137,15583212,15583221,15583278,15583303,15583312,15583322,15583325,15583326,15583332,15583339,15583341,15583343,15583348,15583349,15583352,15583353,15583354,15583359,15583362,15583364,15583368,15583371,15583390,15583392,15583393,15583394,15583395,15583397,15583398,15583399,15583456,15583460,15583462,15583467,15583480,15583481,15583496,15583499,15583515,15583524,15583528,15583529,15583536,15583540,15583548,15583549,15583550,15583551,15583552,15583553,15583555,15583557,15583562,15583563,15583566,15583568,15583572,15583573,15583574,15583576,15583577,15583578,15583580,15583581,15583584,15583590,15583591,15583592,15583593,15583594,15583595,15583596,15583597,15583599,15583601,15583610,15583628,15583629,15583657,15583662,15583668,15583671,15583673,15583674,15583679,15583680,15583681,15583683,15583690,15583691,15583692,15583695,15583701,15583710,15583713,15583715,15583718,15583720,15583724,15583725,15583726,15583730,15583735,15583736,15583738,15583739,15583741,15583748,15583749,15583750,15583755,15583760,15583764,15583768,15583776,15583792,15583825,15583837,15583840,15583843,15583849,15583850,15583851,15583852,15583853,15583854,15583856,15583857,15583861,15583863,15583865,15583871,15583876,15583877,15583880,15583881,15583883,15583897,15583949,15583955,15583976,15584035,15584037,15584042,15584068,15584090,15584091,15584092,15584099,15584113,15584114,15584156,15584190,15584229,15584271,15584288,15584315,15584318,15584320,15584324,15584327,15584338,15584363,15584364,15584452,15584453,15584461,15584462,15584477,15584505,15584515,15584516,15584517,15584518,15584519,15584520,15584523,15584524,15584526,15584531,15584532,15584533,15584535,15584536,15584541,15584545,15584548,15584551,15584553,15584555,15584556,15584560,15584562,15584564,15584566,15584574,15584576,15584577,15584580,15584584,15584588,15584591,15584592,15584594,15584704,15584749,15584750,15584764,15584765,15584766,15584776,15584780,15584785,15584788,15584792,15584795,15584799,15584812,15584828,15584829,15584839,15584848,15584849,15584852,15584858,15584861,15584862,15584866,15584868,15584882,15584892,15584894,15584895,15584897,15584899,15584910,15584925,15584927,15584928,15584929,15584930,15584933,15584958,15584967,15584980,15585031,15585034,15585036,15585037,15585039,15585041,15585047,15585066,15585067,15585068,15585071,15585100,15585106,15585133,15585171,15585192,15585198,15585215,15585221,15585222,15585241,15585246,15585249,15585255,15585256,15585280,15585282,15585284,15585287,15585290,15585300,15585323,15585324,15585326,15585327,15585328,15585340,15585350,15585352,15585362,15585367,15585379,15585380,15585383,15585388,15585427,15585451,15585455,15585466,15585490,15585519,15585526,15585533,15585548,15585576,15585580,15585582,15585588,15585590,15585592,15585594,15585595,15585597,15585598,15585599,15585666,15585735,15585738,15585748,15585766,15585768,15585777,15585799,15585815,15585818,15585819,15585822,15585823,15585824,15585826,15585828,15585829,15585832,15585833,15585835,15585836,15585837,15585839,15585840,15585842,15585843,15585849,15585853,15585855,15585861,15585862,15585865,15585866,15585867,15585868,15585869,15585872,15585874,15585877,15585879,15585885,15585888,15585891,15585892,15585893,15585895,15585901,15585905,15585906,15585907,15585908,15585915,15585920,15585928,15585929,15585935,15585936,15585945,15585947,15585955,15585956,15585961,15585978,15585985,15586021,15586022,15586023,15586026,15586028,15586029,15586035,15586036,15586037,15586039,15586042,15586045,15586061,15586068,15586069,15586085,15586091,15586133,15586174,15586183,15586236,15586243,15586264,15586300,15586310,15586425,15586467,15586475,15586479,15586487,15586504,15586511,15586515,15586517,15586518,15586521,15586522,15586523,15586524,15586525,15586526,15586535,15586536,15586539,15586557,15586566,15586569,15586570,15586585,15586613,15586615,15586622,15586623,15586629,15586639,15586645,15586674,15586675,15586676,15586679,15586680,15586688,15586699,15586713,15586715,15586733,15586752,15586757,15586780,15586800,15586810,15586811,15586821,15586824,15586835,15586840,15586844,15586848,15586849,15586852,15586859,15586860,15586869,15586870,15586872,15586876,15586880,15586896,15586900,15586914,15586915,15586922,15586931,15586935,15586936,15586942,15586945,15586959,15586963,15586966,15586969,15586970,15586974,15586975,15586976,15586977,15586979,15586980,15586993,15586996,15587013,15587037,15587038,15587039,15587063,15587066,15587067,15587069,15587090,15587097,15587133,15587174,15587177,15587210,15587233,15587240,15587266,15587297,15587299,15587311,15587419,15587421,15587443,15587507,15587521,15587531,15587534,15587535,15587536,15587539,15587540,15587541,15587543,15587553,15587557,15587562,15587565,15587566,15587567,15587571,15587572,15587573,15587574,15587576,15587579,15587580,15587581,15587582,15587584,15587585,15587586,15587587,15587590,15587591,15587592,15587593,15587595,15587596,15587597,15587598,15587610,15587611,15587623,15587627,15587640,15587647,15587655,15587690,15587710,15587712,15587719,15587735,15587824,15587835,15587837,15587842,15587880,15587900,15587901,15587907,15587908,15587909,15587912,15587913,15587914,15587929,15587948,15587959,15587963,15587974,15587978,15587980,15588019,15588080,15588090,15588123,15588137,15588186,15588197,15588219,15588235,15588248,15588341,15588350,15588372,15588406,15588419,15588446,15588449,15588450,15588456,15588461,15588480,15588510,15588513,15588515,15588517,15588518,15588521,15588526,15588532,15588533,15588535,15588537,15588540,15588541,15588546,15588550,15588560,15588566,15588580,15588587,15588614,15588621,15588622,15588623,15588670,15588695,15588701,15588833,15588835,15588838,15588839,15588840,15588851,15588854,15588855,15588859,15588865,15588875,15588876,15588880,15588881,15588892,15588903,15588904,15588906,15588907,15588908,15588912,15588914,15588915,15588916,15588917,15588918,15588923,15588928,15588929,15588930,15588933,15588935,15588938,15588939,15588943,15588944,15588945,15588948,15588955,15588958,15588959,15588960,15588964,15588967,15588970,15588974,15588975,15588977,15588986,15588988,15589013,15589017,15589018,15589019,15589030,15589076,15589147,15589195,15589200,15589204,15589207,15589210,15589211,15589212,15589221,15589224,15589230,15589248,15589257,15589268,15589288,15589294,15589296,15589301,15589312,15589319,15589320,15589322,15589323,15589324,15589328,15589329,15589333,15589340,15589344,15589348,15589351,15589358,15589361,15589364,15589375,15589380,15589401,15589410,15589412,15589414,15589415,15589417,15589420,15589421,15589423,15589425,15589426,15589428,15589429,15589430,15589431,15589432,15589433,15589435,15589436,15589437,15589438,15589449,15589450,15589460,15589466,15589475,15589480,15589488,15589493,15589496,15589523,15589537,15589541,15589543,15589544,15589548,15589557,15589561,15589562,15589563,15589572,15589587,15589589,15589614,15589619,15589643,15589660,15589666,15589698,15589714,15589715,15589717,15589725,15589733,15589739,15589741,15589743,15589754,15589775,15589793,15589805,15589862,15589865,15589869,15589872,15589881,15589897,15589899,15589913,15589915,15589919,15589924,15589925,15589928,15589935,15589941,15589947,15589948,15589949,15589951,15589953,15589955,15589962,15589968,15589969,15589974,15589975,15589977,15589992,15590011,15590078,15590086,15590091,15590093,15590094,15590097,15590113,15590127,15590134,15590146,15590148,15590167,15590177,15590188,15590194,15590199,15590201,15590213,15590228,15590234,15590241,15590268,15590320,15590327,15590330,15590344,15590349,15590434,15590476,15590606,15590623,15590637,15590676,15590683,15590713,15590720,15590730,15590810,15590820,15590822,15590823,15590828,15590838,15590846,15590866,15590876,15590888,15590898,15590928,15590930,15590935,15590966,15590993,15591031,15591035,15591036,15591037,15591038,15591039,15591047,15591086,15591087,15591091,15591100,15591107,15591114,15591119,15591123,15591130,15591135,15591140,15591150,15591159,15591167,15591169,15591170,15591174,15591179,15591217,15591240,15591244,15591248,15591250,15591251,15591254,15591256,15591257,15591258,15591264,15591267,15591269,15591270,15591279,15591281,15591286,15591287,15591288,15591289,15591295,15591297,15591299,15591300,15591326,15591333,15591340,15591344,15591345,15591352,15591356,15591357,15591359,15591360,15591386,15591387,15591388,15591399,15591411,15591414,15591425,15591428,15591429,15591430,15591433,15591450,15591457,15591476,15591477,15591489,15591509,15591512,15591513,15591525,15591526,15591528,15591529,15591530,15591546,15591550,15591551,15591552,15591557,15591576,15591577,15591579,15591586,15591589,15591598,15591599,15591604,15591605,15591607,15591608,15591625,15591626,15591627,15591628,15591629,15591640,15591644,15591647,15591657,15591674,15591688,15591689,15591698,15591710,15591711,15591714,15591715,15591716,15591717,15591721,15591722,15591723,15591724,15591727,15591733,15591736,15591740,15591743,15591744,15591747,15591766,15591777,15591811,15591815,15591821,15591822,15591823,15591840,15591871,15591911,15591912,15591914,15591915,15591917,15591922,15591928,15591932,15591942,15591943,15591947,15591948,15591969,15591980,15591985,15591986,15591995,15592000,15592025,15592028,15592029,15592033,15592066,15592069,15592073,15592077,15592078,15592083,15592084,15592089,15592091,15592093,15592094,15592097,15592098,15592104,15592116,15592123,15592194,15592197,15592222,15592225,15592229,15592230,15592232,15592235,15592300,15592301,15592312,15592322,15592326,15592328,15592330,15592344,15592345,15592348,15592350,15592351,15592354,15592356,15592359,15592361,15592378,15592379,15592382,15592386,15592387,15592388,15592389,15592390,15592394,15592398,15592399,15592404,15592406,15592412,15592414,15592436,15592443,15592451,15592454,15592456,15592461,15592467,15592476,15592478,15592479,15592482,15592494,15592497,15592501,15592515,15592522,15592530,15592531,15592535,15592536,15592551,15592560,15592566,15592567,15592570,15592571,15592575,15592577,15592578,15592579,15592581,15592583,15592585,15592591,15592599,15592615,15592625,15592631,15592645,15592646,15592647,15592650,15592655,15592656,15592657,15592661,15592662,15592671,15592685,15592701,15592704,15592705,15592707,15592711,15592712,15592714,15592715,15592716,15592717,15592721,15592726,15592727,15592728,15592731,15592735,15592736,15592740,15592741,15592744,15592745,15592746,15592750,15592751,15592757,15592761,15592762,15592765,15592771,15592773,15592777,15592779,15592781,15592782,15592787,15592797,15592799,15592801,15592810,15592811,15592814,15592815,15592816,15592822,15592824,15592826,15592836,15592839,15592842,15592845,15592846,15592851,15592853,15592854,15592856,15592865,15592877,15592882,15592894,15592901,15592911,15592912,15592913,15592914,15592925,15592926,15592928,15592929,15592935,15592936,15592937,15592945,15592950,15592951,15592956,15592966,15592976,15592977,15592979,15592983,15592984,15592986,15592991,15592994,15592995,15592996,15592999,15593014,15593094,15593096,15593128,15593241,15593250,15593280,15593283,15593295,15593315,15593319,15593323,15593324,15593331,15593339,15593342,15593344,15593345,15593347,15593349,15593359,15593365,15593370,15593377,15593379,15593392,15593394,15593395,15593399,15593412,15593424,15593425,15593426,15593436,15593450,15593454,15593462,15593468,15593470,15593476,15593478,15593479,15593481,15593482,15593491,15593495,15593496,15593497,15593499,15593501,15593510,15593511,15593536,15593537,15593546,15593547,15593550,15593555,15593585,15593594,15593595,15593601,15593622,15593629,15593630,15593636,15593637,15593639,15593650,15593656,15593662,15593671,15593672,15593674,15593679,15593681,15593682,15593683,15593684,15593691,15593692,15593693,15593694,15593695,15593699,15593710,15593712,15593714,15593715,15593718,15593724,15593725,15593726,15593730,15593731,15593736,15593737,15593739,15593750,15593760,15593761,15593762,15593764,15593765,15593769,15593770,15593772,15593773,15593776,15593778,15593779,15593782,15593783,15593784,15593792,15593794,15593795,15593832,15593834,15593838,15593842,15593850,15593851,15593853,15593854,15593856,15593877,15593882,15593883,15593886,15593893,15593895,15593896,15593897,15593900,15593903,15593907,15593914,15593917,15593930,15593931,15593939,15593942,15593943,15593944,15593945,15593946,15593947,15593948,15593949,15593952,15593954,15593956,15593959,15593966,15593969,15593971,15593973,15593974,15593976,15593978,15593979,15593983,15594037,15594038,15594039,15594041,15594042,15594044,15594084,15594091,15594092,15594133,15594270,15594305,15594340,15594349,15594391,15594404,15594408,15594409,15594436,15594446,15594450,15594456,15594514,15594515,15594528,15594530,15594536,15594544,15594551,15594554,15594556,15594557,15594558,15594566,15594576,15594577,15594579,15594580,15594581,15594583,15594584,15594586,15594594,15594598,15594612,15594651,15594685,15594717,15594720,15594761,15594762,15594783,15594786,15594792,15594796,15594801,15594812,15594815,15594816,15594818,15594822,15594827,15594828,15594849,15594851,15594856,15594857,15594858,15594862,15594864,15594865,15594878,15594879,15594882,15594895,15594896,15594897,15594898,15594899,15594901,15594902,15594912,15594915,15594917,15594920,15594923,15594926,15594927,15594928,15594929,15594936,15594944,15594947,15594950,15594967,15595010,15595028,15595029,15595034,15595036,15595037,15595039,15595047,15595067,15595070,15595071,15595135,15595136,15595153,15595156,15595158,15595160,15595162,15595221,15595228,15595282,15595303,15595324,15595350,15595388,15595391,15595426,15595427,15595440,15595447,15595528,15595533,15595537,15595547,15595557,15595563,15595576,15595581,15595583,15595587,15595588,15595591,15595592,15595594,15595596,15595598,15595599,15595609,15595640,15595657,15595713,15595714,15595715,15595720,15595725,15595726,15595727,15595728,15595731,15595733,15595734,15595736,15595737,15595738,15595748,15595765,15595766,15595767,15595815,15595824,15595826,15595827,15595831,15595832,15595834,15595836,15595837,15595841,15595842,15595843,15595854,15595857,15595858,15595863,15595865,15595866,15595872,15595874,15595877,15595878,15595879,15595881,15595883,15595884,15595917,15595928,15595936,15595937,15595960,15595961,15595984,15595991,15596011,15596013,15596021,15596022,15596024,15596028,15596035,15596036,15596039,15596044,15596047,15596060,15596061,15596069,15596074,15596078,15596084,15596088,15596094,15596096,15596097,15596136,15596161,15596165,15596175,15596181,15596228,15596303,15596339,15596379,15596397,15596405,15596414,15596422,15596436,15596455,15596490,15596493,15596494,15596495,15596510,15596511,15596517,15596522,15596523,15596550,15596552,15596563,15596575,15596613,15596621,15596623,15596624,15596629,15596647,15596649,15596671,15596674,15596700,15596710,15596711,15596713,15596717,15596761,15596763,15596787,15596790,15596797,15596800,15596808,15596810,15596811,15596817,15596821,15596824,15596863,15596870,15596877,15596879,15596900,15596901,15596909,15596910,15596913,15596914,15596916,15596917,15596921,15596924,15596929,15596936,15596937,15596939,15596941,15596942,15596944,15596950,15596954,15596961,15596962,15596968,15596969,15596971,15596972,15596973,15596974,15596976,15596979,15596981,15596983,15596984,15596989,15596992,15596996,15597033,15597109,15597131,15597239,15597265,15597309,15597310,15597323,15597385,15597394,15597467,15597487,15597510,15597536,15597562,15597591,15597594,15597596,15597597,15597602,15597610,15597709,15597735,15597739,15597748,15597885,15597896,15597900,15597901,15597905,15597906,15597907,15597908,15597909,15597910,15597912,15597913,15597914,15597915,15597925,15597930,15597933,15597934,15597937,15597939,15597940,15597944,15597945,15597946,15597948,15597949,15597951,15597954,15597956,15597957,15597959,15597962,15597967,15597968,15597973,15597974,15597975,15597978,15597979,15597983,15597984,15597987,15597988,15597989,15598030,15598035,15598044,15598045,15598046,15598047,15598070,15598073,15598086,15598088,15598091,15598096,15598097,15598098,15598157,15598161,15598162,15598175,15598259,15598266,15598275,15598331,15598338,15598341,15598485,15598493,15598510,15598521,15598526,15598528,15598535,15598536,15598560,15598574,15598577,15598580,15598586,15598614,15598695,15598700,15598744,15598751,15598782,15598802,15598835,15598836,15598838,15598839,15598840,15598844,15598846,15598847,15598848,15598851,15598876,15598877,15598879,15598882,15598883,15598884,15598885,15598886,15598891,15598892,15598893,15598894,15598896,15598897,15598940,15598948,15598982,15599004,15599024,15599074,15599076,15599078,15599081,15599084,15599090,15599095,15599120,15599124,15599126,15599131,15599133,15599152,15599182,15599186,15599195,15599211,15599247,15599251,15599268,15599272,15599289,15599301,15599329,15599386,15599403,15599410,15599433,15599440,15599452,15599533,15599535,15599552,15599591,15599660,15599722,15599792,15599829,15599860,15599869,15599956,15599976,15600027,15600086,15600100,15600101,15600103,15600106,15600110,15600173,15600174,15600200,15600258,15600280,15600300,15600337,15600351,15600357,15600366,15600379,15600392,15600399,15600440,15600462,15600478,15600510,15600547,15600554,15600575,15600583,15600626,15600650,15600651,15600688,15600692,15600700,15600708,15600710,15600720,15600724,15600734,15600739,15600746,15600749,15600751,15600781,15600792,15600793,15600803,15600809,15600812,15600813,15600814,15600821,15600826,15600829,15600832,15600834,15600839,15600872,15600879,15600882,15600911,15600912,15600926,15600934,15600974,15600975,15600991,15600992,15600993,15600997,15601004,15601008,15601012,15601019,15601026,15601030,15601045,15601071,15601116,15601169,15601172,15601176,15601184,15601187,15601188,15601200,15601263,15601274,15601324,15601351,15601354,15601383,15601387,15601392,15601410,15601417,15601473,15601503,15601550,15601569,15601589,15601594,15601627,15601659,15601688,15601719,15601787,15601793,15601796,15601797,15601809,15601811,15601848,15601857,15601873,15601892,15601977,15602010,15602030,15602034,15602035,15602043,15602044,15602063,15602073,15602081,15602084,15602088,15602089,15602098,15602124,15602280,15602281,15602282,15602312,15602314,15602324,15602328,15602344,15602354,15602365,15602372,15602373,15602374,15602378,15602379,15602403,15602404,15602405,15602406,15602409,15602412,15602451,15602454,15602456,15602467,15602478,15602479,15602486,15602497,15602498,15602500,15602526,15602541,15602547,15602551,15602554,15602572,15602573,15602574,15602578,15602591,15602708,15602723,15602731,15602735,15602741,15602751,15602756,15602762,15602797,15602801,15602811,15602812,15602814,15602815,15602826,15602831,15602834,15602836,15602841,15602844,15602845,15602846,15602851,15602853,15602854,15602906,15602909,15602913,15602914,15602918,15602928,15602929,15602934,15602937,15602944,15602945,15602949,15602954,15602966,15602967,15602968,15602977,15602979,15602982,15602984,15602986,15602989,15602991,15602994,15602996,15602997,15602998,15602999,15603028,15603034,15603035,15603036,15603037,15603046,15603065,15603068,15603073,15603078,15603082,15603085,15603086,15603088,15603095,15603096,15603097,15603098,15603111,15603134,15603135,15603156,15603170,15603216,15603221,15603247,15603318,15603319,15603323,15603324,15603326,15603328,15603329,15603331,15603344,15603353,15603354,15603372,15603378,15603381,15603382,15603394,15603411,15603413,15603426,15603436,15603437,15603454,15603461,15603463,15603465,15603475,15603478,15603479,15603481,15603482,15603483,15603489,15603525,15603543,15603544,15603550,15603554,15603565,15603582,15603594,15603597,15603631,15603643,15603661,15603665,15603674,15603681,15603683,15603684,15603734,15603739,15603741,15603743,15603744,15603747,15603748,15603749,15603750,15603754,15603761,15603763,15603764,15603765,15603772,15603783,15603784,15603793,15603794,15603826,15603830,15603831,15603840,15603841,15603842,15603843,15603844,15603846,15603849,15603850,15603851,15603854,15603856,15603865,15603870,15603874,15603877,15603880,15603883,15603913,15603914,15603917,15603925,15603926,15603931,15603942,15603949,15603954,15603968,15603976,15603988,15604020,15604035,15604040,15604043,15604044,15604046,15604072,15604073,15604082,15604084,15604093,15604119,15604130,15604138,15604146,15604158,15604217,15604294,15604295,15604304,15604314,15604344,15604345,15604348,15604355,15604444,15604472,15604482,15604494,15604497,15604515,15604516,15604518,15604532,15604536,15604541,15604544,15604545,15604546,15604547,15604551,15604556,15604566,15604576,15604577,15604581,15604582,15604588,15604706,15604712,15604732,15604761,15604762,15604782,15604786,15604787,15604792,15604796,15604797,15604804,15604812,15604813,15604815,15604816,15604817,15604818,15604819,15604820,15604827,15604832,15604834,15604851,15604854,15604862,15604882,15604891,15604918,15604928,15604929,15604931,15604934,15604940,15604941,15604963,15605000,15605020,15605021,15605023,15605024,15605029,15605032,15605034,15605035,15605036,15605037,15605040,15605044,15605050,15605059,15605062,15605066,15605067,15605068,15605072,15605077,15605082,15605097,15605113,15605130,15605134,15605215,15605263,15605264,15605275,15605276,15605279,15605284,15605286,15605293,15605324,15605325,15605326,15605327,15605330,15605333,15605339,15605341,15605346,15605356,15605375,15605423,15605424,15605425,15605426,15605427,15605429,15605441,15605444,15605447,15605455,15605461,15605475,15605515,15605516,15605520,15605521,15605524,15605526,15605529,15605531,15605532,15605534,15605536,15605537,15605545,15605575,15605617,15605622,15605627,15605631,15605665,15605666,15605671,15605672,15605673,15605684,15605698,15605718,15605727,15605734,15605737,15605741,15605742,15605743,15605747,15605763,15605769,15605777,15605782,15605791,15605792,15605804,15605818,15605822,15605826,15605827,15605828,15605831,15605835,15605836,15605837,15605840,15605842,15605844,15605855,15605868,15605870,15605872,15605891,15605892,15605914,15605916,15605917,15605918,15605924,15605925,15605926,15605927,15605928,15605930,15605934,15605936,15605937,15605945,15605950,15605965,15605967,15605968,15605969,15605976,15605986,15606003,15606028,15606036,15606042,15606045,15606050,15606061,15606063,15606070,15606071,15606074,15606076,15606085,15606096,15606097,15606115,15606133,15606149,15606158,15606177,15606229,15606232,15606267,15606273,15606310,15606386,15606397,15606413,15606420,15606434,15606435,15606436,15606465,15606472,15606479,15606486,15606490,15606507,15606552,15606554,15606601,15606613,15606615,15606641,15606683,15606712,15606713,15606733,15606744,15606755,15606761,15606787,15606797,15606801,15606811,15606835,15606836,15606837,15606839,15606841,15606842,15606845,15606847,15606849,15606851,15606852,15606855,15606856,15606859,15606861,15606863,15606871,15606877,15606880,15606885,15606887,15606901,15606907,15606913,15606914,15606915,15606936,15606944,15606945,15607032,15607040,15607065,15607085,15607097,15607098,15607112,15607133,15607142,15607170,15607176,15607178,15607230,15607263,15607269,15607275,15607278,15607301,15607310,15607312,15607314,15607329,15607330,15607347,15607381,15607395,15607424,15607485,15607490,15607509,15607526,15607535,15607536,15607537,15607546,15607551,15607554,15607556,15607557,15607560,15607566,15607568,15607576,15607577,15607579,15607591,15607594,15607597,15607598,15607629,15607632,15607633,15607634,15607637,15607690,15607691,15607713,15607728,15607731,15607732,15607734,15607737,15607743,15607745,15607746,15607748,15607753,15607783,15607784,15607786,15607787,15607797,15607827,15607828,15607829,15607837,15607871,15607874,15607877,15607884,15607932,15607936,15607944,15607946,15607968,15607977,15607986,15607987,15607988,15607990,15607993,15607995,15608114,15608166,15608209,15608226,15608230,15608232,15608246,15608328,15608334,15608338,15608341,15608344,15608381,15608387,15608406,15608408,15608528,15608541,15608544,15608588,15608595,15608653,15608688,15608701,15608715,15608726,15608727,15608744,15608745,15608760,15608804,15608813,15608835,15608845,15608846,15608859,15608873,15608877,15608878,15608879,15608881,15608882,15608886,15608894,15608913,15608916,15608917,15608918,15608963,15608965,15608968,15608977,15609011,15609058,15609070,15609083,15609160,15609176,15609187,15609254,15609286,15609299,15609319,15609323,15609324,15609330,15609356,15609458,15609475,15609501,15609510,15609511,15609513,15609515,15609520,15609521,15609522,15609523,15609525,15609526,15609530,15609531,15609537,15609541,15609544,15609545,15609548,15609558,15609562,15609567,15609570,15609618,15609623,15609627,15609634,15609637,15609643,15609648,15609653,15609669,15609676,15609701,15609704,15609723,15609728,15609733,15609735,15609737,15609738,15609741,15609745,15609758,15609804,15609817,15609823,15609824,15609826,15609827,15609828,15609837,15609844,15609872,15609873,15609874,15609897,15609899,15609906,15609913,15609914,15609928,15609949,15609977,15609987,15609991,15609994,15609997,15609998,15610002,15610042,15610056,15610105,15610106,15610135,15610140,15610147,15610148,15610156,15610161,15610165,15610167,15610226,15610227,15610241,15610271,15610295,15610337,15610343,15610355,15610371,15610379,15610383,15610385,15610390,15610397,15610403,15610416,15610419,15610426,15610433,15610446,15610454,15610457,15610553,15610557,15610563,15610594,15610643,15610653,15610686,15610706,15610711,15610727,15610753,15610755,15610765,15610781,15610786,15610801,15610900,15610902,15610903,15610912,15610922,15610930,15610936,15610972,15610978,15611021,15611023,15611024,15611025,15611029,15611031,15611036,15611053,15611056,15611058,15611063,15611088,15611090,15611091,15611097,15611105,15611111,15611127,15611155,15611157,15611168,15611169,15611171,15611181,15611184,15611185,15611186,15611189,15611191,15611205,15611217,15611221,15611223,15611235,15611247,15611248,15611287,15611315,15611318,15611325,15611329,15611331,15611338,15611365,15611371,15611396,15611409,15611455,15611544,15611551,15611571,15611579,15611580,15611594,15611599,15611612,15611635,15611644,15611669,15611682,15611699,15611706,15611729,15611756,15611759,15611767,15611781,15611786,15611794,15611800,15611848,15611905,15611947,15611961,15611973,15611985,15612007,15612030,15612043,15612050,15612064,15612071,15612073,15612084,15612087,15612092,15612093,15612094,15612095,15612102,15612103,15612107,15612108,15612109,15612139,15612140,15612144,15612176,15612184,15612187,15612191,15612193,15612207,15612227,15612338,15612358,15612363,15612418,15612444,15612455,15612465,15612477,15612478,15612483,15612494,15612505,15612524,15612525,15612594,15612615,15612627,15612633,15612667,15612670,15612729,15612769,15612771,15612776,15612777,15612787,15612832,15612893,15612931,15612966,15613007,15613012,15613014,15613017,15613048,15613085,15613097,15613102,15613103,15613132,15613140,15613141,15613143,15613171,15613172,15613180,15613189,15613192,15613197,15613216,15613282,15613283,15613292,15613319,15613326,15613337,15613346,15613352,15613427,15613444,15613463,15613471,15613476,15613503,15613518,15613594,15613623,15613630,15613656,15613667,15613673,15613699,15613712,15613713,15613734,15613749,15613772,15613774,15613786,15613787,15613816,15613844,15613854,15613863,15613880,15613886,15613923,15613945,15613962,15614034,15614049,15614054,15614087,15614103,15614106,15614110,15614116,15614137,15614168,15614170,15614187,15614212,15614213,15614215,15614220,15614230,15614237,15614260,15614282,15614302,15614337,15614355,15614358,15614361,15614365,15614366,15614397,15614420,15614491,15614507,15614590,15614635,15614643,15614677,15614684,15614687,15614693,15614695,15614716,15614740,15614750,15614757,15614772,15614778,15614782,15614787,15614813,15614818,15614823,15614827,15614828,15614834,15614847,15614877,15614878,15614893,15614896,15614923,15614929,15614934,15614936,15614962,15615003,15615004,15615007,15615012,15615016,15615017,15615020,15615024,15615025,15615028,15615029,15615030,15615032,15615036,15615040,15615047,15615048,15615050,15615060,15615064,15615070,15615091,15615095,15615096,15615097,15615116,15615117,15615125,15615126,15615136,15615140,15615141,15615142,15615176,15615177,15615184,15615185,15615207,15615219,15615245,15615247,15615252,15615254,15615296,15615297,15615322,15615330,15615352,15615355,15615430,15615444,15615456,15615457,15615461,15615473,15615477,15615597,15615624,15615670,15615709,15615753,15615756,15615757,15615797,15615820,15615828,15615832,15615896,15615931,15615938,15615951,15615991,15616028,15616088,15616122,15616126,15616134,15616152,15616155,15616168,15616172,15616213,15616220,15616221,15616236,15616240,15616243,15616252,15616270,15616276,15616280,15616330,15616331,15616334,15616365,15616367,15616380,15616431,15616444,15616451,15616454,15616471,15616476,15616491,15616507,15616525,15616529,15616530,15616550,15616555,15616630,15616666,15616676,15616700,15616709,15616745,15616752,15616811,15616833,15616859,15616922,15616929,15616931,15616954,15617042,15617052,15617054,15617062,15617065,15617078,15617079,15617082,15617116,15617134,15617136,15617138,15617156,15617166,15617168,15617191,15617197,15617203,15617206,15617221,15617225,15617226,15617252,15617298,15617301,15617304,15617320,15617330,15617331,15617347,15617348,15617352,15617398,15617401,15617419,15617432,15617434,15617454,15617471,15617476,15617482,15617486,15617498,15617507,15617518,15617538,15617598,15617603,15617608,15617617,15617644,15617647,15617648,15617655,15617657,15617700,15617703,15617705,15617790,15617804,15617866,15617877,15617934,15617962,15617982,15617991,15617995,15618016,15618018,15618057,15618155,15618171,15618182,15618203,15618213,15618225,15618231,15618239,15618242,15618243,15618245,15618255,15618258,15618280,15618289,15618314,15618363,15618391,15618401,15618406,15618410,15618437,15618443,15618446,15618505,15618507,15618562,15618563,15618566,15618572,15618581,15618647,15618661,15618667,15618695,15618703,15618709,15618750,15618765,15618774,15618782,15618893,15618926,15618936,15618950,15619016,15619029,15619045,15619083,15619087,15619116,15619126,15619130,15619238,15619280,15619304,15619340,15619343,15619348,15619352,15619360,15619407,15619436,15619494,15619508,15619514,15619529,15619537,15619608,15619616,15619626,15619674,15619699,15619708,15619801,15619857,15619898,15619932,15619935,15619953,15619955,15619972,15620000,15620030,15620103,15620111,15620123,15620204,15620268,15620276,15620284,15620338,15620341,15620343,15620344,15620372,15620379,15620398,15620443,15620451,15620494,15620504,15620505,15620507,15620557,15620570,15620572,15620577,15620579,15620676,15620735,15620746,15620750,15620751,15620756,15620758,15620774,15620832,15620836,15620893,15620936,15620981,15621004,15621021,15621031,15621032,15621063,15621064,15621072,15621075,15621083,15621093,15621110,15621115,15621116,15621132,15621140,15621166,15621170,15621195,15621205,15621207,15621208,15621209,15621210,15621216,15621218,15621223,15621227,15621236,15621240,15621241,15621244,15621248,15621267,15621313,15621353,15621383,15621409,15621411,15621413,15621423,15621426,15621427,15621428,15621432,15621433,15621435,15621436,15621439,15621440,15621442,15621446,15621451,15621453,15621456,15621457,15621460,15621466,15621476,15621477,15621480,15621487,15621490,15621514,15621520,15621523,15621540,15621544,15621546,15621550,15621551,15621557,15621608,15621609,15621611,15621622,15621625,15621629,15621633,15621635,15621636,15621644,15621645,15621653,15621654,15621655,15621656,15621658,15621670,15621675,15621682,15621685,15621687,15621689,15621691,15621714,15621715,15621753,15621768,15621783,15621801,15621802,15621813,15621814,15621815,15621818,15621819,15621820,15621821,15621822,15621830,15621834,15621835,15621838,15621854,15621867,15621868,15621870,15621883,15621893,15621974,15622003,15622010,15622017,15622033,15622039,15622053,15622083,15622102,15622117,15622122,15622171,15622172,15622182,15622192,15622212,15622217,15622230,15622289,15622305,15622321,15622350,15622370,15622403,15622404,15622423,15622435,15622436,15622438,15622442,15622443,15622444,15622448,15622453,15622454,15622461,15622463,15622469,15622470,15622478,15622482,15622494,15622507,15622516,15622518,15622521,15622522,15622527,15622528,15622532,15622538,15622547,15622562,15622570,15622575,15622578,15622585,15622599,15622607,15622631,15622658,15622692,15622695,15622729,15622750,15622774,15622801,15622832,15622833,15622834,15622897,15622911,15622985,15622989,15622991,15622993,15623042,15623048,15623053,15623062,15623063,15623072,15623082,15623102,15623107,15623143,15623146,15623180,15623195,15623196,15623197,15623202,15623210,15623219,15623220,15623229,15623277,15623299,15623303,15623314,15623346,15623357,15623369,15623430,15623450,15623471,15623478,15623489,15623491,15623494,15623499,15623502,15623521,15623525,15623536,15623559,15623560,15623566,15623595,15623596,15623630,15623644,15623647,15623648,15623649,15623658,15623667,15623668,15623695,15623712,15623730,15623744,15623787,15623791,15623828,15623927,15623944,15623948,15623966,15623972,15623989,15624013,15624068,15624072,15624145,15624150,15624151,15624170,15624175,15624180,15624185,15624186,15624187,15624188,15624217,15624229,15624269,15624293,15624323,15624339,15624341,15624343,15624347,15624351,15624357,15624369,15624373,15624374,15624385,15624388,15624391,15624395,15624397,15624423,15624424,15624428,15624437,15624443,15624451,15624453,15624466,15624471,15624505,15624510,15624528,15624592,15624595,15624596,15624623,15624633,15624641,15624644,15624648,15624665,15624667,15624677,15624684,15624691,15624695,15624703,15624715,15624729,15624744,15624755,15624775,15624778,15624781,15624793,15624820,15624850,15624863,15624866,15624892,15624893,15624936,15624975,15624993,15624995,15625020,15625021,15625023,15625032,15625034,15625035,15625036,15625042,15625047,15625067,15625068,15625080,15625082,15625083,15625092,15625126,15625141,15625151,15625175,15625185,15625196,15625247,15625252,15625311,15625355,15625391,15625395,15625426,15625461,15625491,15625494,15625495,15625501,15625522,15625524,15625545,15625584,15625586,15625592,15625595,15625596,15625623,15625627,15625632,15625641,15625665,15625675,15625676,15625695,15625698,15625706,15625716,15625740,15625759,15625761,15625776,15625796,15625806,15625819,15625824,15625881,15625882,15625886,15625904,15625905,15625907,15625916,15625919,15625929,15625941,15625942,15625944,15625945,15625946,15625948,15625963,15625964,15626008,15626012,15626040,15626042,15626044,15626114,15626135,15626141,15626143,15626144,15626147,15626153,15626156,15626186,15626212,15626233,15626243,15626310,15626322,15626338,15626383,15626385,15626395,15626412,15626414,15626452,15626457,15626474,15626475,15626478,15626484,15626485,15626491,15626494,15626507,15626578,15626608,15626612,15626628,15626679,15626684,15626695,15626710,15626742,15626774,15626787,15626793,15626795,15626796,15626806,15626882,15626898,15626900,15626948,15626983,15626985,15627000,15627002,15627008,15627010,15627027,15627042,15627054,15627138,15627162,15627166,15627170,15627180,15627185,15627190,15627195,15627197,15627198,15627200,15627202,15627203,15627205,15627212,15627220,15627232,15627251,15627262,15627298,15627299,15627305,15627328,15627352,15627355,15627358,15627360,15627361,15627365,15627377,15627384,15627385,15627395,15627412,15627439,15627484,15627514,15627517,15627637,15627665,15627696,15627697,15627703,15627706,15627741,15627801,15627811,15627813,15627820,15627827,15627829,15627830,15627839,15627840,15627841,15627844,15627847,15627848,15627849,15627850,15627854,15627856,15627858,15627859,15627876,15627878,15627879,15627881,15627995,15627999,15628003,15628007,15628008,15628012,15628033,15628034,15628051,15628059,15628064,15628112,15628117,15628123,15628139,15628143,15628144,15628145,15628150,15628155,15628170,15628180,15628203,15628205,15628219,15628225,15628238,15628243,15628252,15628256,15628272,15628274,15628279,15628283,15628292,15628298,15628303,15628305,15628308,15628319,15628420,15628523,15628530,15628558,15628574,15628583,15628600,15628608,15628666,15628674,15628697,15628715,15628767,15628813,15628824,15628863,15628881,15628883,15628886,15628893,15628894,15628904,15628936,15628947,15628950,15628963,15628972,15628985,15628992,15628999,15629002,15629004,15629008,15629010,15629012,15629018,15629043,15629054,15629062,15629078,15629094,15629117,15629128,15629133,15629150,15629155,15629187,15629238,15629239,15629244,15629249,15629252,15629273,15629279,15629298,15629299,15629303,15629323,15629338,15629377,15629388,15629390,15629448,15629475,15629507,15629511,15629519,15629545,15629551,15629575,15629586,15629603,15629605,15629617,15629677,15629739,15629744,15629750,15629787,15629793,15629846,15629885,15629956,15629985,15630051,15630053,15630067,15630101,15630103,15630104,15630105,15630106,15630108,15630113,15630139,15630147,15630155,15630157,15630161,15630167,15630194,15630195,15630196,15630241,15630244,15630269,15630273,15630321,15630328,15630349,15630353,15630363,15630398,15630402,15630413,15630419,15630441,15630466,15630488,15630511,15630551,15630579,15630592,15630617,15630641,15630648,15630649,15630661,15630698,15630704,15630706,15630709,15630725,15630752,15630857,15630910,15630920,15631035,15631047,15631053,15631054,15631059,15631060,15631063,15631070,15631072,15631075,15631081,15631136,15631159,15631160,15631161,15631162,15631166,15631170,15631172,15631189,15631201,15631210,15631211,15631216,15631217,15631219,15631222,15631240,15631244,15631248,15631250,15631261,15631267,15631305,15631310,15631312,15631315,15631326,15631330,15631333,15631336,15631339,15631359,15631369,15631389,15631392,15631393,15631406,15631408,15631412,15631413,15631433,15631436,15631446,15631450,15631451,15631453,15631457,15631459,15631460,15631461,15631463,15631469,15631471,15631473,15631476,15631478,15631481,15631512,15631550,15631563,15631603,15631607,15631609,15631639,15631642,15631644,15631646,15631653,15631669,15631671,15631674,15631675,15631681,15631682,15631683,15631685,15631689,15631691,15631692,15631693,15631709,15631713,15631714,15631715,15631719,15631721,15631723,15631733,15631734,15631736,15631739,15631751,15631752,15631756,15631766,15631775,15631781,15631800,15631813,15631815,15631816,15631817,15631818,15631821,15631822,15631823,15631828,15631833,15631835,15631838,15631839,15631844,15631848,15631851,15631868,15631869,15631870,15631871,15631872,15631873,15631875,15631877,15631878,15631882,15631884,15631893,15631905,15631912,15631914,15631927,15631935,15631958,15631973,15631989,15632010,15632017,15632050,15632051,15632069,15632079,15632113,15632118,15632125,15632143,15632171,15632185,15632210,15632220,15632238,15632250,15632256,15632262,15632263,15632264,15632265,15632270,15632272,15632275,15632280,15632291,15632293,15632305,15632307,15632323,15632324,15632328,15632344,15632345,15632351,15632365,15632369,15632370,15632436,15632437,15632438,15632439,15632444,15632446,15632448,15632451,15632459,15632461,15632462,15632467,15632472,15632474,15632478,15632503,15632510,15632519,15632521,15632522,15632523,15632525,15632532,15632538,15632545,15632547,15632551,15632569,15632570,15632572,15632575,15632576,15632578,15632579,15632591,15632594,15632599,15632601,15632609,15632650,15632651,15632663,15632665,15632668,15632669,15632670,15632699,15632709,15632735,15632736,15632757,15632761,15632762,15632763,15632779,15632788,15632789,15632793,15632797,15632800,15632801,15632816,15632826,15632830,15632834,15632841,15632846,15632847,15632848,15632850,15632859,15632875,15632879,15632882,15632887,15632899,15632914,15632937,15632939,15632944,15632945,15632946,15632950,15632967,15632977,15632987,15632997,15632999,15633043,15633044,15633046,15633048,15633059,15633064,15633067,15633108,15633111,15633112,15633119,15633141,15633172,15633181,15633194,15633213,15633227,15633228,15633229,15633233,15633247,15633256,15633260,15633274,15633283,15633348,15633352,15633378,15633419,15633461,15633474,15633516,15633525,15633531,15633537,15633546,15633574,15633586,15633597,15633608,15633612,15633640,15633644,15633648,15633650,15633666,15633729,15633768,15633797,15633818,15633830,15633834,15633835,15633840,15633852,15633854,15633857,15633860,15633866,15633870,15633872,15633876,15633877,15633883,15633897,15633922,15633930,15633944,15633950,15634008,15634027,15634051,15634064,15634091,15634099,15634141,15634143,15634146,15634180,15634190,15634218,15634237,15634245,15634246,15634260,15634262,15634267,15634272,15634274,15634275,15634281,15634282,15634295,15634310,15634312,15634326,15634359,15634371,15634373,15634375,15634431,15634491,15634501,15634551,15634564,15634571,15634586,15634591,15634592,15634602,15634606,15634632,15634719,15634772,15634774,15634792,15634815,15634844,15634891,15634892,15634893,15634896,15634936,15634948,15634949,15634950,15634968,15634970,15634974,15634990,15634999,15635029,15635034,15635067,15635078,15635082,15635097,15635116,15635125,15635126,15635141,15635143,15635176,15635177,15635185,15635197,15635203,15635222,15635240,15635242,15635244,15635245,15635246,15635256,15635277,15635285,15635330,15635346,15635356,15635364,15635367,15635371,15635388,15635396,15635435,15635459,15635461,15635500,15635502,15635531,15635597,15635598,15635660,15635684,15635687,15635703,15635728,15635752,15635775,15635776,15635782,15635784,15635792,15635870,15635893,15635905,15635938,15635939,15635942,15635949,15635964,15635968,15635972,15636008,15636016,15636018,15636021,15636022,15636023,15636024,15636031,15636043,15636044,15636047,15636049,15636061,15636069,15636089,15636105,15636127,15636238,15636330,15636339,15636353,15636368,15636373,15636388,15636395,15636396,15636397,15636407,15636411,15636420,15636422,15636423,15636425,15636427,15636428,15636444,15636445,15636454,15636478,15636520,15636521,15636541,15636548,15636551,15636562,15636572,15636578,15636582,15636588,15636589,15636590,15636592,15636595,15636624,15636626,15636634,15636643,15636656,15636658,15636673,15636684,15636700,15636731,15636736,15636750,15636755,15636756,15636767,15636812,15636820,15636860,15636872,15636912,15636935,15636977,15636982,15636990,15636993,15636998,15636999,15637024,15637051,15637073,15637087,15637105,15637110,15637118,15637124,15637131,15637133,15637136,15637138,15637168,15637169,15637178,15637194,15637235,15637240,15637265,15637315,15637324,15637354,15637365,15637366,15637385,15637404,15637411,15637414,15637427,15637428,15637454,15637461,15637474,15637476,15637522,15637570,15637591,15637593,15637599,15637605,15637644,15637646,15637648,15637650,15637678,15637690,15637694,15637717,15637753,15637774,15637778,15637782,15637784,15637788,15637795,15637829,15637891,15637947,15637954,15637976,15637977,15637979,15637993,15638003,15638028,15638044,15638047,15638059,15638067,15638068,15638149,15638157,15638159,15638166,15638171,15638207,15638231,15638247,15638257,15638267,15638269,15638270,15638272,15638289,15638329,15638355,15638463,15638487,15638494,15638502,15638513,15638605,15638607,15638610,15638646,15638730,15638751,15638774,15638797,15638803,15638806,15638871,15638882,15638884,15638889,15638963,15638983,15638988,15638989,15639004,15639077,15639103,15639117,15639133,15639178,15639217,15639223,15639252,15639262,15639263,15639265,15639266,15639271,15639272,15639277,15639278,15639303,15639314,15639322,15639355,15639357,15639368,15639415,15639552,15639576,15639590,15639662,15639665,15639669,15639788,15639882,15639946,15639951,15639992,15640019,15640034,15640038,15640039,15640049,15640056,15640059,15640068,15640074,15640076,15640078,15640079,15640094,15640105,15640106,15640121,15640133,15640161,15640163,15640167,15640194,15640201,15640243,15640244,15640257,15640258,15640260,15640266,15640268,15640280,15640283,15640345,15640357,15640367,15640368,15640377,15640383,15640390,15640407,15640409,15640418,15640421,15640442,15640454,15640463,15640464,15640476,15640491,15640507,15640521,15640580,15640627,15640635,15640648,15640667,15640686,15640769,15640789,15640825,15640846,15640849,15640852,15640855,15640857,15640859,15640865,15640866,15640900,15640905,15640917,15640922,15640953,15641004,15641007,15641008,15641009,15641021,15641029,15641036,15641043,15641075,15641085,15641091,15641093,15641110,15641112,15641114,15641116,15641119,15641122,15641133,15641136,15641150,15641158,15641162,15641165,15641170,15641175,15641180,15641181,15641182,15641183,15641189,15641249,15641250,15641253,15641259,15641267,15641270,15641275,15641276,15641278,15641281,15641282,15641285,15641297,15641298,15641310,15641311,15641312,15641315,15641316,15641319,15641322,15641326,15641327,15641329,15641330,15641342,15641352,15641357,15641358,15641359,15641366,15641374,15641377,15641381,15641387,15641389,15641390,15641391,15641392,15641398,15641399,15641410,15641411,15641412,15641413,15641414,15641416,15641417,15641426,15641433,15641440,15641442,15641457,15641460,15641466,15641474,15641476,15641480,15641490,15641519,15641560,15641574,15641575,15641582,15641585,15641591,15641592,15641595,15641603,15641604,15641608,15641611,15641620,15641621,15641622,15641624,15641625,15641626,15641629,15641630,15641634,15641638,15641640,15641644,15641655,15641656,15641660,15641662,15641670,15641674,15641675,15641678,15641680,15641681,15641682,15641685,15641687,15641688,15641690,15641691,15641692,15641693,15641694,15641695,15641698,15641710,15641715,15641722,15641723,15641730,15641732,15641733,15641734,15641735,15641742,15641745,15641755,15641766,15641773,15641775,15641779,15641782,15641783,15641784,15641785,15641789,15641791,15641792,15641793,15641794,15641810,15641811,15641820,15641822,15641830,15641833,15641835,15641838,15641840,15641842,15641844,15641850,15641860,15641866,15641868,15641870,15641877,15641882,15641884,15641893,15641934,15641994,15642001,15642002,15642004,15642021,15642022,15642034,15642040,15642041,15642053,15642055,15642063,15642069,15642082,15642091,15642093,15642094,15642095,15642098,15642099,15642137,15642160,15642162,15642202,15642222,15642236,15642248,15642289,15642291,15642299,15642312,15642324,15642335,15642336,15642344,15642345,15642346,15642356,15642360,15642365,15642369,15642379,15642390,15642391,15642393,15642394,15642397,15642400,15642404,15642408,15642434,15642438,15642439,15642440,15642448,15642454,15642455,15642456,15642460,15642466,15642467,15642468,15642478,15642479,15642482,15642488,15642491,15642515,15642519,15642523,15642530,15642532,15642533,15642538,15642540,15642544,15642545,15642546,15642555,15642563,15642570,15642575,15642578,15642579,15642585,15642588,15642591,15642610,15642615,15642618,15642619,15642621,15642622,15642635,15642636,15642640,15642655,15642658,15642670,15642675,15642685,15642704,15642705,15642708,15642709,15642710,15642712,15642715,15642718,15642724,15642725,15642726,15642735,15642736,15642740,15642741,15642782,15642785,15642787,15642789,15642792,15642797,15642800,15642810,15642811,15642812,15642815,15642816,15642817,15642818,15642821,15642822,15642823,15642824,15642825,15642826,15642830,15642832,15642833,15642834,15642835,15642836,15642838,15642839,15642848,15642865,15642868,15642870,15642882,15642884,15642885,15642887,15642890,15642899,15642903,15642909,15642912,15642914,15642916,15642926,15642933,15642934,15642937,15642948,15642951,15642955,15642977,15642982,15642985,15642991,15642992,15642994,15642996,15642997,15642998,15642999,15643022,15643024,15643032,15643036,15643042,15643043,15643046,15643048,15643055,15643056,15643059,15643065,15643082,15643097,15643121,15643215,15643294,15643313,15643315,15643319,15643323,15643325,15643326,15643330,15643338,15643340,15643347,15643349,15643354,15643358,15643359,15643361,15643363,15643365,15643372,15643377,15643378,15643382,15643386,15643392,15643393,15643394,15643396,15643397,15643416,15643426,15643427,15643436,15643438,15643439,15643487,15643496,15643498,15643523,15643525,15643526,15643574,15643575,15643601,15643617,15643618,15643622,15643624,15643625,15643626,15643628,15643629,15643630,15643635,15643636,15643643,15643644,15643650,15643653,15643654,15643656,15643657,15643658,15643659,15643660,15643670,15643671,15643674,15643678,15643679,15643682,15643683,15643689,15643690,15643691,15643693,15643694,15643696,15643699,15643752,15643770,15643783,15643790,15643794,15643816,15643817,15643818,15643819,15643821,15643822,15643823,15643824,15643825,15643826,15643828,15643829,15643830,15643831,15643832,15643836,15643844,15643849,15643856,15643858,15643859,15643865,15643870,15643871,15643876,15643880,15643896,15643897,15643903,15643904,15643906,15643907,15643908,15643909,15643913,15643914,15643916,15643917,15643921,15643936,15643942,15643958,15643966,15643967,15643968,15643970,15643972,15643974,15643988,15644005,15644119,15644132,15644174,15644179,15644212,15644280,15644292,15644295,15644296,15644297,15644356,15644361,15644400,15644438,15644443,15644446,15644453,15644501,15644515,15644522,15644525,15644526,15644551,15644555,15644561,15644572,15644574,15644576,15644577,15644591,15644621,15644692,15644699,15644721,15644724,15644727,15644743,15644753,15644761,15644762,15644765,15644786,15644788,15644794,15644796,15644797,15644832,15644833,15644834,15644844,15644846,15644849,15644850,15644851,15644852,15644853,15644854,15644856,15644857,15644858,15644859,15644860,15644861,15644865,15644866,15644868,15644878,15644880,15644881,15644882,15644890,15644891,15644892,15644894,15644896,15644933,15644934,15644962,15644964,15644966,15644967,15644975,15644977,15644994,15645021,15645023,15645024,15645025,15645029,15645030,15645034,15645035,15645045,15645059,15645062,15645074,15645078,15645082,15645094,15645095,15645096,15645097,15645099,15645103,15645136,15645176,15645184,15645196,15645200,15645232,15645248,15645252,15645269,15645271,15645316,15645323,15645341,15645353,15645355,15645368,15645371,15645404,15645463,15645478,15645488,15645492,15645496,15645511,15645517,15645543,15645544,15645569,15645571,15645572,15645593,15645600,15645621,15645637,15645717,15645744,15645766,15645772,15645778,15645806,15645815,15645820,15645827,15645835,15645839,15645843,15645847,15645896,15645904,15645937,15645942,15645956,15646022,15646030,15646034,15646035,15646037,15646055,15646071,15646074,15646078,15646082,15646087,15646088,15646091,15646094,15646095,15646096,15646099,15646152,15646161,15646168,15646190,15646196,15646227,15646262,15646276,15646302,15646310,15646351,15646366,15646372,15646374,15646379,15646490,15646513,15646514,15646515,15646521,15646522,15646526,15646535,15646537,15646539,15646558,15646562,15646563,15646566,15646569,15646577,15646584,15646594,15646609,15646613,15646615,15646616,15646632,15646636,15646710,15646711,15646713,15646715,15646716,15646717,15646719,15646720,15646724,15646725,15646726,15646734,15646735,15646756,15646764,15646765,15646776,15646782,15646784,15646791,15646796,15646798,15646800,15646810,15646811,15646817,15646821,15646824,15646834,15646835,15646849,15646850,15646855,15646856,15646859,15646870,15646875,15646884,15646896,15646921,15646934,15646935,15646936,15646937,15646942,15646969,15646974,15646982,15647008,15647091,15647097,15647099,15647105,15647111,15647191,15647203,15647207,15647259,15647266,15647292,15647295,15647301,15647306,15647311,15647333,15647402,15647453,15647474,15647521,15647535,15647540,15647546,15647562,15647566,15647570,15647572,15647574,15647575,15647576,15647578,15647584,15647588,15647591,15647592,15647593,15647595,15647597,15647725,15647782,15647800,15647807,15647812,15647824,15647826,15647827,15647829,15647830,15647838,15647839,15647842,15647848,15647868,15647874,15647876,15647890,15647891,15647895,15647896,15647898,15647965,15647969,15647974,15647975,15648005,15648030,15648032,15648034,15648035,15648042,15648043,15648044,15648047,15648048,15648050,15648053,15648059,15648064,15648065,15648067,15648068,15648069,15648070,15648074,15648075,15648078,15648082,15648088,15648136,15648178,15648225,15648272,15648320,15648362,15648367,15648455,15648461,15648479,15648489,15648577,15648638,15648681,15648702,15648720,15648721,15648725,15648726,15648728,15648733,15648735,15648736,15648747,15648766,15648767,15648768,15648775,15648780,15648788,15648794,15648800,15648810,15648830,15648857,15648858,15648859,15648872,15648876,15648881,15648884,15648889,15648890,15648891,15648894,15648896,15648898,15648933,15648965,15648966,15648967,15649024,15649033,15649048,15649060,15649064,15649070,15649078,15649086,15649101,15649112,15649121,15649124,15649129,15649136,15649150,15649160,15649183,15649211,15649239,15649280,15649292,15649297,15649354,15649359,15649379,15649400,15649412,15649414,15649415,15649416,15649417,15649418,15649419,15649420,15649422,15649423,15649425,15649428,15649429,15649430,15649433,15649435,15649436,15649438,15649450,15649451,15649457,15649475,15649487,15649499,15649508,15649523,15649526,15649535,15649536,15649545,15649548,15649560,15649563,15649566,15649575,15649616,15649618,15649636,15649638,15649668,15649733,15649738,15649741,15649744,15649748,15649775,15649793,15649800,15649801,15649818,15649834,15649835,15649844,15649853,15649858,15649859,15649862,15649892,15649992,15650018,15650026,15650029,15650034,15650040,15650059,15650066,15650068,15650078,15650086,15650096,15650098,15650155,15650237,15650244,15650258,15650266,15650268,15650288,15650300,15650313,15650322,15650337,15650351,15650357,15650366,15650391,15650392,15650424,15650432,15650437,15650442,15650454,15650487,15650488,15650538,15650545,15650575,15650591,15650670,15650751,15650864,15650889,15650924,15650933,15651001,15651020,15651021,15651022,15651025,15651026,15651037,15651039,15651048,15651052,15651057,15651060,15651063,15651064,15651075,15651084,15651089,15651093,15651095,15651103,15651137,15651140,15651144,15651145,15651150,15651164,15651220,15651222,15651248,15651260,15651264,15651268,15651270,15651272,15651280,15651283,15651287,15651290,15651301,15651310,15651312,15651315,15651316,15651317,15651323,15651325,15651326,15651327,15651333,15651335,15651336,15651337,15651339,15651340,15651342,15651349,15651352,15651353,15651354,15651358,15651359,15651362,15651364,15651366,15651369,15651382,15651387,15651389,15651392,15651410,15651420,15651450,15651460,15651480,15651490,15651525,15651526,15651532,15651533,15651548,15651553,15651554,15651555,15651556,15651563,15651566,15651569,15651575,15651578,15651581,15651583,15651584,15651585,15651589,15651627,15651628,15651629,15651633,15651664,15651674,15651713,15651715,15651733,15651737,15651739,15651740,15651784,15651802,15651803,15651823,15651840,15651848,15651867,15651868,15651874,15651877,15651883,15651900,15651937,15651943,15651955,15651958,15651983,15652028,15652030,15652032,15652034,15652048,15652052,15652053,15652064,15652067,15652069,15652079,15652084,15652085,15652089,15652093,15652097,15652169,15652180,15652192,15652211,15652212,15652214,15652217,15652218,15652266,15652268,15652289,15652291,15652292,15652293,15652320,15652400,15652401,15652469,15652527,15652540,15652613,15652615,15652622,15652625,15652626,15652627,15652657,15652658,15652662,15652665,15652667,15652674,15652676,15652677,15652693,15652698,15652700,15652779,15652782,15652785,15652789,15652791,15652808,15652814,15652835,15652840,15652865,15652882,15652883,15652884,15652885,15652901,15652909,15652910,15652912,15652914,15652915,15652930,15652937,15652939,15652944,15652948,15652949,15652950,15652955,15652960,15652965,15652966,15652967,15652969,15652976,15652979,15652980,15652981,15652983,15652989,15652995,15652999,15653007,15653025,15653030,15653035,15653037,15653048,15653050,15653055,15653060,15653065,15653110,15653147,15653222,15653251,15653253,15653275,15653306,15653315,15653347,15653404,15653455,15653483,15653496,15653521,15653522,15653541,15653547,15653548,15653562,15653563,15653572,15653573,15653574,15653575,15653576,15653577,15653578,15653584,15653595,15653596,15653620,15653696,15653700,15653710,15653713,15653715,15653720,15653741,15653750,15653752,15653753,15653762,15653765,15653768,15653770,15653772,15653773,15653776,15653778,15653779,15653780,15653782,15653784,15653796,15653830,15653837,15653840,15653849,15653852,15653853,15653854,15653857,15653858,15653859,15653871,15653880,15653893,15653937,15653952,15653955,15653957,15653958,15653966,15653967,15653971,15653976,15653997,15654000,15654020,15654025,15654033,15654053,15654060,15654067,15654070,15654074,15654075,15654084,15654093,15654095,15654096,15654106,15654156,15654183,15654211,15654222,15654229,15654230,15654238,15654274,15654292,15654295,15654296,15654300,15654341,15654346,15654372,15654379,15654408,15654409,15654446,15654456,15654457,15654463,15654464,15654489,15654495,15654519,15654531,15654549,15654553,15654559,15654561,15654562,15654572,15654574,15654576,15654577,15654578,15654584,15654611,15654625,15654654,15654673,15654700,15654719,15654720,15654725,15654731,15654733,15654748,15654760,15654765,15654766,15654767,15654771,15654772,15654796,15654851,15654857,15654859,15654862,15654870,15654871,15654875,15654878,15654880,15654882,15654883,15654890,15654891,15654896,15654901,15654933,15654939,15654940,15654950,15654959,15654964,15654971,15654975,15655007,15655025,15655082,15655123,15655171,15655172,15655175,15655185,15655192,15655213,15655227,15655229,15655232,15655233,15655238,15655252,15655269,15655271,15655274,15655298,15655328,15655335,15655339,15655341,15655345,15655360,15655368,15655383,15655389,15655396,15655410,15655430,15655436,15655461,15655464,15655574,15655589,15655590,15655632,15655648,15655658,15655679,15655774,15655794,15655859,15655875,15655896,15655903,15655952,15655961,15656005,15656009,15656027,15656029,15656030,15656060,15656062,15656086,15656094,15656096,15656100,15656121,15656148,15656176,15656188,15656263,15656286,15656293,15656300,15656310,15656318,15656319,15656322,15656328,15656329,15656330,15656333,15656350,15656351,15656413,15656414,15656416,15656417,15656425,15656426,15656427,15656428,15656432,15656433,15656435,15656454,15656461,15656463,15656467,15656468,15656471,15656478,15656484,15656487,15656489,15656492,15656513,15656522,15656523,15656526,15656535,15656537,15656539,15656547,15656563,15656586,15656592,15656593,15656594,15656595,15656596,15656597,15656611,15656613,15656627,15656637,15656678,15656682,15656683,15656707,15656710,15656717,15656720,15656726,15656784,15656812,15656813,15656817,15656820,15656822,15656823,15656824,15656825,15656827,15656829,15656830,15656835,15656840,15656845,15656849,15656853,15656865,15656868,15656869,15656870,15656883,15656884,15656901,15656902,15656912,15656913,15656917,15656921,15656937,15657084,15657085,15657105,15657107,15657139,15657163,15657214,15657228,15657284,15657306,15657314,15657317,15657321,15657342,15657349,15657386,15657396,15657439,15657449,15657527,15657529,15657532,15657533,15657534,15657535,15657537,15657539,15657540,15657546,15657548,15657553,15657560,15657565,15657566,15657571,15657599,15657603,15657637,15657733,15657737,15657747,15657749,15657770,15657771,15657772,15657777,15657778,15657779,15657809,15657810,15657812,15657822,15657825,15657828,15657829,15657830,15657837,15657838,15657852,15657853,15657868,15657881,15657898,15657899,15657937,15657957,15658032,15658042,15658048,15658052,15658057,15658058,15658059,15658067,15658100,15658148,15658158,15658162,15658163,15658169,15658178,15658195,15658211,15658233,15658235,15658238,15658240,15658249,15658258,15658306,15658339,15658360,15658364,15658375,15658409,15658421,15658432,15658435,15658445,15658449,15658459,15658465,15658470,15658475,15658485,15658486,15658487,15658504,15658531,15658532,15658533,15658548,15658568,15658575,15658577,15658589,15658612,15658614,15658622,15658668,15658670,15658675,15658680,15658685,15658693,15658710,15658713,15658716,15658721,15658727,15658732,15658735,15658738,15658749,15658760,15658764,15658767,15658768,15658775,15658799,15658813,15658822,15658829,15658851,15658852,15658868,15658877,15658889,15658890,15658901,15658928,15658929,15658935,15658937,15658939,15658941,15658944,15658946,15658956,15658958,15658960,15658964,15658966,15658968,15658969,15658970,15658971,15658972,15658973,15658975,15658977,15658980,15658982,15658987,15659002,15659031,15659062,15659064,15659078,15659092,15659094,15659095,15659096,15659098,15659100,15659105,15659149,15659156,15659165,15659175,15659194,15659195,15659234,15659236,15659248,15659297,15659301,15659305,15659317,15659320,15659323,15659324,15659325,15659327,15659328,15659329,15659340,15659355,15659364,15659366,15659375,15659399,15659411,15659417,15659420,15659423,15659425,15659426,15659427,15659428,15659429,15659433,15659436,15659475,15659486,15659495,15659499,15659501,15659521,15659557,15659566,15659567,15659568,15659599,15659651,15659656,15659676,15659677,15659685,15659694,15659710,15659736,15659800,15659801,15659807,15659820,15659824,15659843,15659858,15659859,15659900,15659920,15659930,15659931,15659937,15659941,15659946,15659949,15659952,15659958,15659961,15659962,15660101,15660144,15660145,15660147,15660155,15660157,15660200,15660205,15660211,15660227,15660244,15660263,15660269,15660271,15660301,15660316,15660339,15660385,15660390,15660391,15660403,15660408,15660409,15660417,15660422,15660429,15660446,15660450,15660475,15660505,15660524,15660528,15660535,15660541,15660550,15660571,15660602,15660608,15660611,15660636,15660641,15660646,15660648,15660668,15660679,15660688,15660704,15660706,15660735,15660750,15660755,15660768,15660809,15660832,15660833,15660840,15660866,15660871,15660878,15660980,15661007,15661021,15661032,15661034,15661036,15661043,15661053,15661058,15661061,15661062,15661063,15661075,15661158,15661160,15661162,15661198,15661201,15661219,15661222,15661229,15661230,15661247,15661248,15661249,15661258,15661267,15661275,15661286,15661288,15661290,15661295,15661299,15661316,15661330,15661342,15661346,15661348,15661349,15661370,15661380,15661383,15661387,15661389,15661390,15661408,15661409,15661410,15661411,15661412,15661417,15661423,15661433,15661442,15661456,15661457,15661460,15661473,15661474,15661507,15661513,15661524,15661525,15661526,15661528,15661530,15661532,15661533,15661534,15661548,15661551,15661554,15661555,15661577,15661591,15661607,15661608,15661626,15661627,15661628,15661629,15661630,15661640,15661654,15661655,15661656,15661657,15661658,15661670,15661674,15661694,15661702,15661703,15661704,15661706,15661707,15661708,15661709,15661716,15661723,15661733,15661734,15661736,15661743,15661745,15661748,15661755,15661756,15661757,15661766,15661777,15661787,15661813,15661815,15661818,15661821,15661823,15661825,15661827,15661830,15661833,15661871,15661877,15661903,15661907,15661921,15661936,15661945,15661958,15661974,15662021,15662033,15662034,15662035,15662063,15662066,15662067,15662071,15662081,15662084,15662085,15662087,15662088,15662091,15662093,15662097,15662098,15662100,15662151,15662152,15662203,15662221,15662232,15662234,15662238,15662243,15662245,15662263,15662268,15662289,15662291,15662294,15662337,15662346,15662347,15662354,15662355,15662381,15662391,15662393,15662394,15662397,15662398,15662400,15662401,15662403,15662404,15662407,15662408,15662409,15662415,15662421,15662426,15662432,15662433,15662434,15662435,15662438,15662444,15662448,15662453,15662454,15662456,15662458,15662460,15662461,15662463,15662467,15662472,15662478,15662482,15662483,15662486,15662487,15662488,15662491,15662492,15662493,15662494,15662496,15662497,15662500,15662501,15662521,15662523,15662526,15662531,15662532,15662533,15662534,15662538,15662541,15662566,15662571,15662574,15662576,15662577,15662578,15662579,15662583,15662588,15662594,15662597,15662618,15662626,15662628,15662640,15662641,15662643,15662658,15662662,15662664,15662676,15662679,15662688,15662694,15662698,15662708,15662726,15662727,15662736,15662747,15662751,15662756,15662757,15662758,15662773,15662781,15662783,15662788,15662789,15662790,15662792,15662835,15662851,15662854,15662855,15662859,15662865,15662868,15662879,15662884,15662885,15662901,15662903,15662906,15662908,15662909,15662925,15662926,15662929,15662932,15662936,15662937,15662955,15662957,15662976,15662979,15662987,15662989,15662994,15663055,15663134,15663161,15663164,15663168,15663194,15663234,15663249,15663252,15663263,15663294,15663312,15663386,15663410,15663411,15663412,15663415,15663416,15663421,15663437,15663438,15663439,15663440,15663441,15663446,15663478,15663481,15663483,15663487,15663488,15663489,15663496,15663610,15663614,15663616,15663617,15663619,15663621,15663623,15663625,15663629,15663630,15663650,15663662,15663665,15663706,15663710,15663743,15663770,15663773,15663776,15663778,15663783,15663808,15663816,15663817,15663821,15663826,15663827,15663828,15663830,15663833,15663839,15663859,15663873,15663879,15663881,15663883,15663885,15663888,15663889,15663893,15663916,15663917,15663921,15663922,15663923,15663924,15663926,15663927,15663929,15663930,15663932,15663933,15663934,15663936,15663937,15663939,15663941,15663942,15663943,15663954,15663955,15663956,15663964,15663983,15663987,15664001,15664035,15664083,15664103,15664150,15664170,15664204,15664214,15664227,15664237,15664270,15664272,15664311,15664335,15664339,15664343,15664351,15664374,15664432,15664448,15664498,15664503,15664504,15664506,15664516,15664521,15664523,15664526,15664538,15664539,15664540,15664541,15664543,15664544,15664546,15664547,15664551,15664554,15664555,15664572,15664604,15664609,15664610,15664615,15664639,15664661,15664666,15664668,15664681,15664708,15664720,15664734,15664737,15664752,15664753,15664754,15664759,15664772,15664793,15664802,15664808,15664809,15664881,15664895,15664897,15664907,15665008,15665014,15665032,15665043,15665048,15665053,15665056,15665058,15665060,15665062,15665064,15665067,15665068,15665077,15665078,15665079,15665082,15665083,15665084,15665085,15665087,15665088,15665091,15665093,15665094,15665095,15665096,15665110,15665129,15665141,15665158,15665159,15665168,15665177,15665180,15665181,15665221,15665222,15665238,15665283,15665294,15665327,15665330,15665340,15665374,15665376,15665377,15665378,15665385,15665402,15665411,15665412,15665416,15665421,15665423,15665429,15665438,15665439,15665440,15665460,15665470,15665520,15665521,15665523,15665524,15665530,15665531,15665532,15665534,15665535,15665536,15665537,15665538,15665539,15665543,15665555,15665559,15665560,15665579,15665580,15665584,15665585,15665588,15665589,15665590,15665611,15665629,15665630,15665634,15665638,15665639,15665678,15665695,15665736,15665759,15665760,15665766,15665782,15665783,15665784,15665785,15665790,15665794,15665802,15665811,15665823,15665827,15665829,15665830,15665834,15665843,15665856,15665858,15665859,15665861,15665865,15665868,15665872,15665879,15665884,15665886,15665887,15665890,15665891,15665892,15665893,15665937,15665939,15665941,15665943,15665946,15665947,15665951,15665955,15665956,15665957,15665958,15665963,15665984,15665987,15665988,15666012,15666021,15666030,15666032,15666034,15666043,15666047,15666055,15666057,15666058,15666060,15666061,15666062,15666064,15666076,15666094,15666095,15666096,15666097,15666141,15666166,15666173,15666175,15666181,15666190,15666197,15666200,15666252,15666295,15666297,15666332,15666430,15666453,15666548,15666559,15666633,15666652,15666675,15666711,15666767,15666810,15666821,15666822,15666823,15666824,15666826,15666835,15666839,15666843,15666853,15666855,15666856,15666858,15666859,15666871,15666883,15666884,15666885,15666886,15666887,15666893,15666894,15666916,15666921,15666926,15666939,15666943,15666945,15666957,15666959,15666962,15666973,15666975,15666981,15666982,15666983,15666985,15666987,15667035,15667043,15667046,15667078,15667084,15667085,15667087,15667090,15667093,15667094,15667095,15667134,15667215,15667216,15667265,15667289,15667321,15667392,15667416,15667417,15667418,15667421,15667423,15667424,15667429,15667432,15667434,15667435,15667438,15667439,15667440,15667451,15667458,15667460,15667476,15667480,15667521,15667534,15667539,15667542,15667546,15667553,15667554,15667556,15667559,15667564,15667572,15667573,15667579,15667584,15667588,15667590,15667595,15667602,15667610,15667620,15667621,15667632,15667633,15667634,15667651,15667742,15667751,15667830,15667871,15667873,15667889,15667891,15667896,15667932,15667933,15667934,15667937,15667938,15667942,15667944,15667945,15667946,15667947,15667958,15667959,15667964,15667971,15667972,15667974,15667984,15668009,15668032,15668054,15668055,15668057,15668058,15668059,15668071,15668072,15668081,15668085,15668104,15668142,15668233,15668235,15668248,15668270,15668283,15668289,15668309,15668323,15668347,15668385,15668444,15668445,15668460,15668472,15668504,15668521,15668528,15668535,15668560,15668575,15668580,15668594,15668629,15668669,15668679,15668695,15668723,15668747,15668760,15668767,15668768,15668774,15668775,15668799,15668815,15668818,15668823,15668830,15668843,15668853,15668854,15668858,15668860,15668882,15668883,15668884,15668885,15668886,15668889,15668890,15668893,15668894,15668942,15668943,15668944,15668958,15668960,15668971,15669011,15669060,15669064,15669169,15669175,15669190,15669200,15669204,15669219,15669253,15669262,15669281,15669282,15669286,15669290,15669293,15669301,15669326,15669327,15669342,15669401,15669410,15669411,15669412,15669414,15669417,15669490,15669491,15669501,15669511,15669516,15669589,15669599,15669606,15669611,15669656,15669664,15669741,15669781,15669782,15669783,15669805,15669887,15669899,15669921,15669922,15669926,15669944,15669945,15669946,15669947,15669949,15669951,15669952,15669955,15669956,15669957,15669983,15669985,15669987,15669991,15669994,15670019,15670028,15670029,15670030,15670034,15670037,15670039,15670049,15670058,15670059,15670066,15670080,15670103,15670104,15670121,15670161,15670163,15670172,15670174,15670199,15670205,15670266,15670291,15670316,15670343,15670345,15670354,15670355,15670374,15670379,15670385,15670416,15670427,15670481,15670492,15670496,15670528,15670552,15670562,15670571,15670572,15670576,15670584,15670587,15670602,15670615,15670619,15670646,15670662,15670668,15670702,15670738,15670753,15670755,15670756,15670822,15670823,15670859,15670866,15670873,15670889,15670898,15670914,15670943,15670947,15671014,15671026,15671032,15671053,15671055,15671057,15671058,15671074,15671081,15671086,15671087,15671090,15671091,15671092,15671093,15671096,15671097,15671104,15671124,15671134,15671137,15671138,15671139,15671142,15671144,15671147,15671148,15671149,15671158,15671165,15671170,15671172,15671173,15671174,15671209,15671210,15671219,15671220,15671221,15671222,15671229,15671230,15671239,15671241,15671242,15671243,15671244,15671249,15671250,15671251,15671253,15671256,15671258,15671261,15671274,15671284,15671287,15671293,15671294,15671299,15671310,15671312,15671322,15671326,15671327,15671330,15671332,15671333,15671334,15671336,15671339,15671340,15671342,15671343,15671344,15671345,15671349,15671350,15671351,15671353,15671354,15671355,15671358,15671366,15671369,15671374,15671378,15671379,15671383,15671385,15671387,15671389,15671390,15671392,15671394,15671403,15671407,15671408,15671409,15671410,15671412,15671420,15671433,15671440,15671443,15671450,15671460,15671480,15671490,15671510,15671512,15671513,15671514,15671519,15671522,15671528,15671529,15671530,15671534,15671542,15671543,15671550,15671574,15671581,15671582,15671583,15671584,15671585,15671590,15671591,15671605,15671606,15671608,15671609,15671610,15671624,15671629,15671630,15671634,15671640,15671655,15671656,15671670,15671674,15671687,15671690,15671697,15671703,15671708,15671709,15671710,15671714,15671715,15671722,15671726,15671729,15671730,15671733,15671734,15671739,15671748,15671749,15671753,15671755,15671756,15671758,15671763,15671765,15671766,15671769,15671770,15671774,15671779,15671782,15671783,15671787,15671789,15671800,15671803,15671843,15671910,15671912,15671915,15671917,15671924,15671927,15671928,15671930,15671932,15671934,15671942,15671952,15671958,15671969,15671973,15671987,15672012,15672016,15672021,15672022,15672026,15672027,15672028,15672029,15672034,15672048,15672050,15672051,15672053,15672054,15672056,15672069,15672073,15672081,15672082,15672084,15672087,15672091,15672092,15672093,15672096,15672097,15672115,15672132,15672144,15672145,15672152,15672185,15672210,15672216,15672219,15672237,15672238,15672242,15672246,15672268,15672294,15672297,15672299,15672325,15672327,15672328,15672330,15672332,15672334,15672354,15672355,15672356,15672357,15672358,15672359,15672361,15672362,15672363,15672364,15672367,15672372,15672374,15672381,15672390,15672394,15672397,15672403,15672409,15672421,15672428,15672429,15672430,15672431,15672432,15672433,15672434,15672437,15672441,15672443,15672447,15672448,15672451,15672460,15672461,15672463,15672466,15672467,15672469,15672476,15672480,15672481,15672482,15672483,15672491,15672497,15672499,15672516,15672530,15672574,15672577,15672578,15672583,15672588,15672591,15672594,15672595,15672596,15672597,15672598,15672599,15672600,15672610,15672613,15672617,15672618,15672626,15672628,15672631,15672634,15672637,15672638,15672639,15672640,15672650,15672651,15672660,15672673,15672683,15672692,15672693,15672698,15672704,15672708,15672709,15672730,15672751,15672754,15672756,15672758,15672762,15672773,15672777,15672778,15672787,15672791,15672794,15672796,15672797,15672798,15672801,15672820,15672821,15672826,15672827,15672829,15672833,15672834,15672835,15672836,15672846,15672854,15672855,15672857,15672875,15672881,15672884,15672885,15672893,15672894,15672898,15672899,15672901,15672903,15672905,15672908,15672909,15672910,15672912,15672920,15672928,15672929,15672930,15672933,15672934,15672935,15672937,15672938,15672939,15672940,15672941,15672945,15672950,15672955,15672958,15672960,15672966,15672967,15672968,15672970,15672973,15672975,15672977,15672979,15672980,15672983,15672987,15672989,15672991,15672992,15672994,15672995,15672999,15673020,15673024,15673026,15673030,15673055,15673065,15673073,15673074,15673077,15673081,15673082,15673084,15673092,15673093,15673094,15673180,15673223,15673238,15673316,15673319,15673327,15673329,15673333,15673338,15673339,15673342,15673344,15673347,15673359,15673361,15673363,15673365,15673367,15673369,15673370,15673371,15673372,15673381,15673387,15673391,15673392,15673394,15673439,15673481,15673501,15673519,15673520,15673521,15673522,15673526,15673529,15673538,15673539,15673541,15673550,15673551,15673562,15673570,15673571,15673573,15673576,15673578,15673579,15673580,15673582,15673584,15673591,15673594,15673599,15673619,15673620,15673629,15673632,15673639,15673693,15673738,15673739,15673747,15673760,15673761,15673765,15673772,15673776,15673779,15673789,15673790,15673793,15673798,15673817,15673819,15673820,15673828,15673837,15673847,15673857,15673865,15673868,15673870,15673873,15673876,15673877,15673878,15673882,15673883,15673888,15673890,15673893,15673906,15673907,15673908,15673917,15673921,15673928,15673929,15673936,15673939,15673955,15673958,15673971,15673976,15673980,15673982,15673984,15673995,15674000,15674067,15674110,15674148,15674149,15674156,15674179,15674188,15674206,15674254,15674328,15674331,15674343,15674368,15674398,15674431,15674433,15674435,15674439,15674440,15674442,15674443,15674519,15674524,15674526,15674531,15674533,15674537,15674539,15674541,15674543,15674551,15674554,15674558,15674573,15674576,15674583,15674584,15674586,15674591,15674594,15674620,15674637,15674671,15674672,15674673,15674678,15674720,15674721,15674722,15674724,15674727,15674729,15674733,15674740,15674741,15674749,15674750,15674760,15674765,15674787,15674811,15674820,15674822,15674827,15674829,15674830,15674832,15674833,15674834,15674839,15674840,15674842,15674844,15674849,15674850,15674851,15674853,15674856,15674858,15674860,15674868,15674870,15674872,15674875,15674880,15674882,15674884,15674890,15674896,15674910,15674916,15674920,15674922,15674927,15674928,15674929,15674930,15674931,15674932,15674934,15674939,15674940,15674947,15674950,15674951,15674991,15675092,15675119,15675141,15675176,15675180,15675184,15675185,15675190,15675192,15675203,15675206,15675231,15675238,15675243,15675252,15675256,15675263,15675301,15675316,15675320,15675328,15675337,15675360,15675380,15675410,15675430,15675431,15675450,15675501,15675518,15675550,15675562,15675585,15675626,15675650,15675675,15675749,15675776,15675787,15675791,15675794,15675842,15675854,15675888,15675896,15675926,15675949,15675963,15675964,15675983,15675985,15676082,15676091,15676094,15676117,15676156,15676353,15676404,15676433,15676513,15676515,15676517,15676518,15676519,15676520,15676521,15676526,15676538,15676542,15676547,15676563,15676570,15676571,15676576,15676586,15676598,15676610,15676617,15676645,15676670,15676707,15676713,15676715,15676725,15676733,15676748,15676767,15676774,15676778,15676780,15676810,15676812,15676827,15676828,15676835,15676852,15676865,15676869,15676870,15676890,15676895,15676902,15676905,15676909,15676910,15676913,15676917,15676929,15676932,15676933,15676934,15676936,15676937,15676939,15676941,15676942,15676944,15676952,15676957,15676958,15676959,15676966,15676968,15676971,15676981,15676983,15676985,15676986,15677020,15677026,15677035,15677042,15677049,15677051,15677052,15677053,15677055,15677056,15677057,15677058,15677059,15677062,15677064,15677067,15677071,15677081,15677082,15677085,15677087,15677091,15677092,15677094,15677095,15677096,15677105,15677112,15677115,15677117,15677123,15677135,15677136,15677141,15677146,15677147,15677148,15677149,15677153,15677173,15677184,15677217,15677223,15677243,15677249,15677288,15677294,15677295,15677305,15677307,15677310,15677314,15677317,15677321,15677324,15677328,15677336,15677340,15677369,15677371,15677377,15677382,15677387,15677395,15677429,15677435,15677457,15677510,15677512,15677519,15677521,15677522,15677524,15677529,15677532,15677538,15677539,15677540,15677546,15677550,15677551,15677557,15677558,15677559,15677567,15677571,15677576,15677580,15677586,15677597,15677599,15677610,15677712,15677720,15677732,15677739,15677763,15677764,15677765,15677783,15677784,15677785,15677794,15677796,15677804,15677818,15677828,15677871,15677903,15677904,15677906,15677907,15677908,15677909,15677917,15677920,15677921,15677922,15677929,15677937,15677938,15677939,15677946,15677955,15677958,15678022,15678034,15678056,15678057,15678058,15678059,15678067,15678068,15678129,15678146,15678168,15678173,15678201,15678206,15678210,15678284,15678317,15678333,15678385,15678412,15678427,15678460,15678497,15678571,15678572,15678626,15678714,15678715,15678717,15678718,15678720,15678721,15678722,15678725,15678726,15678727,15678728,15678729,15678730,15678743,15678750,15678760,15678775,15678779,15678780,15678799,15678835,15678868,15678876,15678882,15678886,15678906,15678908,15678910,15678915,15678916,15678917,15678920,15678928,15678937,15678943,15678944,15678947,15678958,15678970,15678971,15678975,15679024,15679025,15679028,15679029,15679030,15679037,15679048,15679060,15679062,15679064,15679075,15679103,15679131,15679136,15679145,15679148,15679200,15679211,15679249,15679254,15679283,15679284,15679294,15679297,15679299,15679307,15679394,15679410,15679429,15679453,15679531,15679550,15679551,15679587,15679610,15679611,15679612,15679616,15679617,15679620,15679622,15679623,15679642,15679643,15679651,15679663,15679668,15679673,15679693,15679710,15679717,15679718,15679722,15679723,15679733,15679737,15679738,15679739,15679740,15679743,15679760,15679770,15679780,15679800,15679801,15679804,15679808,15679810,15679811,15679813,15679814,15679818,15679820,15679870,15679884,15679903,15679908,15679909,15679914,15679928,15679941,15679946,15679955,15679958,15679960,15679961,15679962,15679963,15679966,15679967,15679968,15679969,15679971,15679972,15679973,15679978,15679981,15679983,15679984,15679985,15679987,15679991,15679992,15679994,15679998,15680001,15680039,15680046,15680049,15680068,15680103,15680112,15680139,15680141,15680143,15680144,15680150,15680155,15680161,15680167,15680169,15680197,15680201,15680215,15680224,15680234,15680243,15680278,15680301,15680303,15680339,15680340,15680345,15680346,15680366,15680370,15680389,15680399,15680405,15680421,15680430,15680436,15680449,15680492,15680517,15680587,15680597,15680611,15680643,15680650,15680666,15680677,15680682,15680683,15680685,15680706,15680727,15680736,15680752,15680753,15680755,15680767,15680772,15680779,15680797,15680804,15680805,15680843,15680855,15680863,15680895,15680901,15680914,15680916,15680918,15680920,15680953,15680970,15680998,15681007,15681018,15681020,15681026,15681032,15681036,15681046,15681047,15681048,15681058,15681059,15681060,15681061,15681068,15681070,15681074,15681075,15681076,15681081,15681092,15681093,15681096,15681112,15681114,15681115,15681126,15681147,15681158,15681165,15681174,15681175,15681180,15681196,15681203,15681204,15681205,15681206,15681208,15681210,15681219,15681220,15681226,15681227,15681244,15681258,15681260,15681270,15681272,15681274,15681275,15681287,15681292,15681295,15681297,15681299,15681316,15681326,15681327,15681332,15681334,15681337,15681342,15681346,15681349,15681357,15681402,15681406,15681409,15681410,15681411,15681412,15681419,15681420,15681423,15681425,15681428,15681429,15681433,15681439,15681440,15681446,15681450,15681453,15681455,15681456,15681457,15681460,15681461,15681463,15681465,15681476,15681479,15681480,15681506,15681509,15681525,15681530,15681542,15681550,15681551,15681552,15681553,15681554,15681555,15681556,15681562,15681574,15681576,15681581,15681606,15681610,15681627,15681629,15681645,15681647,15681652,15681654,15681655,15681656,15681658,15681662,15681665,15681669,15681671,15681674,15681675,15681677,15681679,15681694,15681695,15681697,15681703,15681705,15681706,15681708,15681709,15681710,15681720,15681723,15681740,15681753,15681755,15681757,15681758,15681765,15681766,15681768,15681774,15681776,15681828,15681840,15681843,15681875,15681878,15681883,15681884,15681887,15681893,15681910,15681912,15681913,15681914,15681915,15681917,15681921,15681923,15681924,15681925,15681928,15681954,15681955,15681956,15681958,15681990,15681991,15682021,15682048,15682049,15682055,15682060,15682065,15682069,15682070,15682071,15682072,15682084,15682099,15682211,15682218,15682221,15682236,15682237,15682263,15682265,15682266,15682268,15682269,15682273,15682274,15682276,15682277,15682293,15682318,15682322,15682328,15682330,15682348,15682351,15682353,15682355,15682356,15682357,15682358,15682359,15682364,15682368,15682369,15682372,15682406,15682412,15682421,15682424,15682425,15682428,15682429,15682430,15682431,15682432,15682434,15682435,15682436,15682437,15682439,15682442,15682443,15682444,15682445,15682446,15682447,15682448,15682451,15682453,15682454,15682455,15682456,15682457,15682459,15682460,15682461,15682462,15682463,15682465,15682467,15682468,15682469,15682472,15682473,15682474,15682476,15682478,15682479,15682489,15682501,15682518,15682521,15682522,15682523,15682526,15682527,15682528,15682530,15682531,15682532,15682533,15682534,15682535,15682537,15682538,15682539,15682540,15682541,15682547,15682548,15682550,15682551,15682554,15682555,15682558,15682559,15682563,15682565,15682570,15682573,15682574,15682576,15682578,15682583,15682585,15682594,15682600,15682601,15682612,15682616,15682618,15682619,15682626,15682627,15682628,15682631,15682637,15682638,15682639,15682641,15682643,15682647,15682655,15682657,15682660,15682663,15682668,15682686,15682692,15682693,15682708,15682722,15682728,15682736,15682738,15682741,15682744,15682751,15682754,15682756,15682757,15682758,15682759,15682773,15682778,15682780,15682788,15682819,15682822,15682826,15682827,15682830,15682834,15682839,15682841,15682844,15682852,15682855,15682856,15682857,15682860,15682862,15682865,15682868,15682874,15682879,15682880,15682882,15682890,15682893,15682902,15682909,15682912,15682914,15682918,15682919,15682920,15682922,15682923,15682924,15682925,15682927,15682928,15682929,15682933,15682939,15682943,15682944,15682947,15682948,15682952,15682955,15682956,15682960,15682963,15682967,15682968,15682974,15682975,15682977,15682979,15682980,15682993,15682994,15682995,15683007,15683026,15683027,15683053,15683055,15683059,15683075,15683118,15683124,15683157,15683168,15683183,15683195,15683213,15683276,15683319,15683322,15683323,15683334,15683337,15683338,15683339,15683340,15683342,15683344,15683348,15683352,15683353,15683359,15683362,15683363,15683365,15683369,15683374,15683375,15683377,15683378,15683379,15683381,15683383,15683403,15683416,15683428,15683439,15683450,15683460,15683461,15683462,15683463,15683468,15683469,15683471,15683472,15683479,15683481,15683483,15683487,15683496,15683503,15683517,15683520,15683521,15683522,15683523,15683524,15683525,15683526,15683528,15683534,15683538,15683539,15683541,15683543,15683544,15683552,15683553,15683554,15683555,15683560,15683562,15683570,15683572,15683576,15683578,15683582,15683584,15683591,15683593,15683594,15683599,15683610,15683613,15683614,15683615,15683616,15683618,15683621,15683624,15683625,15683627,15683628,15683629,15683637,15683639,15683640,15683657,15683658,15683660,15683668,15683677,15683680,15683681,15683682,15683683,15683693,15683714,15683718,15683758,15683816,15683817,15683818,15683819,15683820,15683825,15683826,15683828,15683841,15683842,15683852,15683861,15683863,15683871,15683873,15683883,15683892,15683902,15683907,15683908,15683909,15683913,15683914,15683917,15683920,15683921,15683925,15683928,15683930,15683941,15683942,15683947,15683952,15683953,15683955,15683956,15683957,15683958,15683964,15683967,15683971,15683972,15683973,15683974,15683976,15683977,15683980,15683993,15684010,15684011,15684042,15684103,15684147,15684148,15684167,15684171,15684173,15684181,15684193,15684196,15684198,15684211,15684248,15684267,15684269,15684270,15684294,15684295,15684296,15684316,15684317,15684318,15684319,15684320,15684321,15684328,15684340,15684348,15684358,15684367,15684395,15684419,15684421,15684430,15684435,15684438,15684439,15684440,15684441,15684442,15684444,15684461,15684463,15684469,15684474,15684484,15684512,15684514,15684516,15684517,15684518,15684519,15684526,15684528,15684532,15684536,15684537,15684542,15684544,15684548,15684551,15684555,15684556,15684558,15684563,15684573,15684582,15684584,15684592,15684596,15684645,15684696,15684719,15684720,15684721,15684722,15684726,15684733,15684744,15684765,15684768,15684769,15684771,15684775,15684776,15684783,15684788,15684801,15684820,15684840,15684844,15684849,15684851,15684852,15684854,15684858,15684859,15684860,15684861,15684862,15684865,15684868,15684869,15684870,15684871,15684875,15684890,15684902,15684903,15684906,15684907,15684908,15684909,15684914,15684920,15684921,15684922,15684924,15684925,15684926,15684928,15684929,15684930,15684933,15684934,15684936,15684941,15684946,15684947,15684949,15684950,15684951,15684952,15684954,15684955,15684956,15684957,15684958,15684959,15684960,15684961,15684967,15684995,15684999,15685018,15685019,15685078,15685096,15685109,15685226,15685241,15685243,15685245,15685279,15685290,15685300,15685301,15685309,15685310,15685314,15685320,15685328,15685329,15685330,15685340,15685346,15685357,15685372,15685476,15685500,15685521,15685524,15685536,15685537,15685555,15685556,15685564,15685576,15685588,15685628,15685640,15685653,15685654,15685706,15685765,15685768,15685792,15685805,15685818,15685819,15685820,15685826,15685828,15685829,15685837,15685838,15685839,15685841,15685842,15685843,15685844,15685849,15685852,15685855,15685858,15685860,15685861,15685865,15685868,15685869,15685872,15685876,15685884,15685893,15685895,15685920,15685927,15685933,15685947,15685961,15685963,15685980,15685997,15686060,15686099,15686137,15686164,15686219,15686236,15686242,15686255,15686302,15686345,15686436,15686437,15686461,15686463,15686517,15686518,15686538,15686539,15686546,15686563,15686564,15686565,15686568,15686569,15686576,15686588,15686592,15686594,15686610,15686611,15686612,15686614,15686619,15686628,15686655,15686656,15686660,15686670,15686676,15686696,15686718,15686743,15686776,15686780,15686800,15686810,15686812,15686813,15686815,15686817,15686822,15686824,15686825,15686834,15686835,15686836,15686837,15686840,15686845,15686855,15686857,15686858,15686859,15686860,15686869,15686870,15686875,15686880,15686885,15686887,15686894,15686903,15686907,15686909,15686910,15686913,15686914,15686915,15686916,15686917,15686918,15686924,15686926,15686936,15686937,15686939,15686941,15686942,15686943,15686946,15686952,15686953,15686954,15686955,15686957,15686958,15686959,15686961,15686963,15686964,15686965,15686966,15686968,15686974,15686975,15686976,15686977,15686978,15686980,15686983,15686984,15686985,15686999,15687001,15687020,15687035,15687042,15687045,15687046,15687056,15687065,15687068,15687069,15687071,15687078,15687079,15687081,15687085,15687087,15687093,15687094,15687096,15687098,15687099,15687130,15687153,15687162,15687171,15687218,15687270,15687310,15687318,15687321,15687329,15687363,15687372,15687385,15687411,15687412,15687413,15687421,15687424,15687431,15687477,15687491,15687492,15687519,15687520,15687521,15687527,15687529,15687539,15687540,15687551,15687552,15687553,15687555,15687556,15687557,15687558,15687559,15687563,15687565,15687585,15687588,15687607,15687616,15687626,15687628,15687633,15687634,15687638,15687648,15687655,15687688,15687719,15687738,15687752,15687765,15687812,15687821,15687828,15687852,15687884,15687903,15687907,15687909,15687913,15687914,15687922,15687946,15687951,15687952,15687955,15687958,15687959,15687973,15688007,15688056,15688057,15688058,15688059,15688061,15688063,15688068,15688069,15688071,15688074,15688075,15688078,15688079,15688081,15688082,15688085,15688095,15688128,15688157,15688172,15688193,15688251,15688264,15688289,15688337,15688395,15688409,15688498,15688563,15688612,15688619,15688626,15688637,15688644,15688689,15688691,15688713,15688760,15688820,15688835,15688839,15688843,15688845,15688846,15688849,15688851,15688855,15688858,15688859,15688860,15688861,15688865,15688868,15688873,15688874,15688876,15688877,15688880,15688902,15688914,15688917,15688922,15688928,15688933,15688943,15688944,15688951,15688958,15688959,15688963,15688964,15688967,15688968,15688969,15688971,15688972,15688973,15688984,15689031,15689044,15689061,15689081,15689095,15689096,15689098,15689147,15689152,15689160,15689162,15689168,15689176,15689178,15689201,15689237,15689266,15689268,15689288,15689294,15689318,15689322,15689323,15689325,15689328,15689329,15689341,15689342,15689344,15689351,15689361,15689410,15689412,15689413,15689414,15689425,15689430,15689451,15689492,15689511,15689512,15689514,15689515,15689516,15689517,15689524,15689526,15689544,15689572,15689594,15689598,15689602,15689611,15689612,15689613,15689614,15689616,15689618,15689633,15689634,15689636,15689661,15689671,15689692,15689751,15689781,15689782,15689786,15689886,15689921,15689924,15689928,15689944,15689946,15689948,15689951,15689952,15689953,15689955,15689957,15689958,15689959,15689963,15689980,15690007,15690017,15690019,15690021,15690022,15690034,15690039,15690046,15690059,15690078,15690079,15690113,15690130,15690132,15690133,15690134,15690140,15690144,15690146,15690150,15690153,15690155,15690164,15690167,15690169,15690170,15690172,15690175,15690177,15690182,15690184,15690185,15690188,15690192,15690197,15690199,15690209,15690231,15690330,15690332,15690337,15690440,15690452,15690456,15690492,15690520,15690526,15690535,15690540,15690543,15690546,15690576,15690589,15690596,15690598,15690620,15690630,15690638,15690640,15690647,15690650,15690664,15690670,15690673,15690678,15690680,15690690,15690692,15690695,15690730,15690731,15690733,15690737,15690739,15690740,15690741,15690743,15690744,15690746,15690763,15690772,15690783,15690789,15690796,15690829,15690889,15690898,15690906,15690920,15690925,15690932,15690933,15690934,15690935,15690936,15690937,15690939,15690940,15690954,15690958,15690959,15690968,15691004,15691011,15691026,15691061,15691104,15691111,15691119,15691150,15691170,15691287,15691298,15691316,15691323,15691326,15691332,15691337,15691356,15691360,15691369,15691382,15691387,15691389,15691392,15691396,15691416,15691426,15691446,15691451,15691452,15691457,15691477,15691483,15691504,15691513,15691515,15691516,15691517,15691524,15691526,15691533,15691543,15691544,15691554,15691623,15691624,15691625,15691627,15691628,15691633,15691647,15691653,15691656,15691665,15691674,15691699,15691701,15691702,15691703,15691704,15691705,15691706,15691707,15691708,15691709,15691717,15691718,15691721,15691723,15691725,15691726,15691728,15691733,15691734,15691740,15691742,15691743,15691744,15691747,15691757,15691758,15691775,15691777,15691785,15691787,15691792,15691793,15691802,15691803,15691804,15691805,15691807,15691808,15691809,15691810,15691811,15691815,15691816,15691817,15691818,15691821,15691822,15691823,15691825,15691826,15691827,15691828,15691829,15691839,15691840,15691857,15691863,15691867,15691868,15691870,15691871,15691872,15691873,15691874,15691875,15691877,15691883,15691887,15691889,15691890,15691893,15691901,15691902,15691905,15691906,15691908,15691910,15691917,15691921,15691926,15691943,15691950,15691952,15691956,15692002,15692021,15692022,15692027,15692028,15692029,15692050,15692060,15692064,15692069,15692070,15692089,15692091,15692097,15692098,15692110,15692132,15692137,15692207,15692216,15692219,15692221,15692224,15692226,15692228,15692229,15692230,15692231,15692232,15692234,15692235,15692258,15692259,15692271,15692274,15692275,15692277,15692284,15692288,15692291,15692292,15692317,15692325,15692326,15692403,15692404,15692405,15692406,15692407,15692408,15692409,15692416,15692425,15692426,15692428,15692430,15692434,15692443,15692453,15692454,15692456,15692460,15692463,15692467,15692472,15692488,15692489,15692496,15692497,15692498,15692501,15692532,15692563,15692566,15692568,15692570,15692571,15692574,15692575,15692577,15692578,15692579,15692583,15692585,15692588,15692591,15692594,15692599,15692615,15692626,15692631,15692641,15692646,15692656,15692658,15692661,15692664,15692668,15692671,15692674,15692676,15692677,15692679,15692683,15692688,15692692,15692697,15692705,15692716,15692717,15692718,15692721,15692726,15692728,15692730,15692731,15692735,15692741,15692743,15692744,15692750,15692756,15692757,15692758,15692761,15692779,15692782,15692783,15692785,15692788,15692791,15692792,15692816,15692817,15692819,15692823,15692826,15692828,15692834,15692844,15692846,15692879,15692882,15692902,15692906,15692912,15692913,15692915,15692916,15692917,15692918,15692925,15692926,15692927,15692928,15692929,15692930,15692931,15692932,15692944,15692946,15692956,15692960,15692977,15692991,15692996,15693018,15693088,15693131,15693162,15693199,15693200,15693203,15693205,15693233,15693259,15693264,15693267,15693313,15693319,15693322,15693323,15693326,15693331,15693332,15693337,15693343,15693347,15693349,15693352,15693364,15693374,15693375,15693376,15693377,15693379,15693381,15693382,15693387,15693394,15693398,15693418,15693437,15693462,15693463,15693464,15693465,15693468,15693471,15693477,15693478,15693479,15693481,15693482,15693483,15693487,15693496,15693499,15693501,15693521,15693522,15693523,15693524,15693525,15693526,15693529,15693537,15693543,15693544,15693577,15693582,15693591,15693594,15693613,15693617,15693622,15693625,15693626,15693627,15693629,15693637,15693639,15693658,15693663,15693665,15693674,15693678,15693679,15693681,15693683,15693690,15693691,15693693,15693694,15693698,15693699,15693704,15693713,15693716,15693721,15693723,15693726,15693731,15693732,15693737,15693741,15693742,15693743,15693752,15693764,15693768,15693769,15693770,15693771,15693772,15693773,15693776,15693789,15693793,15693794,15693798,15693813,15693814,15693816,15693817,15693818,15693819,15693821,15693824,15693825,15693826,15693831,15693834,15693843,15693844,15693846,15693852,15693861,15693864,15693871,15693872,15693874,15693877,15693880,15693881,15693883,15693888,15693892,15693893,15693896,15693897,15693902,15693903,15693904,15693905,15693906,15693907,15693908,15693917,15693926,15693943,15693947,15693954,15693971,15693996,15694039,15694098,15694120,15694125,15694129,15694143,15694144,15694160,15694188,15694192,15694200,15694209,15694237,15694253,15694272,15694288,15694296,15694300,15694301,15694316,15694321,15694349,15694366,15694369,15694376,15694381,15694395,15694404,15694408,15694409,15694410,15694413,15694414,15694425,15694430,15694431,15694440,15694442,15694444,15694450,15694453,15694456,15694460,15694463,15694506,15694510,15694515,15694517,15694518,15694519,15694521,15694526,15694528,15694530,15694531,15694532,15694540,15694550,15694551,15694556,15694558,15694566,15694572,15694576,15694577,15694581,15694588,15694601,15694628,15694641,15694644,15694666,15694667,15694677,15694717,15694721,15694765,15694805,15694816,15694817,15694818,15694819,15694821,15694826,15694827,15694829,15694830,15694839,15694844,15694849,15694852,15694853,15694854,15694856,15694858,15694859,15694860,15694861,15694864,15694872,15694874,15694878,15694879,15694890,15694926,15694928,15694946,15694951,15694958,15694995,15695028,15695029,15695034,15695042,15695071,15695078,15695079,15695084,15695085,15695088,15695093,15695094,15695095,15695096,15695097,15695098,15695103,15695129,15695148,15695168,15695174,15695177,15695197,15695225,15695280,15695299,15695301,15695339,15695356,15695383,15695465,15695474,15695475,15695492,15695499,15695585,15695632,15695679,15695699,15695733,15695734,15695737,15695739,15695741,15695743,15695764,15695768,15695787,15695788,15695789,15695792,15695796,15695798,15695813,15695818,15695819,15695824,15695825,15695826,15695827,15695828,15695839,15695840,15695841,15695842,15695844,15695846,15695865,15695872,15695874,15695877,15695879,15695884,15695888,15695890,15695895,15695932,15696028,15696031,15696034,15696041,15696043,15696047,15696054,15696059,15696061,15696074,15696088,15696096,15696097,15696098,15696099,15696120,15696141,15696175,15696231,15696277,15696287,15696301,15696355,15696361,15696371,15696373,15696388,15696506,15696513,15696514,15696534,15696537,15696539,15696544,15696552,15696564,15696569,15696585,15696588,15696589,15696605,15696637,15696674,15696703,15696710,15696711,15696713,15696715,15696726,15696729,15696733,15696734,15696739,15696743,15696744,15696760,15696761,15696764,15696765,15696796,15696800,15696810,15696812,15696813,15696817,15696818,15696819,15696823,15696824,15696825,15696826,15696827,15696829,15696830,15696834,15696835,15696837,15696839,15696840,15696842,15696843,15696844,15696849,15696852,15696854,15696857,15696858,15696859,15696865,15696870,15696871,15696899,15696900,15696901,15696902,15696903,15696904,15696905,15696906,15696909,15696944,15696952,15696954,15696958,15696961,15696963,15696964,15696969,15696972,15696974,15696985,15696988,15696989,15697000,15697020,15697025,15697026,15697028,15697029,15697034,15697035,15697036,15697039,15697042,15697043,15697045,15697047,15697048,15697065,15697095,15697098,15697100,15697127,15697129,15697183,15697197,15697201,15697214,15697249,15697270,15697283,15697301,15697307,15697310,15697311,15697318,15697321,15697322,15697329,15697330,15697360,15697391,15697424,15697440,15697441,15697450,15697460,15697470,15697480,15697497,15697499,15697510,15697512,15697532,15697534,15697543,15697546,15697553,15697554,15697557,15697559,15697567,15697571,15697573,15697574,15697584,15697588,15697589,15697591,15697593,15697596,15697597,15697598,15697606,15697625,15697678,15697686,15697691,15697702,15697789,15697801,15697825,15697844,15697871,15697876,15697879,15697881,15697896,15697900,15697901,15697906,15697932,15697934,15697939,15697948,15697949,15697952,15697954,15697959,15697964,15697971,15697979,15697985,15697989,15697997,15698028,15698031,15698034,15698042,15698052,15698084,15698089,15698117,15698167,15698184,15698246,15698271,15698285,15698294,15698321,15698324,15698335,15698349,15698442,15698462,15698474,15698510,15698516,15698521,15698522,15698528,15698532,15698537,15698572,15698619,15698664,15698703,15698714,15698716,15698727,15698732,15698749,15698758,15698786,15698791,15698792,15698816,15698828,15698829,15698839,15698846,15698852,15698872,15698874,15698877,15698879,15698888,15698889,15698902,15698903,15698904,15698918,15698927,15698932,15698953,15698958,15698964,15698971,15698972,15699004,15699005,15699047,15699054,15699078,15699095,15699195,15699217,15699225,15699238,15699247,15699257,15699268,15699284,15699289,15699294,15699309,15699325,15699329,15699335,15699340,15699350,15699358,15699380,15699384,15699389,15699415,15699430,15699433,15699435,15699446,15699450,15699461,15699462,15699467,15699468,15699486,15699492,15699494,15699507,15699515,15699523,15699524,15699532,15699550,15699591,15699598,15699619,15699634,15699637,15699739,15699753,15699772,15699790,15699797,15699830,15699839,15699911,15699963,15700083,15700174,15700209,15700212,15700223,15700251,15700255,15700258,15700263,15700264,15700287,15700295,15700300,15700325,15700353,15700366,15700383,15700394,15700424,15700460,15700476,15700487,15700511,15700540,15700549,15700575,15700601,15700627,15700649,15700650,15700654,15700656,15700657,15700688,15700693,15700696,15700710,15700714,15700749,15700752,15700772,15700801,15700813,15700826,15700854,15700864,15700865,15700933,15700934,15700937,15700946,15700964,15700965,15701012,15701037,15701042,15701054,15701056,15701071,15701072,15701074,15701081,15701086,15701095,15701096,15701109,15701116,15701121,15701125,15701152,15701160,15701164,15701166,15701167,15701175,15701176,15701177,15701184,15701200,15701206,15701212,15701221,15701238,15701256,15701263,15701265,15701280,15701281,15701291,15701300,15701301,15701333,15701351,15701352,15701354,15701358,15701364,15701375,15701376,15701392,15701395,15701412,15701439,15701488,15701522,15701524,15701537,15701550,15701555,15701602,15701605,15701687,15701688,15701738,15701741,15701765,15701768,15701796,15701809,15701885,15701932,15701936,15701946,15701962,15701972,15702014,15702030,15702050,15702060,15702061,15702065,15702071,15702089,15702095,15702138,15702145,15702149,15702175,15702190,15702210,15702263,15702274,15702277,15702280,15702290,15702292,15702293,15702294,15702298,15702299,15702300,15702301,15702337,15702369,15702377,15702380,15702381,15702388,15702413,15702416,15702420,15702426,15702429,15702430,15702432,15702434,15702435,15702437,15702438,15702439,15702442,15702444,15702454,15702456,15702460,15702461,15702462,15702463,15702464,15702467,15702468,15702469,15702470,15702478,15702479,15702483,15702486,15702488,15702499,15702531,15702534,15702541,15702561,15702566,15702571,15702574,15702577,15702581,15702585,15702588,15702594,15702599,15702601,15702626,15702627,15702631,15702634,15702637,15702641,15702642,15702643,15702656,15702669,15702683,15702715,15702735,15702741,15702744,15702769,15702771,15702778,15702779,15702788,15702789,15702797,15702801,15702806,15702812,15702814,15702841,15702844,15702846,15702854,15702879,15702899,15702901,15702909,15702912,15702913,15702914,15702918,15702919,15702920,15702934,15702937,15702939,15702944,15702954,15702956,15702968,15702979,15702983,15702984,15702988,15702999,15703019,15703032,15703037,15703046,15703048,15703059,15703068,15703097,15703106,15703117,15703119,15703143,15703167,15703177,15703199,15703205,15703264,15703292,15703322,15703337,15703341,15703342,15703344,15703354,15703371,15703377,15703378,15703382,15703392,15703394,15703399,15703413,15703437,15703438,15703455,15703461,15703462,15703469,15703481,15703482,15703483,15703494,15703521,15703522,15703523,15703534,15703537,15703540,15703541,15703542,15703543,15703544,15703546,15703549,15703554,15703563,15703582,15703584,15703585,15703588,15703594,15703599,15703613,15703614,15703634,15703637,15703639,15703654,15703679,15703682,15703693,15703705,15703707,15703713,15703714,15703720,15703722,15703723,15703728,15703729,15703731,15703737,15703741,15703743,15703744,15703746,15703749,15703754,15703763,15703764,15703771,15703774,15703776,15703778,15703779,15703783,15703784,15703788,15703793,15703794,15703814,15703820,15703830,15703840,15703842,15703850,15703851,15703856,15703880,15703890,15703901,15703904,15703908,15703909,15703913,15703914,15703916,15703917,15703923,15703924,15703937,15703942,15703944,15703949,15703953,15703954,15703963,15703968,15703972,15703974,15703979,15703983,15703988,15704014,15704020,15704045,15704053,15704054,15704061,15704065,15704068,15704072,15704074,15704081,15704082,15704083,15704084,15704144,15704160,15704168,15704209,15704231,15704250,15704284,15704294,15704315,15704330,15704340,15704378,15704431,15704442,15704443,15704461,15704466,15704481,15704482,15704483,15704484,15704498,15704509,15704517,15704581,15704583,15704586,15704587,15704588,15704594,15704595,15704596,15704597,15704598,15704620,15704629,15704651,15704681,15704720,15704741,15704742,15704743,15704744,15704746,15704762,15704763,15704765,15704768,15704769,15704770,15704771,15704782,15704788,15704795,15704796,15704797,15704813,15704816,15704817,15704819,15704820,15704844,15704849,15704851,15704854,15704859,15704861,15704862,15704899,15704910,15704913,15704917,15704919,15704921,15704922,15704926,15704934,15704937,15704944,15704946,15704950,15704951,15704954,15704960,15704962,15704963,15704987,15705001,15705009,15705020,15705025,15705034,15705042,15705045,15705050,15705053,15705059,15705060,15705062,15705070,15705078,15705082,15705085,15705113,15705174,15705212,15705280,15705281,15705298,15705300,15705301,15705309,15705312,15705313,15705343,15705350,15705352,15705378,15705379,15705383,15705388,15705403,15705443,15705448,15705458,15705506,15705511,15705512,15705514,15705515,15705517,15705519,15705521,15705523,15705541,15705543,15705544,15705562,15705599,15705620,15705632,15705638,15705639,15705640,15705657,15705663,15705676,15705699,15705707,15705727,15705735,15705737,15705738,15705739,15705741,15705742,15705764,15705765,15705768,15705769,15705777,15705782,15705785,15705789,15705790,15705795,15705798,15705799,15705819,15705820,15705827,15705829,15705835,15705849,15705853,15705860,15705861,15705865,15705868,15705870,15705879,15705884,15705885,15705890,15705893,15705895,15705898,15705899,15705909,15705914,15705917,15705918,15705919,15705922,15705934,15705944,15705952,15705953,15705958,15705961,15705963,15705968,15705978,15705991,15705994,15706005,15706021,15706028,15706030,15706031,15706036,15706037,15706044,15706046,15706050,15706054,15706059,15706061,15706069,15706071,15706091,15706095,15706097,15706115,15706116,15706128,15706131,15706148,15706163,15706170,15706177,15706185,15706186,15706190,15706196,15706199,15706200,15706217,15706223,15706229,15706232,15706242,15706263,15706267,15706268,15706269,15706365,15706394,15706458,15706459,15706463,15706464,15706534,15706537,15706538,15706539,15706546,15706552,15706554,15706557,15706563,15706585,15706586,15706587,15706588,15706589,15706593,15706594,15706595,15706598,15706602,15706609,15706637,15706647,15706696,15706706,15706713,15706729,15706734,15706737,15706739,15706744,15706761,15706762,15706763,15706764,15706768,15706769,15706770,15706796,15706797,15706799,15706800,15706810,15706811,15706815,15706840,15706843,15706844,15706850,15706869,15706899,15706900,15706904,15706905,15706909,15706913,15706926,15706934,15706937,15706954,15706968,15707001,15707007,15707011,15707020,15707023,15707025,15707035,15707037,15707042,15707045,15707046,15707048,15707050,15707052,15707054,15707061,15707065,15707078,15707085,15707107,15707110,15707114,15707120,15707132,15707138,15707144,15707162,15707184,15707189,15707199,15707219,15707242,15707291,15707294,15707322,15707323,15707362,15707425,15707441,15707473,15707479,15707488,15707489,15707492,15707505,15707541,15707543,15707546,15707551,15707553,15707558,15707559,15707563,15707567,15707571,15707593,15707596,15707598,15707599,15707602,15707608,15707634,15707637,15707674,15707681,15707696,15707837,15707861,15707904,15707912,15707913,15707918,15707920,15707922,15707946,15707952,15707961,15707962,15707963,15707968,15707971,15707974,15707986,15707989,15708003,15708046,15708048,15708053,15708063,15708139,15708162,15708193,15708196,15708212,15708228,15708236,15708256,15708265,15708289,15708357,15708362,15708422,15708504,15708505,15708511,15708513,15708521,15708523,15708534,15708608,15708610,15708650,15708693,15708710,15708713,15708714,15708715,15708716,15708744,15708791,15708814,15708844,15708846,15708852,15708853,15708867,15708868,15708888,15708902,15708904,15708916,15708917,15708971,15709016,15709117,15709135,15709136,15709142,15709183,15709199,15709222,15709232,15709252,15709256,15709263,15709295,15709317,15709324,15709335,15709338,15709354,15709368,15709387,15709394,15709396,15709405,15709412,15709441,15709443,15709444,15709447,15709451,15709458,15709459,15709474,15709476,15709478,15709485,15709487,15709511,15709517,15709521,15709523,15709531,15709541,15709546,15709563,15709570,15709604,15709616,15709618,15709621,15709622,15709625,15709626,15709634,15709637,15709638,15709639,15709643,15709653,15709737,15709741,15709743,15709766,15709780,15709846,15709861,15709862,15709901,15709913,15709914,15709916,15709917,15709919,15709920,15709922,15709925,15709928,15709929,15709931,15709941,15709946,15709952,15709961,15709968,15709969,15709978,15709987,15709991,15709994,15709995,15709997,15710012,15710034,15710053,15710085,15710087,15710103,15710104,15710105,15710106,15710108,15710111,15710121,15710127,15710132,15710138,15710142,15710144,15710147,15710161,15710164,15710167,15710187,15710203,15710206,15710257,15710271,15710295,15710315,15710316,15710337,15710338,15710347,15710352,15710365,15710366,15710375,15710377,15710383,15710385,15710390,15710397,15710403,15710404,15710408,15710409,15710421,15710424,15710450,15710454,15710456,15710457,15710543,15710553,15710557,15710593,15710643,15710671,15710687,15710689,15710726,15710743,15710825,15710853,15710928,15710978,15711015,15711021,15711026,15711028,15711029,15711047,15711052,15711062,15711063,15711093,15711130,15711157,15711177,15711181,15711215,15711217,15711218,15711219,15711222,15711224,15711248,15711249,15711251,15711254,15711261,15711269,15711287,15711288,15711293,15711299,15711309,15711314,15711316,15711352,15711366,15711374,15711376,15711378,15711385,15711386,15711389,15711396,15711398,15711409,15711446,15711455,15711457,15711468,15711486,15711489,15711490,15711521,15711540,15711571,15711572,15711579,15711588,15711589,15711602,15711618,15711635,15711644,15711686,15711687,15711706,15711733,15711742,15711750,15711759,15711786,15711789,15711794,15711825,15711834,15711843,15711889,15711901,15711905,15711917,15711921,15711961,15711970,15711977,15711991,15712121,15712176,15712198,15712199,15712241,15712264,15712272,15712287,15712303,15712338,15712356,15712358,15712403,15712408,15712409,15712455,15712478,15712483,15712494,15712497,15712543,15712551,15712568,15712578,15712584,15712588,15712596,15712604,15712607,15712608,15712769,15712772,15712774,15712776,15712777,15712807,15712813,15712825,15712849,15712877,15712893,15712903,15712930,15712938,15712960,15713038,15713043,15713046,15713079,15713088,15713094,15713140,15713144,15713153,15713160,15713250,15713267,15713283,15713303,15713309,15713343,15713346,15713347,15713350,15713352,15713354,15713355,15713358,15713374,15713378,15713379,15713426,15713444,15713463,15713476,15713479,15713483,15713490,15713504,15713505,15713517,15713532,15713559,15713578,15713579,15713594,15713596,15713599,15713604,15713608,15713621,15713632,15713637,15713644,15713655,15713659,15713673,15713769,15713770,15713774,15713819,15713826,15713834,15713835,15713843,15713844,15713845,15713849,15713850,15713852,15713853,15713854,15713855,15713869,15713877,15713890,15713912,15713949,15713962,15713975,15713983,15714004,15714006,15714041,15714046,15714054,15714057,15714062,15714080,15714085,15714087,15714097,15714215,15714227,15714236,15714237,15714240,15714241,15714256,15714260,15714331,15714355,15714366,15714380,15714391,15714394,15714397,15714431,15714466,15714485,15714493,15714553,15714567,15714575,15714586,15714596,15714610,15714634,15714642,15714643,15714649,15714656,15714658,15714659,15714673,15714689,15714691,15714750,15714781,15714789,15714832,15714874,15714877,15714883,15714893,15714920,15714939,15714962,15714970,15714981,15714993,15715003,15715029,15715031,15715047,15715068,15715078,15715079,15715116,15715117,15715125,15715131,15715140,15715142,15715160,15715164,15715176,15715184,15715199,15715206,15715208,15715236,15715271,15715295,15715297,15715299,15715355,15715394,15715414,15715510,15715512,15715519,15715527,15715532,15715541,15715597,15715622,15715638,15715643,15715667,15715673,15715689,15715690,15715707,15715709,15715715,15715744,15715745,15715750,15715757,15715769,15715796,15715877,15715888,15715907,15715920,15715939,15715941,15715951,15715979,15715988,15715994,15716000,15716023,15716024,15716041,15716078,15716082,15716085,15716087,15716088,15716096,15716134,15716155,15716156,15716160,15716164,15716168,15716186,15716218,15716236,15716270,15716276,15716281,15716284,15716286,15716324,15716328,15716334,15716336,15716347,15716357,15716381,15716397,15716420,15716443,15716446,15716449,15716454,15716471,15716491,15716501,15716518,15716555,15716605,15716608,15716609,15716619,15716643,15716781,15716800,15716822,15716835,15716926,15716954,15716984,15716990,15716994,15716995,15717010,15717046,15717052,15717056,15717059,15717062,15717065,15717078,15717082,15717087,15717116,15717138,15717155,15717156,15717164,15717166,15717167,15717185,15717191,15717197,15717221,15717225,15717231,15717245,15717286,15717290,15717291,15717307,15717328,15717348,15717350,15717398,15717426,15717454,15717486,15717489,15717490,15717498,15717527,15717560,15717629,15717643,15717700,15717705,15717706,15717736,15717770,15717790,15717800,15717879,15717893,15717894,15717898,15717935,15717937,15717962,15717984,15717995,15718018,15718039,15718057,15718071,15718106,15718116,15718153,15718155,15718182,15718197,15718203,15718216,15718225,15718227,15718231,15718241,15718242,15718243,15718245,15718246,15718247,15718255,15718267,15718280,15718281,15718289,15718291,15718369,15718406,15718443,15718446,15718454,15718465,15718473,15718507,15718531,15718572,15718581,15718588,15718667,15718673,15718689,15718703,15718765,15718769,15718773,15718779,15718780,15718789,15718839,15718843,15718852,15718893,15718912,15718921,15718944,15719017,15719090,15719170,15719212,15719238,15719265,15719276,15719294,15719302,15719304,15719309,15719347,15719348,15719352,15719377,15719407,15719409,15719426,15719479,15719489,15719494,15719508,15719541,15719559,15719561,15719579,15719606,15719608,15719624,15719649,15719778,15719809,15719827,15719856,15719858,15719898,15719931,15719950,15719958,15719971,15719991,15720050,15720071,15720083,15720110,15720123,15720134,15720155,15720187,15720203,15720276,15720284,15720291,15720305,15720338,15720341,15720353,15720354,15720371,15720463,15720508,15720509,15720511,15720529,15720557,15720559,15720579,15720583,15720636,15720637,15720644,15720649,15720653,15720661,15720662,15720671,15720676,15720687,15720689,15720702,15720706,15720713,15720725,15720745,15720758,15720774,15720814,15720820,15720847,15720893,15720910,15720912,15720919,15720922,15720924,15720929,15720939,15720941,15720942,15720943,15720968,15720971,15720988,15721007,15721019,15721024,15721043,15721047,15721053,15721063,15721116,15721180,15721181,15721189,15721205,15721207,15721208,15721209,15721251,15721256,15721267,15721281,15721282,15721291,15721292,15721303,15721307,15721336,15721366,15721377,15721380,15721383,15721387,15721393,15721416,15721419,15721423,15721426,15721427,15721433,15721435,15721453,15721457,15721460,15721462,15721463,15721469,15721476,15721477,15721482,15721483,15721487,15721492,15721504,15721507,15721539,15721569,15721579,15721582,15721585,15721586,15721587,15721591,15721592,15721626,15721635,15721653,15721655,15721658,15721660,15721685,15721709,15721715,15721719,15721730,15721735,15721740,15721755,15721758,15721770,15721778,15721779,15721787,15721793,15721815,15721835,15721845,15721917,15721921,15721935,15721937,15721950,15721989,15721991,15722003,15722004,15722010,15722037,15722053,15722061,15722072,15722083,15722090,15722104,15722117,15722122,15722172,15722199,15722212,15722246,15722269,15722289,15722322,15722350,15722364,15722403,15722404,15722405,15722407,15722408,15722409,15722424,15722425,15722428,15722429,15722433,15722434,15722435,15722442,15722454,15722456,15722467,15722473,15722478,15722479,15722485,15722499,15722528,15722529,15722532,15722533,15722534,15722535,15722538,15722543,15722548,15722558,15722574,15722578,15722579,15722582,15722583,15722588,15722594,15722599,15722604,15722611,15722632,15722692,15722701,15722731,15722758,15722765,15722772,15722813,15722845,15722894,15722902,15722965,15722991,15722993,15723004,15723006,15723008,15723013,15723014,15723028,15723029,15723032,15723043,15723053,15723063,15723064,15723082,15723105,15723123,15723127,15723138,15723143,15723153,15723169,15723216,15723217,15723220,15723229,15723250,15723263,15723287,15723299,15723305,15723307,15723318,15723320,15723339,15723346,15723350,15723369,15723373,15723400,15723411,15723437,15723481,15723484,15723485,15723488,15723489,15723604,15723613,15723620,15723649,15723654,15723658,15723659,15723661,15723671,15723685,15723703,15723705,15723706,15723717,15723720,15723721,15723737,15723786,15723793,15723818,15723827,15723851,15723856,15723857,15723858,15723864,15723872,15723873,15723884,15723886,15723894,15723907,15723950,15723989,15724076,15724091,15724092,15724099,15724121,15724126,15724127,15724141,15724145,15724149,15724150,15724151,15724152,15724153,15724154,15724161,15724167,15724181,15724185,15724186,15724188,15724200,15724216,15724221,15724223,15724251,15724254,15724269,15724281,15724282,15724293,15724296,15724302,15724307,15724321,15724333,15724334,15724336,15724339,15724343,15724391,15724402,15724423,15724428,15724443,15724444,15724451,15724453,15724454,15724466,15724536,15724543,15724563,15724605,15724620,15724623,15724632,15724641,15724648,15724661,15724663,15724667,15724679,15724684,15724689,15724691,15724692,15724719,15724764,15724834,15724838,15724848,15724851,15724852,15724853,15724854,15724858,15724863,15724866,15724868,15724876,15724889,15724890,15724892,15724944,15724955,15725002,15725020,15725023,15725024,15725025,15725026,15725028,15725029,15725034,15725036,15725039,15725059,15725060,15725064,15725068,15725072,15725076,15725078,15725079,15725080,15725088,15725094,15725141,15725151,15725153,15725166,15725169,15725221,15725222,15725238,15725302,15725303,15725355,15725395,15725429,15725461,15725474,15725478,15725509,15725511,15725586,15725660,15725663,15725664,15725665,15725679,15725698,15725706,15725709,15725737,15725759,15725762,15725767,15725772,15725774,15725776,15725778,15725794,15725796,15725818,15725835,15725882,15725906,15725945,15725997,15726032,15726036,15726042,15726043,15726044,15726046,15726047,15726048,15726053,15726058,15726069,15726079,15726085,15726087,15726088,15726103,15726153,15726163,15726166,15726167,15726170,15726178,15726179,15726183,15726186,15726212,15726234,15726250,15726267,15726287,15726289,15726310,15726313,15726354,15726358,15726403,15726466,15726478,15726484,15726487,15726490,15726491,15726494,15726507,15726556,15726557,15726578,15726588,15726589,15726608,15726631,15726676,15726679,15726687,15726695,15726720,15726747,15726774,15726776,15726791,15726795,15726796,15726814,15726832,15726855,15726882,15726931,15726945,15726960,15727010,15727021,15727041,15727056,15727059,15727138,15727160,15727177,15727186,15727191,15727197,15727198,15727250,15727251,15727262,15727281,15727291,15727299,15727307,15727311,15727317,15727350,15727352,15727353,15727358,15727361,15727362,15727377,15727381,15727384,15727391,15727408,15727421,15727467,15727480,15727484,15727489,15727490,15727498,15727499,15727539,15727546,15727556,15727605,15727619,15727688,15727696,15727703,15727706,15727811,15727819,15727829,15727857,15727868,15727891,15727915,15727919,15727944,15728005,15728006,15728010,15728012,15728015,15728017,15728042,15728043,15728052,15728058,15728063,15728064,15728069,15728078,15728082,15728084,15728112,15728167,15728224,15728333,15728352,15728359,15728362,15728456,15728474,15728480,15728505,15728523,15728542,15728605,15728608,15728666,15728669,15728673,15728683,15728684,15728692,15728693,15728703,15728705,15728706,15728709,15728773,15728806,15728823,15728829,15728838,15728885,15728906,15728912,15728917,15728963,15729010,15729018,15729019,15729040,15729054,15729059,15729065,15729076,15729078,15729083,15729087,15729097,15729109,15729110,15729117,15729128,15729130,15729179,15729201,15729224,15729238,15729246,15729273,15729279,15729339,15729355,15729358,15729359,15729362,15729377,15729435,15729448,15729454,15729489,15729515,15729557,15729574,15729579,15729582,15729586,15729599,15729619,15729668,15729669,15729677,15729688,15729689,15729706,15729718,15729763,15729771,15729774,15729789,15729793,15729804,15729832,15729836,15729885,15729908,15729954,15729956,15729958,15729970,15729975,15729984,15729991,15730038,15730044,15730048,15730058,15730059,15730076,15730090,15730131,15730137,15730150,15730151,15730156,15730160,15730161,15730163,15730169,15730257,15730271,15730272,15730273,15730287,15730338,15730339,15730345,15730353,15730355,15730359,15730360,15730390,15730394,15730397,15730418,15730433,15730441,15730447,15730448,15730460,15730469,15730487,15730488,15730489,15730540,15730579,15730590,15730597,15730639,15730641,15730657,15730662,15730673,15730688,15730698,15730705,15730735,15730738,15730759,15730779,15730793,15730796,15730810,15730956,15730963,15731021,15731022,15731026,15731032,15731047,15731059,15731116,15731117,15731144,15731148,15731166,15731216,15731241,15731246,15731251,15731261,15731267,15731281,15731287,15731323,15731326,15731327,15731330,15731333,15731383,15731390,15731411,15731412,15731413,15731417,15731442,15731460,15731462,15731463,15731467,15731478,15731511,15731519,15731522,15731529,15731533,15731534,15731535,15731542,15731543,15731544,15731545,15731553,15731555,15731556,15731563,15731569,15731585,15731586,15731660,15731669,15731670,15731719,15731730,15731738,15731740,15731744,15731748,15731750,15731751,15731752,15731754,15731755,15731756,15731760,15731770,15731771,15731781,15731790,15731794,15731815,15731821,15731829,15731835,15731838,15731840,15731855,15731863,15731865,15731881,15731885,15731890,15731921,15731951,15731989,15732020,15732061,15732102,15732110,15732112,15732113,15732175,15732202,15732216,15732235,15732237,15732263,15732264,15732265,15732266,15732268,15732269,15732270,15732277,15732280,15732281,15732291,15732293,15732294,15732297,15732298,15732299,15732303,15732307,15732321,15732334,15732336,15732337,15732340,15732343,15732344,15732364,15732369,15732372,15732378,15732404,15732409,15732413,15732430,15732434,15732435,15732436,15732437,15732438,15732442,15732444,15732448,15732451,15732461,15732462,15732463,15732465,15732467,15732468,15732469,15732473,15732476,15732478,15732479,15732488,15732532,15732534,15732537,15732563,15732566,15732569,15732570,15732574,15732577,15732580,15732581,15732593,15732599,15732610,15732616,15732617,15732620,15732621,15732622,15732625,15732627,15732628,15732629,15732631,15732632,15732635,15732637,15732638,15732639,15732640,15732641,15732642,15732643,15732644,15732650,15732658,15732660,15732661,15732664,15732665,15732668,15732669,15732670,15732672,15732673,15732674,15732678,15732681,15732688,15732691,15732693,15732694,15732704,15732708,15732709,15732721,15732740,15732744,15732751,15732758,15732760,15732763,15732773,15732774,15732778,15732781,15732785,15732786,15732788,15732789,15732790,15732791,15732793,15732794,15732797,15732798,15732824,15732832,15732842,15732844,15732850,15732851,15732868,15732887,15732903,15732913,15732917,15732931,15732935,15732937,15732941,15732942,15732943,15732944,15732951,15732952,15732963,15732967,15732969,15732976,15732977,15732978,15732981,15732982,15732987,15732988,15732997,15733014,15733018,15733031,15733032,15733042,15733043,15733044,15733059,15733111,15733114,15733119,15733138,15733141,15733143,15733169,15733181,15733213,15733227,15733229,15733233,15733234,15733247,15733261,15733271,15733283,15733297,15733303,15733341,15733352,15733358,15733361,15733381,15733387,15733429,15733444,15733447,15733460,15733474,15733476,15733489,15733491,15733522,15733538,15733574,15733581,15733597,15733602,15733661,15733705,15733768,15733777,15733796,15733797,15733842,15733876,15733883,15733888,15733904,15733964,15733966,15733974,15733976,15734005,15734006,15734008,15734012,15734025,15734044,15734058,15734085,15734146,15734161,15734177,15734202,15734218,15734246,15734267,15734312,15734359,15734365,15734438,15734461,15734462,15734463,15734485,15734491,15734496,15734519,15734524,15734525,15734528,15734539,15734560,15734565,15734569,15734574,15734578,15734586,15734588,15734606,15734609,15734610,15734626,15734628,15734634,15734649,15734659,15734674,15734689,15734711,15734737,15734762,15734765,15734792,15734850,15734886,15734891,15734892,15734894,15734917,15734942,15734948,15734970,15734987,15734999,15735025,15735060,15735078,15735080,15735106,15735116,15735138,15735177,15735199,15735203,15735221,15735222,15735223,15735244,15735246,15735250,15735263,15735264,15735270,15735277,15735287,15735297,15735302,15735330,15735339,15735346,15735353,15735358,15735367,15735388,15735400,15735402,15735428,15735439,15735522,15735549,15735572,15735578,15735597,15735632,15735636,15735643,15735673,15735687,15735688,15735719,15735776,15735777,15735782,15735783,15735784,15735799,15735837,15735849,15735893,15735909,15735968,15736008,15736016,15736028,15736041,15736043,15736044,15736058,15736063,15736069,15736077,15736078,15736088,15736109,15736112,15736122,15736124,15736126,15736138,15736146,15736154,15736228,15736246,15736249,15736250,15736254,15736267,15736271,15736274,15736287,15736292,15736295,15736299,15736302,15736327,15736330,15736371,15736388,15736396,15736397,15736399,15736407,15736413,15736420,15736425,15736431,15736438,15736461,15736463,15736471,15736487,15736491,15736495,15736510,15736519,15736533,15736534,15736557,15736578,15736601,15736656,15736684,15736717,15736730,15736760,15736769,15736778,15736816,15736879,15736905,15736963,15736982,15736990,15736993,15736999,15737021,15737025,15737026,15737029,15737037,15737042,15737045,15737047,15737048,15737051,15737065,15737071,15737078,15737080,15737088,15737104,15737133,15737160,15737169,15737173,15737194,15737240,15737264,15737265,15737267,15737275,15737294,15737310,15737324,15737339,15737354,15737385,15737402,15737404,15737408,15737414,15737452,15737476,15737489,15737506,15737509,15737521,15737524,15737541,15737542,15737543,15737546,15737551,15737556,15737558,15737571,15737576,15737586,15737602,15737605,15737627,15737644,15737646,15737647,15737678,15737687,15737690,15737705,15737709,15737732,15737735,15737741,15737748,15737774,15737778,15737782,15737792,15737795,15737814,15737832,15737888,15737909,15737910,15737954,15737961,15737971,15737977,15737981,15737993,15737998,15738018,15738042,15738043,15738044,15738057,15738063,15738068,15738069,15738088,15738148,15738149,15738150,15738166,15738168,15738170,15738171,15738174,15738181,15738183,15738186,15738191,15738193,15738219,15738220,15738224,15738231,15738247,15738248,15738257,15738267,15738272,15738291,15738318,15738411,15738448,15738476,15738487,15738494,15738497,15738501,15738546,15738588,15738605,15738627,15738634,15738662,15738672,15738715,15738721,15738751,15738785,15738803,15738835,15738884,15738885,15738889,15738907,15738954,15738963,15738980,15738983,15739042,15739046,15739054,15739059,15739068,15739077,15739087,15739110,15739123,15739124,15739131,15739160,15739168,15739169,15739174,15739178,15739183,15739189,15739191,15739194,15739223,15739240,15739248,15739265,15739271,15739272,15739278,15739292,15739357,15739438,15739476,15739502,15739548,15739552,15739576,15739578,15739592,15739672,15739692,15739693,15739803,15739847,15739850,15739857,15739858,15739931,15739938,15739952,15739972,15739997,15740043,15740066,15740072,15740076,15740078,15740103,15740104,15740105,15740106,15740132,15740137,15740140,15740147,15740150,15740151,15740160,15740161,15740164,15740165,15740167,15740168,15740170,15740172,15740175,15740176,15740180,15740220,15740223,15740237,15740257,15740258,15740264,15740268,15740283,15740291,15740294,15740345,15740356,15740357,15740383,15740400,15740402,15740404,15740405,15740406,15740407,15740411,15740428,15740442,15740458,15740463,15740470,15740476,15740487,15740494,15740507,15740509,15740527,15740557,15740602,15740630,15740648,15740846,15740868,15740878,15740900,15741028,15741029,15741031,15741032,15741034,15741036,15741045,15741046,15741047,15741049,15741072,15741075,15741078,15741084,15741085,15741091,15741092,15741094,15741097,15741098,15741175,15741180,15741181,15741195,15741197,15741245,15741275,15741280,15741295,15741326,15741336,15741340,15741342,15741349,15741380,15741385,15741387,15741391,15741392,15741403,15741413,15741416,15741417,15741429,15741433,15741452,15741479,15741526,15741534,15741535,15741550,15741551,15741553,15741554,15741629,15741633,15741643,15741652,15741653,15741654,15741693,15741718,15741719,15741720,15741723,15741728,15741745,15741753,15741775,15741835,15741865,15741869,15741875,15741921,15741988,15742009,15742019,15742028,15742030,15742034,15742091,15742097,15742100,15742116,15742126,15742172,15742204,15742210,15742211,15742216,15742218,15742228,15742229,15742232,15742236,15742238,15742265,15742268,15742269,15742272,15742280,15742281,15742282,15742285,15742291,15742292,15742293,15742294,15742297,15742298,15742299,15742323,15742328,15742331,15742332,15742336,15742338,15742344,15742355,15742358,15742360,15742364,15742368,15742369,15742372,15742373,15742378,15742390,15742404,15742405,15742406,15742408,15742409,15742418,15742438,15742442,15742467,15742472,15742473,15742477,15742480,15742483,15742488,15742504,15742511,15742532,15742534,15742538,15742543,15742566,15742574,15742575,15742577,15742581,15742606,15742607,15742609,15742613,15742617,15742618,15742619,15742626,15742628,15742629,15742631,15742632,15742634,15742638,15742640,15742642,15742653,15742658,15742662,15742665,15742668,15742669,15742672,15742674,15742678,15742679,15742681,15742718,15742728,15742731,15742736,15742778,15742781,15742793,15742798,15742809,15742816,15742819,15742820,15742821,15742823,15742824,15742828,15742829,15742832,15742834,15742835,15742838,15742839,15742841,15742845,15742848,15742849,15742851,15742854,15742856,15742860,15742865,15742868,15742872,15742876,15742879,15742885,15742886,15742892,15742893,15742896,15742898,15742903,15742911,15742931,15742934,15742945,15742967,15742971,15742977,15742985,15743007,15743016,15743025,15743028,15743029,15743032,15743040,15743043,15743044,15743050,15743051,15743058,15743063,15743065,15743067,15743068,15743070,15743071,15743073,15743075,15743076,15743080,15743081,15743085,15743093,15743097,15743149,15743153,15743169,15743192,15743193,15743196,15743204,15743211,15743213,15743215,15743225,15743236,15743245,15743279,15743293,15743295,15743312,15743334,15743342,15743360,15743364,15743370,15743376,15743378,15743411,15743452,15743456,15743460,15743463,15743469,15743470,15743471,15743478,15743480,15743482,15743490,15743495,15743498,15743514,15743515,15743521,15743522,15743528,15743532,15743535,15743538,15743540,15743541,15743543,15743545,15743547,15743550,15743570,15743574,15743578,15743580,15743581,15743582,15743590,15743591,15743592,15743594,15743595,15743607,15743614,15743615,15743617,15743618,15743625,15743628,15743629,15743631,15743669,15743671,15743672,15743673,15743681,15743691,15743693,15743709,15743713,15743714,15743716,15743717,15743759,15743760,15743813,15743816,15743817,15743818,15743819,15743828,15743829,15743844,15743858,15743865,15743871,15743880,15743890,15743895,15743905,15743952,15743955,15743976,15743979,15743980,15744008,15744028,15744035,15744041,15744043,15744044,15744045,15744046,15744091,15744103,15744105,15744109,15744127,15744136,15744200,15744240,15744244,15744250,15744273,15744279,15744295,15744307,15744327,15744355,15744368,15744398,15744402,15744423,15744517,15744518,15744526,15744528,15744529,15744531,15744544,15744551,15744553,15744576,15744577,15744578,15744582,15744584,15744593,15744606,15744607,15744614,15744657,15744688,15744689,15744691,15744692,15744695,15744709,15744717,15744770,15744789,15744843,15744894,15744913,15744914,15744919,15744942,15744979,15744983,15745001,15745012,15745016,15745030,15745048,15745080,15745083,15745084,15745085,15745088,15745103,15745105,15745106,15745107,15745109,15745196,15745200,15745201,15745232,15745248,15745250,15745266,15745293,15745294,15745295,15745306,15745307,15745323,15745324,15745353,15745354,15745355,15745375,15745394,15745399,15745404,15745417,15745431,15745433,15745452,15745455,15745527,15745533,15745543,15745605,15745621,15745624,15745716,15745794,15745796,15745804,15745826,15745827,15745843,15745844,15746012,15746014,15746028,15746035,15746041,15746042,15746043,15746044,15746046,15746064,15746065,15746068,15746069,15746074,15746076,15746078,15746079,15746088,15746127,15746139,15746183,15746190,15746199,15746203,15746258,15746277,15746333,15746338,15746341,15746410,15746411,15746413,15746422,15746451,15746458,15746461,15746471,15746490,15746495,15746510,15746511,15746515,15746521,15746538,15746552,15746553,15746564,15746569,15746594,15746662,15746663,15746664,15746668,15746670,15746674,15746676,15746678,15746693,15746695,15746704,15746708,15746717,15746719,15746725,15746726,15746734,15746737,15746741,15746743,15746748,15746749,15746802,15746804,15746805,15746809,15746810,15746812,15746821,15746824,15746825,15746834,15746842,15746844,15746848,15746851,15746860,15746870,15746872,15746875,15746878,15746880,15746895,15746901,15746902,15746954,15746955,15746964,15746970,15746976,15746979,15746981,15746985,15746986,15746989,15746995,15747000,15747012,15747014,15747017,15747042,15747043,15747065,15747078,15747080,15747093,15747094,15747097,15747098,15747130,15747168,15747174,15747208,15747222,15747224,15747265,15747288,15747297,15747307,15747323,15747345,15747358,15747458,15747483,15747503,15747510,15747524,15747532,15747534,15747541,15747542,15747553,15747554,15747557,15747558,15747573,15747588,15747591,15747593,15747597,15747598,15747647,15747649,15747677,15747724,15747757,15747762,15747772,15747795,15747797,15747807,15747808,15747821,15747854,15747867,15747878,15747927,15747960,15747974,15747980,15748028,15748032,15748034,15748049,15748064,15748067,15748068,15748069,15748072,15748114,15748116,15748118,15748123,15748231,15748327,15748352,15748360,15748365,15748381,15748414,15748432,15748472,15748473,15748480,15748481,15748489,15748499,15748532,15748552,15748553,15748589,15748595,15748597,15748608,15748616,15748625,15748639,15748647,15748649,15748660,15748671,15748673,15748691,15748696,15748752,15748797,15748807,15748828,15748854,15748856,15748882,15748920,15748936,15748947,15748986,15748995,15749004,15749019,15749050,15749072,15749074,15749093,15749094,15749103,15749114,15749117,15749119,15749123,15749130,15749167,15749170,15749177,15749217,15749265,15749300,15749328,15749345,15749353,15749358,15749365,15749377,15749381,15749401,15749461,15749472,15749482,15749489,15749492,15749499,15749519,15749528,15749557,15749583,15749638,15749669,15749671,15749679,15749688,15749691,15749693,15749696,15749727,15749799,15749807,15749851,15749905,15749928,15749946,15749947,15749951,15749955,15749964,15749975,15749985,15750014,15750015,15750024,15750034,15750040,15750041,15750049,15750053,15750054,15750055,15750056,15750068,15750069,15750080,15750099,15750103,15750104,15750120,15750121,15750141,15750142,15750153,15750155,15750156,15750161,15750181,15750243,15750248,15750258,15750264,15750299,15750335,15750366,15750396,15750407,15750410,15750441,15750447,15750454,15750458,15750466,15750476,15750492,15750497,15750498,15750545,15750547,15750549,15750567,15750569,15750574,15750602,15750637,15750638,15750649,15750654,15750658,15750664,15750671,15750684,15750728,15750731,15750749,15750751,15750755,15750765,15750769,15750772,15750776,15750778,15750803,15750805,15750806,15750807,15750808,15750811,15750813,15750829,15750833,15750837,15750838,15750839,15750856,15750867,15750868,15750869,15750874,15750879,15750921,15750923,15750924,15750929,15750931,15750937,15750970,15751021,15751022,15751023,15751026,15751032,15751033,15751036,15751043,15751052,15751053,15751054,15751056,15751057,15751058,15751064,15751071,15751081,15751083,15751084,15751092,15751093,15751120,15751131,15751132,15751136,15751137,15751138,15751139,15751148,15751149,15751156,15751158,15751161,15751165,15751167,15751168,15751172,15751175,15751176,15751177,15751180,15751181,15751185,15751193,15751201,15751203,15751205,15751206,15751207,15751208,15751209,15751227,15751272,15751292,15751297,15751299,15751333,15751336,15751337,15751340,15751355,15751369,15751385,15751387,15751392,15751412,15751445,15751453,15751455,15751461,15751477,15751487,15751491,15751493,15751524,15751525,15751528,15751531,15751533,15751534,15751538,15751548,15751549,15751550,15751553,15751554,15751561,15751562,15751566,15751581,15751585,15751592,15751593,15751594,15751627,15751628,15751674,15751688,15751698,15751710,15751723,15751728,15751734,15751740,15751743,15751748,15751749,15751750,15751755,15751756,15751771,15751773,15751774,15751775,15751780,15751781,15751788,15751793,15751912,15751926,15751943,15751944,15752026,15752030,15752047,15752048,15752056,15752061,15752072,15752079,15752081,15752084,15752088,15752091,15752097,15752137,15752139,15752186,15752193,15752277,15752278,15752294,15752300,15752301,15752310,15752312,15752321,15752331,15752337,15752338,15752342,15752344,15752360,15752362,15752366,15752369,15752372,15752375,15752377,15752380,15752381,15752388,15752390,15752401,15752403,15752404,15752405,15752408,15752409,15752412,15752424,15752426,15752429,15752433,15752434,15752454,15752455,15752463,15752467,15752472,15752474,15752475,15752477,15752478,15752479,15752483,15752484,15752485,15752488,15752493,15752499,15752502,15752503,15752507,15752508,15752509,15752526,15752528,15752529,15752531,15752534,15752547,15752573,15752574,15752577,15752578,15752588,15752597,15752601,15752612,15752618,15752622,15752625,15752626,15752628,15752629,15752631,15752637,15752647,15752650,15752674,15752678,15752693,15752694,15752700,15752708,15752709,15752715,15752721,15752726,15752731,15752738,15752801,15752809,15752811,15752812,15752814,15752815,15752816,15752825,15752826,15752831,15752834,15752835,15752836,15752838,15752839,15752840,15752841,15752843,15752845,15752846,15752847,15752849,15752851,15752853,15752854,15752855,15752856,15752857,15752858,15752862,15752867,15752869,15752871,15752878,15752879,15752884,15752885,15752888,15752892,15752894,15752899,15752905,15752908,15752909,15752912,15752913,15752915,15752916,15752917,15752951,15752953,15752955,15752956,15752976,15752978,15752979,15752982,15752983,15752984,15752987,15752991,15752993,15752994,15752995,15752996,15752997,15752999,15753026,15753028,15753032,15753035,15753037,15753055,15753056,15753059,15753065,15753068,15753074,15753078,15753079,15753081,15753084,15753091,15753092,15753093,15753094,15753097,15753102,15753110,15753161,15753213,15753215,15753229,15753248,15753329,15753332,15753337,15753338,15753341,15753342,15753343,15753344,15753347,15753353,15753361,15753362,15753365,15753368,15753370,15753371,15753372,15753375,15753378,15753381,15753382,15753392,15753394,15753520,15753521,15753522,15753531,15753534,15753537,15753538,15753541,15753542,15753544,15753546,15753547,15753549,15753550,15753562,15753566,15753572,15753574,15753578,15753579,15753581,15753591,15753639,15753655,15753679,15753737,15753754,15753779,15753801,15753828,15753830,15753831,15753834,15753837,15753839,15753840,15753841,15753842,15753843,15753846,15753847,15753849,15753850,15753856,15753859,15753861,15753863,15753866,15753871,15753874,15753876,15753877,15753880,15753881,15753888,15753893,15753895,15753897,15753898,15753912,15753937,15753939,15753947,15753949,15753955,15753969,15753971,15753972,15753976,15753983,15754020,15754038,15754042,15754044,15754048,15754049,15754052,15754053,15754068,15754072,15754073,15754074,15754081,15754083,15754084,15754085,15754112,15754168,15754177,15754261,15754267,15754301,15754342,15754441,15754442,15754443,15754462,15754488,15754491,15754494,15754498,15754513,15754526,15754531,15754536,15754537,15754543,15754549,15754551,15754554,15754556,15754565,15754566,15754569,15754574,15754577,15754578,15754579,15754584,15754588,15754593,15754594,15754595,15754597,15754599,15754605,15754649,15754713,15754736,15754747,15754796,15754797,15754813,15754816,15754820,15754821,15754847,15754849,15754851,15754854,15754856,15754858,15754860,15754861,15754863,15754868,15754870,15754872,15754874,15754894,15754910,15754915,15754917,15754918,15754919,15754920,15754921,15754922,15754923,15754926,15754929,15754930,15754931,15754932,15754933,15754939,15754940,15754950,15754951,15754952,15754954,15754958,15754980,15754999,15755018,15755025,15755048,15755067,15755077,15755083,15755084,15755088,15755138,15755196,15755209,15755223,15755225,15755239,15755241,15755242,15755262,15755330,15755405,15755416,15755521,15755638,15755642,15755648,15755649,15755678,15755722,15755726,15755731,15755738,15755739,15755798,15755813,15755818,15755820,15755825,15755826,15755829,15755831,15755832,15755835,15755837,15755862,15755863,15755865,15755868,15755869,15755872,15755875,15755884,15755885,15755890,15755892,15755893,15755895,15755899,15755978,15756021,15756022,15756026,15756031,15756036,15756050,15756056,15756061,15756066,15756069,15756070,15756071,15756076,15756078,15756091,15756094,15756095,15756097,15756118,15756125,15756148,15756150,15756155,15756167,15756196,15756241,15756272,15756277,15756299,15756301,15756305,15756321,15756333,15756420,15756425,15756429,15756461,15756462,15756465,15756471,15756472,15756475,15756481,15756510,15756511,15756518,15756520,15756521,15756526,15756531,15756535,15756537,15756538,15756539,15756540,15756544,15756560,15756563,15756565,15756569,15756570,15756585,15756594,15756595,15756598,15756599,15756610,15756613,15756625,15756629,15756637,15756638,15756650,15756655,15756660,15756662,15756668,15756669,15756670,15756674,15756680,15756695,15756699,15756743,15756751,15756778,15756799,15756804,15756811,15756820,15756821,15756822,15756823,15756824,15756825,15756833,15756834,15756835,15756837,15756840,15756844,15756845,15756847,15756848,15756850,15756855,15756865,15756868,15756869,15756870,15756871,15756874,15756875,15756880,15756894,15756895,15756901,15756909,15756910,15756912,15756913,15756915,15756916,15756917,15756918,15756919,15756920,15756921,15756922,15756924,15756925,15756926,15756927,15756928,15756931,15756932,15756935,15756936,15756937,15756939,15756940,15756942,15756945,15756952,15756954,15756955,15756956,15756958,15756964,15756969,15756974,15756975,15756992,15757015,15757140,15757188,15757306,15757310,15757385,15757408,15757425,15757433,15757434,15757467,15757473,15757478,15757521,15757531,15757533,15757534,15757535,15757537,15757538,15757539,15757540,15757541,15757543,15757547,15757549,15757551,15757553,15757556,15757557,15757558,15757559,15757566,15757567,15757568,15757571,15757572,15757576,15757577,15757579,15757588,15757595,15757597,15757599,15757607,15757608,15757616,15757617,15757618,15757628,15757629,15757632,15757633,15757634,15757638,15757648,15757661,15757711,15757735,15757737,15757738,15757747,15757759,15757761,15757771,15757772,15757773,15757777,15757778,15757788,15757797,15757810,15757811,15757812,15757820,15757821,15757825,15757826,15757827,15757829,15757831,15757832,15757837,15757861,15757865,15757866,15757867,15757871,15757872,15757874,15757876,15757877,15757891,15757893,15757895,15757896,15757897,15757910,15757911,15757912,15757913,15757916,15757919,15757921,15757922,15757923,15757925,15757929,15757931,15757932,15757933,15757939,15757941,15757952,15757958,15757959,15757962,15757971,15757981,15758013,15758020,15758021,15758023,15758024,15758025,15758026,15758028,15758031,15758032,15758033,15758034,15758035,15758038,15758039,15758043,15758045,15758046,15758048,15758049,15758050,15758051,15758052,15758053,15758055,15758056,15758059,15758063,15758067,15758071,15758073,15758074,15758077,15758081,15758085,15758095,15758116,15758136,15758171,15758252,15758306,15758362,15758372,15758449,15758451,15758477,15758510,15758511,15758513,15758514,15758526,15758528,15758531,15758534,15758537,15758538,15758541,15758546,15758566,15758606,15758639,15758685,15758692,15758693,15758702,15758725,15758726,15758731,15758749,15758750,15758755,15758766,15758767,15758769,15758775,15758780,15758813,15758815,15758828,15758831,15758835,15758836,15758837,15758838,15758839,15758843,15758845,15758846,15758847,15758849,15758851,15758852,15758854,15758856,15758868,15758869,15758872,15758873,15758874,15758881,15758888,15758895,15758901,15758939,15759021,15759029,15759034,15759038,15759039,15759048,15759049,15759066,15759067,15759069,15759075,15759133,15759140,15759184,15759244,15759250,15759274,15759284,15759290,15759298,15759310,15759317,15759322,15759381,15759387,15759399,15759401,15759410,15759416,15759436,15759456,15759467,15759475,15759480,15759499,15759516,15759531,15759532,15759537,15759541,15759555,15759566,15759567,15759618,15759684,15759690,15759717,15759731,15759733,15759735,15759737,15759741,15759751,15759805,15759869,15759872,15759874,15759892,15759901,15759911,15759913,15759914,15759915,15759917,15759919,15759931,15759937,15759941,15759953,15759955,15759966,15759974,15759981,15759991,15760043,15760060,15760063,15760075,15760085,15760090,15760093,15760094,15760095,15760102,15760104,15760105,15760121,15760153,15760155,15760161,15760177,15760216,15760234,15760244,15760264,15760271,15760294,15760394,15760427,15760431,15760446,15760450,15760455,15760456,15760514,15760521,15760524,15760550,15760568,15760570,15760571,15760641,15760646,15760662,15760749,15760774,15760849,15760851,15760861,15760865,15760873,15760880,15760988,15761018,15761023,15761024,15761026,15761029,15761031,15761032,15761043,15761045,15761047,15761048,15761053,15761054,15761055,15761057,15761058,15761073,15761075,15761076,15761083,15761094,15761102,15761117,15761128,15761131,15761132,15761136,15761141,15761148,15761149,15761151,15761154,15761157,15761158,15761162,15761168,15761180,15761205,15761206,15761207,15761208,15761241,15761246,15761247,15761248,15761276,15761286,15761287,15761290,15761291,15761292,15761294,15761295,15761297,15761298,15761332,15761333,15761340,15761344,15761347,15761348,15761364,15761374,15761387,15761417,15761453,15761455,15761457,15761472,15761477,15761482,15761483,15761487,15761491,15761492,15761493,15761497,15761506,15761525,15761534,15761542,15761550,15761553,15761554,15761600,15761606,15761607,15761620,15761621,15761628,15761632,15761654,15761655,15761663,15761670,15761672,15761674,15761675,15761679,15761685,15761692,15761693,15761694,15761695,15761702,15761703,15761704,15761705,15761706,15761708,15761713,15761716,15761717,15761718,15761720,15761721,15761723,15761725,15761726,15761728,15761730,15761732,15761733,15761736,15761738,15761745,15761750,15761753,15761754,15761763,15761764,15761768,15761773,15761774,15761775,15761783,15761793,15761826,15761835,15761851,15761854,15761871,15761886,15761916,15761921,15761927,15761950,15761951,15761976,15761986,15762021,15762028,15762045,15762047,15762048,15762091,15762098,15762110,15762123,15762134,15762138,15762169,15762172,15762198,15762216,15762218,15762228,15762236,15762238,15762268,15762277,15762298,15762322,15762328,15762331,15762332,15762342,15762349,15762351,15762354,15762361,15762362,15762372,15762381,15762391,15762392,15762394,15762397,15762403,15762404,15762405,15762409,15762414,15762416,15762418,15762423,15762455,15762462,15762463,15762475,15762482,15762488,15762499,15762515,15762532,15762533,15762543,15762566,15762573,15762574,15762577,15762578,15762583,15762588,15762597,15762601,15762605,15762613,15762615,15762618,15762626,15762628,15762632,15762638,15762642,15762650,15762662,15762674,15762682,15762688,15762698,15762704,15762705,15762707,15762708,15762709,15762715,15762716,15762718,15762728,15762729,15762731,15762732,15762733,15762738,15762741,15762745,15762748,15762760,15762762,15762764,15762768,15762773,15762782,15762783,15762789,15762792,15762793,15762796,15762797,15762798,15762799,15762802,15762803,15762816,15762821,15762822,15762823,15762825,15762826,15762829,15762832,15762839,15762841,15762854,15762855,15762856,15762857,15762858,15762882,15762883,15762884,15762885,15762886,15762891,15762892,15762896,15762897,15762902,15762908,15762913,15762916,15762917,15762922,15762927,15762928,15762929,15762932,15762937,15762951,15762955,15762957,15762976,15762977,15762979,15762984,15762987,15762991,15762992,15762993,15762994,15762997,15763023,15763027,15763029,15763031,15763033,15763048,15763053,15763055,15763063,15763065,15763067,15763083,15763092,15763093,15763095,15763097,15763107,15763108,15763111,15763148,15763171,15763194,15763215,15763217,15763218,15763274,15763301,15763317,15763318,15763326,15763327,15763328,15763330,15763347,15763381,15763383,15763394,15763397,15763412,15763414,15763415,15763416,15763418,15763421,15763429,15763431,15763460,15763478,15763482,15763487,15763514,15763515,15763520,15763538,15763544,15763545,15763547,15763577,15763579,15763612,15763613,15763614,15763616,15763617,15763618,15763620,15763625,15763629,15763639,15763662,15763665,15763679,15763683,15763691,15763693,15763697,15763704,15763706,15763743,15763747,15763760,15763773,15763776,15763779,15763783,15763784,15763798,15763806,15763808,15763809,15763826,15763857,15763859,15763861,15763873,15763874,15763877,15763895,15763897,15763898,15763902,15763903,15763904,15763906,15763907,15763922,15763923,15763927,15763966,15763980,15764021,15764023,15764028,15764033,15764047,15764048,15764067,15764068,15764072,15764073,15764105,15764146,15764153,15764170,15764174,15764185,15764195,15764204,15764209,15764226,15764227,15764237,15764246,15764253,15764270,15764284,15764293,15764294,15764295,15764314,15764335,15764339,15764343,15764351,15764405,15764409,15764419,15764431,15764444,15764448,15764491,15764498,15764537,15764604,15764609,15764618,15764654,15764661,15764671,15764681,15764708,15764726,15764793,15764841,15764892,15764916,15764922,15764927,15765014,15765036,15765043,15765047,15765048,15765060,15765083,15765087,15765093,15765097,15765163,15765173,15765192,15765197,15765257,15765258,15765263,15765266,15765282,15765283,15765287,15765297,15765300,15765306,15765311,15765322,15765328,15765340,15765345,15765375,15765402,15765404,15765405,15765415,15765432,15765457,15765461,15765476,15765487,15765518,15765520,15765531,15765532,15765537,15765643,15765650,15765694,15765706,15765732,15765806,15765812,15765846,15765857,15765952,15765982,15765990,15766006,15766017,15766043,15766044,15766047,15766048,15766064,15766066,15766137,15766139,15766163,15766166,15766167,15766172,15766173,15766181,15766183,15766185,15766186,15766192,15766205,15766253,15766270,15766272,15766288,15766289,15766295,15766355,15766374,15766378,15766389,15766415,15766417,15766420,15766421,15766422,15766427,15766428,15766458,15766467,15766490,15766552,15766563,15766575,15766609,15766649,15766659,15766663,15766667,15766682,15766683,15766686,15766689,15766708,15766734,15766741,15766746,15766765,15766776,15766787,15766807,15766810,15766826,15766869,15766887,15766896,15766906,15766908,15767064,15767066,15767129,15767138,15767231,15767264,15767294,15767333,15767339,15767358,15767367,15767383,15767391,15767414,15767432,15767442,15767474,15767488,15767498,15767588,15767593,15767594,15767597,15767598,15767626,15767681,15767722,15767757,15767774,15767777,15767781,15767793,15767794,15767795,15767804,15767818,15767821,15767864,15767871,15767884,15767889,15767891,15767893,15767894,15767896,15767921,15767954,15767994,15767995,15768006,15768025,15768072,15768095,15768098,15768100,15768104,15768106,15768111,15768120,15768124,15768137,15768150,15768151,15768153,15768162,15768163,15768164,15768193,15768201,15768219,15768233,15768244,15768246,15768267,15768270,15768282,15768289,15768293,15768295,15768300,15768339,15768340,15768342,15768346,15768355,15768359,15768367,15768387,15768409,15768444,15768449,15768451,15768455,15768465,15768471,15768474,15768476,15768487,15768495,15768530,15768566,15768574,15768595,15768600,15768682,15768683,15768689,15768746,15768763,15768777,15768816,15768881,15768885,15768887,15768945,15768963,15769005,15769006,15769042,15769051,15769110,15769117,15769125,15769169,15769191,15769216,15769222,15769238,15769246,15769248,15769254,15769272,15769302,15769308,15769312,15769346,15769353,15769356,15769357,15769364,15769375,15769381,15769389,15769394,15769402,15769406,15769412,15769451,15769499,15769504,15769516,15769522,15769548,15769571,15769572,15769582,15769586,15769590,15769596,15769641,15769645,15769703,15769781,15769783,15769818,15769829,15769902,15769903,15769905,15769907,15769913,15769915,15769917,15769945,15769947,15769948,15769955,15769957,15769959,15769964,15769974,15769975,15769977,15769980,15769984,15769987,15769991,15770030,15770033,15770039,15770041,15770051,15770053,15770069,15770091,15770101,15770103,15770104,15770121,15770130,15770131,15770141,15770143,15770147,15770154,15770159,15770170,15770172,15770174,15770177,15770180,15770181,15770185,15770188,15770193,15770194,15770205,15770209,15770214,15770225,15770255,15770258,15770291,15770306,15770309,15770345,15770354,15770375,15770404,15770405,15770406,15770420,15770424,15770435,15770474,15770476,15770498,15770508,15770516,15770525,15770539,15770543,15770546,15770548,15770552,15770554,15770565,15770567,15770576,15770631,15770711,15770719,15770811,15770961,15770968,15770974,15770984,15770995,15771000,15771029,15771030,15771031,15771034,15771053,15771054,15771058,15771059,15771064,15771070,15771071,15771081,15771086,15771087,15771096,15771137,15771139,15771141,15771143,15771144,15771148,15771149,15771154,15771179,15771210,15771211,15771219,15771222,15771223,15771227,15771229,15771234,15771236,15771240,15771241,15771242,15771249,15771251,15771261,15771267,15771268,15771270,15771272,15771281,15771284,15771285,15771286,15771287,15771290,15771291,15771292,15771299,15771323,15771330,15771333,15771334,15771344,15771372,15771373,15771383,15771384,15771389,15771390,15771403,15771408,15771409,15771412,15771415,15771417,15771429,15771433,15771436,15771442,15771443,15771451,15771457,15771467,15771471,15771473,15771477,15771482,15771483,15771487,15771490,15771491,15771509,15771510,15771519,15771522,15771529,15771530,15771534,15771535,15771536,15771537,15771539,15771540,15771542,15771543,15771549,15771550,15771569,15771570,15771571,15771573,15771577,15771580,15771582,15771584,15771586,15771591,15771592,15771599,15771629,15771636,15771640,15771642,15771655,15771666,15771668,15771669,15771671,15771673,15771674,15771698,15771702,15771704,15771706,15771708,15771716,15771722,15771728,15771729,15771733,15771734,15771738,15771741,15771742,15771747,15771749,15771750,15771754,15771756,15771757,15771758,15771769,15771770,15771772,15771773,15771774,15771798,15771802,15771804,15771816,15771829,15771835,15771836,15771838,15771841,15771843,15771850,15771854,15771855,15771856,15771857,15771867,15771871,15771873,15771883,15771886,15771930,15771942,15771966,15771977,15771997,15772009,15772028,15772030,15772050,15772051,15772054,15772063,15772071,15772073,15772148,15772215,15772216,15772217,15772222,15772228,15772231,15772232,15772236,15772237,15772238,15772250,15772262,15772274,15772283,15772293,15772299,15772328,15772329,15772330,15772331,15772333,15772336,15772337,15772338,15772341,15772343,15772351,15772356,15772360,15772361,15772363,15772373,15772374,15772377,15772379,15772381,15772383,15772390,15772393,15772394,15772398,15772399,15772401,15772404,15772405,15772409,15772412,15772415,15772417,15772421,15772422,15772423,15772425,15772428,15772429,15772430,15772431,15772433,15772434,15772441,15772442,15772447,15772456,15772461,15772466,15772467,15772472,15772473,15772474,15772475,15772479,15772481,15772482,15772483,15772488,15772489,15772491,15772492,15772497,15772499,15772501,15772503,15772510,15772525,15772528,15772529,15772531,15772532,15772542,15772547,15772550,15772551,15772557,15772558,15772559,15772571,15772572,15772573,15772574,15772576,15772577,15772583,15772601,15772604,15772615,15772616,15772618,15772626,15772627,15772628,15772630,15772631,15772632,15772646,15772650,15772658,15772662,15772708,15772736,15772741,15772743,15772751,15772760,15772761,15772762,15772773,15772777,15772778,15772781,15772785,15772786,15772792,15772798,15772799,15772801,15772826,15772834,15772835,15772842,15772846,15772854,15772862,15772882,15772885,15772886,15772887,15772891,15772892,15772893,15772895,15772896,15772898,15772901,15772909,15772922,15772926,15772928,15772933,15772939,15772941,15772942,15772955,15772958,15772967,15772973,15772977,15772979,15772982,15772983,15772986,15772989,15772991,15772992,15772993,15772994,15772995,15772996,15772997,15773013,15773017,15773029,15773039,15773044,15773045,15773049,15773059,15773071,15773073,15773077,15773079,15773081,15773082,15773090,15773091,15773094,15773096,15773097,15773098,15773146,15773242,15773260,15773273,15773276,15773277,15773283,15773322,15773323,15773337,15773338,15773344,15773347,15773348,15773351,15773378,15773382,15773383,15773384,15773389,15773391,15773392,15773394,15773398,15773421,15773431,15773437,15773438,15773439,15773447,15773456,15773467,15773469,15773471,15773479,15773481,15773487,15773489,15773491,15773501,15773503,15773512,15773520,15773529,15773530,15773539,15773542,15773550,15773569,15773570,15773576,15773578,15773582,15773590,15773591,15773594,15773598,15773605,15773612,15773617,15773629,15773638,15773639,15773672,15773679,15773693,15773709,15773714,15773723,15773731,15773741,15773747,15773749,15773751,15773769,15773776,15773778,15773779,15773782,15773783,15773789,15773790,15773792,15773793,15773796,15773798,15773801,15773803,15773804,15773809,15773820,15773828,15773829,15773830,15773831,15773836,15773843,15773851,15773852,15773854,15773858,15773859,15773866,15773868,15773869,15773871,15773873,15773876,15773878,15773880,15773883,15773886,15773889,15773890,15773891,15773893,15773894,15773895,15773896,15773897,15773898,15773904,15773906,15773909,15773922,15773923,15773929,15773934,15773936,15773939,15773942,15773946,15773949,15773969,15773971,15773972,15773973,15773977,15774098,15774103,15774104,15774127,15774133,15774151,15774155,15774157,15774164,15774166,15774179,15774192,15774224,15774250,15774258,15774262,15774277,15774285,15774316,15774328,15774336,15774358,15774382,15774393,15774401,15774402,15774439,15774491,15774507,15774510,15774516,15774526,15774530,15774531,15774536,15774537,15774539,15774551,15774552,15774554,15774556,15774586,15774591,15774594,15774630,15774696,15774720,15774722,15774723,15774727,15774729,15774731,15774732,15774733,15774736,15774737,15774738,15774743,15774744,15774747,15774748,15774749,15774772,15774794,15774796,15774829,15774833,15774834,15774835,15774836,15774838,15774839,15774842,15774847,15774849,15774851,15774852,15774854,15774856,15774857,15774858,15774859,15774861,15774868,15774869,15774872,15774878,15774882,15774895,15774899,15774921,15774928,15774931,15774936,15774940,15775041,15775042,15775047,15775048,15775050,15775054,15775067,15775072,15775075,15775082,15775102,15775104,15775105,15775108,15775116,15775123,15775124,15775126,15775131,15775153,15775185,15775192,15775203,15775206,15775231,15775233,15775234,15775235,15775238,15775247,15775254,15775293,15775294,15775295,15775299,15775306,15775307,15775318,15775335,15775339,15775430,15775433,15775490,15775510,15775562,15775572,15775590,15775605,15775625,15775627,15775650,15775662,15775663,15775676,15775678,15775691,15775703,15775705,15775709,15775720,15775741,15775750,15775756,15775761,15775787,15775794,15775797,15775803,15775804,15775806,15775807,15775809,15775826,15775880,15775886,15775888,15775891,15775892,15775905,15775912,15775949,15775990,15776046,15776047,15776048,15776051,15776069,15776071,15776076,15776079,15776087,15776091,15776096,15776098,15776128,15776156,15776211,15776223,15776229,15776231,15776233,15776348,15776404,15776421,15776422,15776427,15776429,15776433,15776434,15776454,15776467,15776494,15776495,15776518,15776521,15776522,15776523,15776526,15776539,15776545,15776547,15776558,15776569,15776575,15776576,15776596,15776597,15776598,15776605,15776621,15776624,15776629,15776631,15776644,15776701,15776712,15776713,15776715,15776717,15776723,15776725,15776729,15776733,15776734,15776739,15776743,15776745,15776765,15776767,15776780,15776785,15776807,15776810,15776811,15776822,15776824,15776825,15776827,15776844,15776845,15776849,15776850,15776851,15776854,15776855,15776859,15776880,15776890,15776921,15776922,15776925,15776929,15776931,15776934,15776935,15776936,15776937,15776939,15776942,15776945,15776946,15776947,15776951,15776954,15776955,15776969,15776975,15776985,15776986,15776987,15776989,15777025,15777033,15777034,15777041,15777045,15777046,15777047,15777049,15777050,15777051,15777052,15777054,15777055,15777056,15777058,15777060,15777064,15777065,15777067,15777075,15777076,15777079,15777081,15777082,15777087,15777097,15777122,15777179,15777210,15777211,15777214,15777217,15777224,15777315,15777352,15777436,15777452,15777459,15777504,15777522,15777529,15777539,15777542,15777550,15777551,15777553,15777555,15777573,15777576,15777579,15777585,15777586,15777588,15777589,15777591,15777594,15777595,15777596,15777597,15777598,15777599,15777610,15777614,15777615,15777616,15777639,15777708,15777738,15777739,15777743,15777759,15777772,15777777,15777778,15777783,15777784,15777785,15777786,15777794,15777796,15777797,15777798,15777826,15777827,15777828,15777830,15777835,15777853,15777854,15777855,15777873,15777891,15777892,15777893,15777894,15777895,15777896,15777897,15777898,15777901,15777903,15777904,15777905,15777906,15777907,15777908,15777909,15777921,15777922,15777923,15777924,15777928,15777958,15777959,15777961,15777983,15777986,15777994,15778005,15778022,15778024,15778025,15778033,15778034,15778044,15778046,15778048,15778049,15778056,15778068,15778072,15778073,15778085,15778086,15778089,15778091,15778096,15778103,15778104,15778135,15778140,15778142,15778149,15778154,15778155,15778157,15778158,15778190,15778192,15778196,15778219,15778287,15778290,15778294,15778304,15778320,15778345,15778347,15778368,15778395,15778410,15778418,15778463,15778481,15778489,15778515,15778526,15778528,15778545,15778550,15778589,15778604,15778605,15778694,15778696,15778701,15778715,15778727,15778738,15778744,15778750,15778752,15778825,15778830,15778831,15778835,15778837,15778838,15778839,15778843,15778845,15778851,15778854,15778856,15778858,15778861,15778862,15778865,15778868,15778880,15778881,15778882,15778883,15778884,15778885,15778886,15778891,15778893,15778897,15778899,15778904,15778915,15778929,15778930,15778932,15778934,15778935,15778936,15778938,15778943,15778944,15778945,15778947,15778948,15778955,15778958,15778959,15778960,15778966,15778972,15778974,15778975,15778980,15779052,15779059,15779064,15779103,15779133,15779166,15779174,15779176,15779182,15779183,15779207,15779247,15779267,15779390,15779411,15779414,15779423,15779481,15779492,15779496,15779497,15779515,15779522,15779523,15779526,15779531,15779544,15779547,15779550,15779570,15779581,15779584,15779586,15779589,15779611,15779620,15779627,15779659,15779711,15779714,15779722,15779733,15779738,15779741,15779742,15779743,15779744,15779799,15779809,15779823,15779861,15779862,15779865,15779886,15779903,15779904,15779905,15779906,15779908,15779913,15779914,15779915,15779924,15779931,15779947,15779953,15779963,15779969,15779973,15779974,15779983,15779984,15779985,15779991,15779992,15779993,15779994,15779996,15779997,15779998,15780019,15780021,15780022,15780024,15780030,15780033,15780034,15780037,15780038,15780039,15780041,15780042,15780043,15780045,15780056,15780059,15780063,15780065,15780078,15780088,15780093,15780104,15780106,15780112,15780121,15780122,15780124,15780128,15780129,15780130,15780133,15780134,15780136,15780139,15780140,15780141,15780142,15780144,15780145,15780146,15780150,15780167,15780188,15780212,15780215,15780219,15780220,15780224,15780234,15780254,15780256,15780359,15780362,15780386,15780389,15780409,15780411,15780432,15780449,15780492,15780498,15780514,15780554,15780561,15780572,15780587,15780602,15780650,15780654,15780662,15780677,15780682,15780716,15780719,15780746,15780770,15780779,15780804,15780805,15780835,15780846,15780870,15780909,15780925,15780954,15780961,15780984,15780989,15781023,15781034,15781036,15781127,15781129,15781131,15781207,15781210,15781218,15781219,15781220,15781222,15781223,15781227,15781228,15781229,15781234,15781236,15781241,15781242,15781250,15781251,15781267,15781272,15781273,15781274,15781276,15781280,15781297,15781298,15781299,15781307,15781316,15781323,15781326,15781327,15781330,15781334,15781335,15781342,15781347,15781365,15781377,15781389,15781405,15781406,15781409,15781410,15781419,15781430,15781440,15781443,15781446,15781451,15781452,15781453,15781454,15781455,15781457,15781459,15781460,15781462,15781465,15781473,15781481,15781486,15781487,15781489,15781495,15781496,15781497,15781498,15781505,15781525,15781530,15781533,15781534,15781535,15781542,15781550,15781553,15781554,15781559,15781563,15781574,15781577,15781579,15781582,15781586,15781589,15781590,15781593,15781594,15781598,15781599,15781608,15781614,15781618,15781619,15781625,15781626,15781627,15781653,15781654,15781659,15781665,15781668,15781674,15781677,15781678,15781679,15781689,15781690,15781698,15781700,15781702,15781710,15781719,15781769,15781777,15781778,15781790,15781802,15781822,15781835,15781855,15781865,15781870,15781871,15781873,15781874,15781875,15781877,15781878,15781879,15781882,15781884,15781885,15781893,15781903,15781914,15781975,15781987,15781989,15782089,15782096,15782100,15782121,15782144,15782159,15782181,15782210,15782211,15782212,15782219,15782222,15782224,15782225,15782226,15782231,15782235,15782236,15782247,15782268,15782299,15782300,15782310,15782311,15782330,15782334,15782335,15782336,15782341,15782342,15782343,15782351,15782356,15782360,15782363,15782364,15782369,15782380,15782390,15782399,15782402,15782403,15782404,15782405,15782408,15782413,15782418,15782419,15782427,15782429,15782430,15782434,15782442,15782453,15782454,15782456,15782458,15782459,15782463,15782467,15782468,15782469,15782472,15782473,15782474,15782475,15782478,15782479,15782488,15782489,15782498,15782499,15782519,15782530,15782531,15782534,15782538,15782541,15782550,15782559,15782560,15782566,15782568,15782569,15782572,15782574,15782578,15782579,15782585,15782586,15782589,15782591,15782594,15782598,15782599,15782608,15782615,15782625,15782630,15782631,15782641,15782642,15782650,15782658,15782659,15782660,15782688,15782692,15782718,15782719,15782728,15782730,15782734,15782735,15782736,15782741,15782746,15782754,15782756,15782758,15782760,15782768,15782778,15782789,15782793,15782801,15782802,15782805,15782806,15782809,15782825,15782830,15782834,15782835,15782840,15782846,15782849,15782850,15782854,15782855,15782856,15782857,15782859,15782860,15782865,15782868,15782875,15782876,15782879,15782899,15782903,15782905,15782906,15782909,15782912,15782914,15782918,15782919,15782923,15782926,15782929,15782930,15782933,15782939,15782941,15782942,15782943,15782945,15782950,15782955,15782958,15782963,15782967,15782968,15782977,15782988,15782989,15782992,15782993,15782994,15782997,15782998,15782999,15783007,15783019,15783020,15783022,15783023,15783025,15783026,15783027,15783028,15783029,15783030,15783031,15783033,15783034,15783039,15783040,15783042,15783059,15783060,15783063,15783065,15783068,15783073,15783083,15783090,15783093,15783097,15783099,15783225,15783276,15783290,15783305,15783322,15783337,15783338,15783339,15783341,15783342,15783345,15783349,15783369,15783378,15783383,15783392,15783393,15783398,15783415,15783444,15783462,15783463,15783477,15783478,15783487,15783489,15783501,15783511,15783521,15783522,15783523,15783525,15783526,15783528,15783538,15783539,15783540,15783541,15783542,15783544,15783546,15783550,15783555,15783558,15783579,15783582,15783588,15783589,15783591,15783592,15783593,15783594,15783601,15783610,15783614,15783615,15783617,15783625,15783627,15783629,15783630,15783632,15783659,15783678,15783679,15783683,15783688,15783691,15783696,15783715,15783718,15783723,15783752,15783764,15783770,15783789,15783801,15783816,15783817,15783819,15783826,15783835,15783840,15783842,15783849,15783850,15783855,15783856,15783857,15783859,15783861,15783862,15783865,15783871,15783875,15783878,15783880,15783881,15783882,15783883,15783893,15783897,15783901,15783902,15783903,15783954,15783955,15783958,15783959,15783962,15783979,15784000,15784010,15784041,15784042,15784051,15784053,15784056,15784058,15784078,15784082,15784084,15784088,15784091,15784092,15784096,15784097,15784098,15784099,15784124,15784130,15784148,15784161,15784180,15784192,15784209,15784218,15784280,15784286,15784301,15784318,15784320,15784321,15784323,15784361,15784445,15784491,15784495,15784497,15784526,15784530,15784557,15784558,15784566,15784586,15784590,15784591,15784592,15784593,15784594,15784596,15784597,15784599,15784676,15784687,15784700,15784733,15784736,15784737,15784738,15784742,15784749,15784760,15784761,15784762,15784769,15784780,15784788,15784792,15784796,15784832,15784833,15784844,15784849,15784852,15784854,15784859,15784861,15784862,15784865,15784868,15784878,15784880,15784890,15784896,15784918,15784923,15784950,15784958,15785024,15785034,15785041,15785042,15785047,15785074,15785076,15785078,15785080,15785085,15785092,15785167,15785170,15785183,15785215,15785290,15785326,15785330,15785339,15785342,15785350,15785352,15785358,15785367,15785373,15785385,15785406,15785485,15785488,15785489,15785490,15785511,15785518,15785519,15785521,15785522,15785526,15785529,15785531,15785532,15785534,15785536,15785542,15785547,15785559,15785563,15785564,15785576,15785586,15785588,15785591,15785592,15785596,15785654,15785662,15785705,15785766,15785782,15785794,15785798,15785811,15785815,15785818,15785819,15785823,15785825,15785829,15785830,15785831,15785834,15785835,15785837,15785839,15785842,15785849,15785859,15785860,15785863,15785865,15785868,15785869,15785870,15785875,15785876,15785878,15785879,15785880,15785891,15785892,15785899,15785910,15785920,15785975,15785980,15786014,15786025,15786026,15786027,15786034,15786035,15786042,15786043,15786061,15786063,15786068,15786069,15786070,15786076,15786078,15786088,15786153,15786167,15786170,15786196,15786199,15786200,15786209,15786215,15786237,15786249,15786308,15786389,15786392,15786395,15786410,15786417,15786418,15786426,15786428,15786429,15786433,15786438,15786454,15786460,15786463,15786469,15786471,15786488,15786519,15786522,15786523,15786526,15786527,15786539,15786544,15786546,15786563,15786569,15786570,15786589,15786594,15786610,15786611,15786612,15786615,15786617,15786618,15786619,15786622,15786629,15786638,15786643,15786655,15786669,15786670,15786676,15786680,15786717,15786718,15786738,15786739,15786789,15786792,15786800,15786810,15786842,15786845,15786847,15786850,15786852,15786855,15786859,15786870,15786880,15786904,15786905,15786909,15786919,15786959,15786989,15786993,15787014,15787025,15787026,15787035,15787042,15787046,15787051,15787059,15787071,15787081,15787151,15787155,15787161,15787168,15787174,15787189,15787196,15787204,15787222,15787227,15787229,15787255,15787258,15787318,15787322,15787459,15787470,15787510,15787519,15787529,15787530,15787534,15787536,15787550,15787553,15787556,15787558,15787559,15787563,15787567,15787570,15787589,15787591,15787593,15787595,15787596,15787597,15787598,15787599,15787602,15787611,15787618,15787619,15787627,15787655,15787693,15787699,15787710,15787756,15787772,15787788,15787810,15787811,15787825,15787826,15787827,15787829,15787830,15787835,15787837,15787838,15787848,15787866,15787875,15787884,15787903,15787905,15787907,15788020,15788022,15788025,15788028,15788033,15788042,15788051,15788052,15788058,15788067,15788068,15788085,15788090,15788096,15788118,15788123,15788126,15788131,15788151,15788153,15788183,15788186,15788189,15788215,15788218,15788219,15788224,15788291,15788367,15788442,15788448,15788480,15788483,15788494,15788539,15788556,15788634,15788659,15788676,15788683,15788686,15788775,15788776,15788797,15788806,15788920,15788930,15788946,15789014,15789055,15789095,15789097,15789109,15789158,15789201,15789247,15789313,15789334,15789339,15789371,15789379,15789410,15789413,15789414,15789418,15789425,15789430,15789432,15789436,15789439,15789460,15789475,15789480,15789484,15789498,15789520,15789523,15789530,15789541,15789546,15789548,15789550,15789557,15789560,15789562,15789563,15789582,15789611,15789669,15789714,15789753,15789800,15789815,15789819,15789834,15789853,15789858,15789859,15789861,15789862,15789863,15789865,15789872,15789874,15789891,15789895,15789899,15789904,15789909,15789914,15789915,15789919,15789924,15789925,15789936,15789939,15789941,15789942,15789946,15789947,15789955,15789972,15789974,15790011,15790013,15790017,15790019,15790022,15790024,15790029,15790030,15790047,15790059,15790066,15790067,15790074,15790078,15790079,15790090,15790093,15790097,15790113,15790204,15790227,15790235,15790247,15790254,15790267,15790282,15790299,15790314,15790322,15790324,15790327,15790336,15790339,15790348,15790355,15790379,15790412,15790442,15790448,15790449,15790455,15790489,15790497,15790564,15790588,15790594,15790595,15790596,15790599,15790630,15790658,15790659,15790670,15790678,15790683,15790689,15790716,15790717,15790729,15790730,15790732,15790733,15790744,15790748,15790750,15790757,15790762,15790763,15790764,15790770,15790782,15790796,15790809,15790810,15790811,15790822,15790829,15790830,15790835,15790839,15790846,15790860,15790929,15790935,15790953,15790958,15790966,15790968,15791006,15791026,15791029,15791030,15791031,15791032,15791033,15791034,15791035,15791038,15791039,15791040,15791041,15791043,15791044,15791045,15791047,15791050,15791052,15791053,15791054,15791059,15791070,15791102,15791111,15791114,15791117,15791131,15791132,15791139,15791141,15791148,15791149,15791150,15791154,15791167,15791169,15791170,15791172,15791173,15791174,15791177,15791179,15791181,15791191,15791216,15791217,15791218,15791220,15791221,15791234,15791241,15791246,15791249,15791253,15791264,15791267,15791268,15791270,15791272,15791273,15791274,15791279,15791280,15791290,15791302,15791304,15791310,15791313,15791316,15791317,15791321,15791323,15791324,15791326,15791329,15791330,15791333,15791336,15791342,15791343,15791344,15791346,15791347,15791356,15791357,15791359,15791360,15791365,15791366,15791369,15791373,15791379,15791387,15791452,15791501,15791513,15791516,15791517,15791519,15791525,15791530,15791533,15791534,15791535,15791549,15791550,15791563,15791565,15791569,15791593,15791625,15791627,15791629,15791644,15791647,15791649,15791665,15791667,15791668,15791670,15791674,15791679,15791681,15791700,15791738,15791739,15791740,15791742,15791743,15791744,15791747,15791748,15791749,15791750,15791759,15791760,15791765,15791767,15791768,15791769,15791770,15791771,15791773,15791774,15791775,15791777,15791778,15791779,15791780,15791788,15791817,15791823,15791824,15791829,15791834,15791836,15791837,15791842,15791847,15791848,15791850,15791851,15791852,15791854,15791856,15791867,15791871,15791873,15791875,15791877,15791879,15791881,15791912,15791925,15791927,15791928,15791932,15791943,15791944,15791947,15791949,15791958,15791977,15791988,15791991,15792004,15792008,15792028,15792029,15792064,15792077,15792102,15792107,15792110,15792151,15792176,15792180,15792222,15792225,15792226,15792227,15792228,15792229,15792230,15792231,15792232,15792233,15792250,15792272,15792275,15792281,15792305,15792321,15792323,15792324,15792325,15792326,15792327,15792328,15792329,15792330,15792338,15792342,15792344,15792349,15792350,15792358,15792360,15792363,15792364,15792365,15792366,15792367,15792368,15792369,15792370,15792373,15792375,15792377,15792378,15792381,15792388,15792390,15792408,15792413,15792416,15792417,15792418,15792423,15792425,15792430,15792442,15792453,15792454,15792460,15792463,15792464,15792465,15792466,15792467,15792468,15792469,15792472,15792473,15792474,15792475,15792477,15792479,15792481,15792483,15792486,15792488,15792489,15792497,15792522,15792525,15792531,15792532,15792543,15792549,15792550,15792565,15792566,15792567,15792570,15792574,15792575,15792577,15792578,15792581,15792588,15792601,15792605,15792607,15792615,15792620,15792625,15792626,15792627,15792628,15792629,15792630,15792631,15792632,15792636,15792640,15792641,15792642,15792646,15792647,15792649,15792650,15792654,15792659,15792660,15792664,15792665,15792666,15792667,15792668,15792669,15792670,15792674,15792677,15792679,15792699,15792714,15792716,15792717,15792718,15792720,15792721,15792722,15792726,15792727,15792728,15792729,15792730,15792731,15792736,15792741,15792743,15792748,15792750,15792760,15792773,15792774,15792781,15792788,15792791,15792812,15792818,15792819,15792824,15792825,15792826,15792838,15792846,15792850,15792854,15792862,15792864,15792868,15792869,15792877,15792879,15792882,15792913,15792914,15792915,15792916,15792917,15792918,15792919,15792922,15792923,15792925,15792926,15792927,15792929,15792931,15792934,15792937,15792941,15792945,15792950,15792951,15792954,15792966,15792967,15792968,15792969,15792979,15792983,15792984,15792985,15792986,15792988,15792989,15792991,15792992,15792993,15792994,15792995,15792996,15792999,15793024,15793026,15793028,15793029,15793030,15793034,15793039,15793042,15793043,15793044,15793046,15793048,15793049,15793053,15793063,15793067,15793068,15793070,15793077,15793078,15793085,15793090,15793095,15793097,15793116,15793135,15793196,15793197,15793247,15793307,15793311,15793313,15793314,15793316,15793317,15793318,15793321,15793322,15793323,15793324,15793325,15793326,15793327,15793328,15793329,15793331,15793333,15793335,15793337,15793338,15793339,15793341,15793342,15793343,15793345,15793346,15793347,15793349,15793351,15793365,15793366,15793379,15793410,15793413,15793415,15793417,15793421,15793424,15793425,15793426,15793433,15793450,15793455,15793460,15793463,15793465,15793468,15793469,15793471,15793477,15793478,15793479,15793481,15793487,15793491,15793492,15793498,15793525,15793544,15793581,15793585,15793617,15793625,15793629,15793641,15793650,15793665,15793668,15793671,15793678,15793679,15793680,15793682,15793683,15793685,15793688,15793690,15793691,15793692,15793693,15793694,15793695,15793696,15793714,15793715,15793716,15793722,15793723,15793724,15793726,15793739,15793760,15793764,15793772,15793790,15793793,15793801,15793803,15793811,15793813,15793814,15793816,15793817,15793818,15793823,15793824,15793825,15793826,15793828,15793829,15793831,15793841,15793842,15793843,15793846,15793847,15793849,15793850,15793851,15793852,15793853,15793854,15793856,15793857,15793858,15793859,15793861,15793865,15793868,15793871,15793873,15793874,15793876,15793877,15793878,15793879,15793880,15793881,15793882,15793883,15793884,15793888,15793890,15793892,15793893,15793895,15793896,15793897,15793898,15793901,15793905,15793906,15793913,15793923,15793929,15793931,15793939,15793941,15793943,15793944,15793946,15793949,15793952,15793954,15793958,15793960,15793969,15793971,15793973,15793978,15793979,15793988,15794014,15794048,15794049,15794053,15794056,15794101,15794122,15794137,15794142,15794159,15794171,15794178,15794182,15794187,15794204,15794236,15794253,15794257,15794261,15794266,15794273,15794276,15794278,15794297,15794323,15794345,15794356,15794360,15794396,15794401,15794413,15794414,15794416,15794424,15794427,15794442,15794448,15794450,15794460,15794462,15794467,15794468,15794469,15794472,15794475,15794477,15794478,15794479,15794480,15794491,15794493,15794494,15794495,15794496,15794497,15794498,15794510,15794513,15794515,15794525,15794530,15794536,15794539,15794549,15794550,15794560,15794564,15794566,15794568,15794570,15794577,15794579,15794580,15794581,15794588,15794593,15794661,15794792,15794811,15794813,15794819,15794820,15794825,15794839,15794841,15794842,15794844,15794846,15794847,15794849,15794850,15794851,15794853,15794854,15794860,15794864,15794865,15794867,15794868,15794869,15794870,15794874,15794875,15794878,15794880,15794888,15794890,15794895,15794896,15794902,15794915,15794916,15794917,15794918,15794925,15794929,15794931,15794932,15794939,15794940,15794941,15794947,15794949,15794950,15794959,15794960,15794962,15794964,15794969,15794970,15794975,15794980,15795079,15795124,15795129,15795132,15795149,15795151,15795166,15795180,15795186,15795224,15795228,15795246,15795275,15795290,15795298,15795370,15795429,15795458,15795510,15795511,15795517,15795519,15795521,15795522,15795523,15795524,15795527,15795531,15795532,15795536,15795537,15795539,15795540,15795541,15795542,15795543,15795546,15795547,15795560,15795562,15795564,15795565,15795566,15795569,15795570,15795571,15795572,15795574,15795576,15795578,15795580,15795586,15795588,15795599,15795640,15795690,15795719,15795723,15795732,15795734,15795735,15795737,15795738,15795739,15795742,15795747,15795749,15795759,15795765,15795766,15795767,15795788,15795831,15795839,15795865,15795868,15795869,15795872,15795874,15795875,15795878,15795879,15795881,15795882,15795884,15795885,15795890,15795893,15795895,15795896,15795901,15795903,15795906,15795923,15795927,15795931,15795932,15795933,15795934,15795935,15795936,15795937,15795950,15795953,15795954,15795956,15795957,15795958,15795959,15795960,15795961,15795963,15795964,15795965,15795968,15795975,15795978,15795980,15795983,15795990,15796015,15796065,15796067,15796069,15796071,15796073,15796074,15796076,15796081,15796095,15796101,15796111,15796115,15796167,15796179,15796218,15796219,15796230,15796231,15796310,15796313,15796328,15796329,15796334,15796336,15796343,15796351,15796360,15796370,15796413,15796414,15796417,15796420,15796423,15796425,15796428,15796429,15796434,15796449,15796450,15796463,15796465,15796467,15796470,15796471,15796479,15796480,15796481,15796483,15796489,15796490,15796493,15796495,15796496,15796498,15796517,15796537,15796564,15796569,15796612,15796613,15796617,15796638,15796650,15796660,15796670,15796671,15796680,15796711,15796713,15796716,15796717,15796721,15796722,15796723,15796724,15796727,15796728,15796729,15796731,15796733,15796739,15796747,15796749,15796750,15796764,15796767,15796769,15796771,15796787,15796790,15796793,15796796,15796810,15796811,15796812,15796813,15796814,15796819,15796822,15796823,15796824,15796829,15796834,15796835,15796838,15796839,15796843,15796846,15796849,15796852,15796864,15796868,15796869,15796874,15796890,15796901,15796906,15796910,15796913,15796918,15796926,15796930,15796942,15796944,15796945,15796956,15796957,15796958,15796959,15796963,15796964,15796965,15796968,15796969,15796970,15796974,15796975,15796979,15796983,15796985,15796993,15797065,15797081,15797113,15797141,15797142,15797149,15797160,15797165,15797167,15797188,15797190,15797194,15797197,15797204,15797219,15797227,15797246,15797276,15797290,15797293,15797318,15797324,15797329,15797381,15797389,15797424,15797475,15797502,15797595,15797627,15797665,15797686,15797692,15797709,15797710,15797713,15797720,15797722,15797723,15797725,15797729,15797733,15797736,15797737,15797738,15797739,15797740,15797747,15797748,15797750,15797751,15797760,15797764,15797765,15797767,15797768,15797770,15797772,15797773,15797777,15797779,15797787,15797796,15797822,15797828,15797900,15797903,15797904,15797905,15797906,15797909,15797910,15797911,15797913,15797918,15797919,15797923,15797925,15797930,15797932,15797933,15797938,15797948,15797949,15797960,15797961,15797962,15797964,15797986,15797989,15798024,15798025,15798028,15798032,15798034,15798046,15798048,15798053,15798054,15798058,15798064,15798079,15798084,15798200,15798256,15798310,15798341,15798348,15798385,15798398,15798410,15798422,15798424,15798428,15798429,15798435,15798439,15798460,15798470,15798510,15798513,15798516,15798521,15798522,15798523,15798524,15798528,15798532,15798560,15798566,15798605,15798615,15798659,15798709,15798737,15798749,15798750,15798769,15798813,15798816,15798824,15798825,15798828,15798834,15798835,15798837,15798838,15798844,15798846,15798849,15798850,15798851,15798853,15798854,15798856,15798860,15798868,15798872,15798874,15798884,15798885,15798888,15798890,15798892,15798894,15798895,15798896,15798897,15798903,15798906,15798915,15798916,15798930,15798932,15798935,15798942,15798943,15798948,15798949,15798958,15798959,15798960,15798964,15798970,15798972,15798975,15798980,15799031,15799042,15799128,15799156,15799186,15799211,15799217,15799294,15799300,15799350,15799357,15799358,15799377,15799384,15799386,15799389,15799403,15799422,15799435,15799450,15799468,15799472,15799479,15799480,15799488,15799489,15799494,15799515,15799555,15799600,15799631,15799635,15799641,15799649,15799652,15799653,15799720,15799763,15799785,15799790,15799811,15799815,15799859,15799910,15799924,15799925,15799932,15799964,15799966,15799998,15800031,15800043,15800049,15800061,15800062,15800068,15800083,15800116,15800212,15800215,15800223,15800227,15800228,15800229,15800233,15800251,15800268,15800280,15800287,15800295,15800326,15800366,15800412,15800434,15800440,15800482,15800488,15800514,15800515,15800517,15800524,15800554,15800583,15800620,15800688,15800703,15800736,15800746,15800772,15800773,15800784,15800793,15800805,15800806,15800811,15800812,15800814,15800842,15800845,15800856,15800861,15800874,15800890,15800961,15800965,15801062,15801072,15801074,15801125,15801152,15801169,15801175,15801184,15801188,15801189,15801206,15801218,15801247,15801256,15801263,15801265,15801277,15801293,15801296,15801316,15801336,15801338,15801351,15801352,15801376,15801392,15801395,15801411,15801414,15801417,15801418,15801431,15801436,15801438,15801441,15801462,15801466,15801473,15801488,15801490,15801548,15801550,15801559,15801566,15801605,15801627,15801630,15801658,15801699,15801701,15801767,15801768,15801788,15801811,15801817,15801821,15801824,15801832,15801842,15801863,15801904,15801915,15801920,15801922,15801924,15801932,15801934,15801942,15801946,15801994,15802009,15802030,15802041,15802050,15802054,15802060,15802061,15802069,15802070,15802071,15802077,15802081,15802086,15802088,15802089,15802106,15802118,15802207,15802220,15802256,15802265,15802268,15802274,15802277,15802290,15802342,15802351,15802360,15802361,15802362,15802364,15802368,15802369,15802370,15802375,15802377,15802378,15802381,15802390,15802463,15802466,15802467,15802468,15802478,15802481,15802483,15802484,15802486,15802487,15802488,15802489,15802501,15802555,15802560,15802568,15802570,15802575,15802585,15802588,15802593,15802617,15802625,15802641,15802669,15802715,15802722,15802728,15802729,15802731,15802735,15802741,15802751,15802757,15802758,15802778,15802785,15802788,15802789,15802801,15802809,15802817,15802822,15802823,15802835,15802839,15802848,15802853,15802856,15802857,15802860,15802868,15802869,15802885,15802909,15802912,15802917,15802955,15803005,15803032,15803037,15803038,15803046,15803053,15803060,15803064,15803066,15803067,15803068,15803075,15803078,15803121,15803136,15803202,15803322,15803337,15803338,15803339,15803347,15803349,15803352,15803353,15803358,15803365,15803368,15803378,15803386,15803406,15803413,15803415,15803439,15803450,15803455,15803456,15803457,15803462,15803465,15803468,15803475,15803479,15803481,15803486,15803501,15803522,15803524,15803526,15803533,15803536,15803538,15803541,15803544,15803556,15803585,15803588,15803633,15803651,15803654,15803679,15803681,15803682,15803683,15803684,15803685,15803686,15803689,15803691,15803696,15803706,15803714,15803716,15803726,15803732,15803742,15803747,15803748,15803749,15803750,15803760,15803763,15803764,15803768,15803773,15803778,15803789,15803790,15803793,15803801,15803804,15803810,15803813,15803814,15803815,15803816,15803817,15803820,15803825,15803826,15803827,15803831,15803832,15803837,15803840,15803841,15803842,15803846,15803849,15803850,15803852,15803865,15803868,15803876,15803877,15803879,15803880,15803881,15803882,15803886,15803888,15803896,15803901,15803906,15803907,15803908,15803909,15803923,15803925,15803929,15803941,15803942,15803946,15803947,15803949,15803954,15803955,15803956,15803957,15803964,15803965,15803966,15803968,15803976,15803977,15803978,15803979,15803982,15804002,15804003,15804005,15804006,15804009,15804017,15804023,15804025,15804038,15804039,15804042,15804044,15804045,15804048,15804052,15804053,15804054,15804057,15804058,15804062,15804064,15804068,15804072,15804075,15804078,15804082,15804093,15804095,15804104,15804131,15804142,15804150,15804211,15804235,15804256,15804295,15804357,15804394,15804401,15804441,15804442,15804460,15804468,15804480,15804482,15804483,15804485,15804486,15804491,15804496,15804515,15804516,15804526,15804545,15804586,15804602,15804610,15804709,15804721,15804733,15804762,15804763,15804769,15804771,15804781,15804787,15804791,15804796,15804797,15804814,15804815,15804816,15804820,15804822,15804844,15804849,15804851,15804853,15804862,15804863,15804864,15804867,15804874,15804887,15804910,15804912,15804915,15804919,15804926,15804932,15804937,15804946,15804950,15804952,15804953,15804954,15804956,15804957,15804958,15804959,15804960,15804962,15805020,15805025,15805050,15805062,15805070,15805078,15805085,15805104,15805112,15805162,15805212,15805254,15805260,15805261,15805303,15805312,15805327,15805413,15805425,15805426,15805429,15805449,15805463,15805522,15805523,15805524,15805525,15805564,15805565,15805627,15805637,15805638,15805660,15805671,15805676,15805690,15805704,15805737,15805739,15805741,15805764,15805765,15805767,15805768,15805769,15805827,15805860,15805865,15805868,15805879,15805882,15805883,15805884,15805885,15805887,15805909,15805937,15805953,15805954,15805955,15805957,15805960,15805968,15805978,15805989,15806027,15806030,15806042,15806046,15806049,15806060,15806062,15806064,15806070,15806071,15806081,15806085,15806099,15806132,15806134,15806184,15806210,15806230,15806249,15806256,15806284,15806300,15806307,15806310,15806318,15806326,15806327,15806332,15806360,15806370,15806389,15806403,15806407,15806413,15806420,15806424,15806425,15806426,15806433,15806434,15806437,15806438,15806447,15806448,15806461,15806463,15806467,15806471,15806486,15806488,15806495,15806511,15806513,15806522,15806523,15806526,15806535,15806539,15806546,15806560,15806563,15806569,15806570,15806585,15806593,15806596,15806688,15806709,15806760,15806771,15806780,15806787,15806795,15806796,15806808,15806810,15806811,15806817,15806822,15806823,15806833,15806836,15806837,15806838,15806839,15806840,15806845,15806850,15806855,15806868,15806870,15806880,15806901,15806905,15806906,15806909,15806912,15806913,15806915,15806916,15806917,15806918,15806919,15806921,15806922,15806923,15806924,15806925,15806926,15806927,15806929,15806932,15806933,15806934,15806936,15806937,15806938,15806939,15806941,15806944,15806945,15806946,15806947,15806953,15806954,15806955,15806956,15806957,15806958,15806959,15806961,15806963,15806964,15806966,15806967,15806968,15806969,15806975,15806978,15806981,15806982,15806983,15806985,15807002,15807003,15807005,15807007,15807008,15807009,15807045,15807052,15807065,15807075,15807078,15807085,15807107,15807120,15807163,15807167,15807194,15807221,15807239,15807245,15807246,15807263,15807265,15807269,15807293,15807294,15807305,15807312,15807314,15807316,15807330,15807335,15807340,15807350,15807360,15807432,15807433,15807457,15807473,15807475,15807481,15807523,15807524,15807525,15807526,15807527,15807529,15807535,15807538,15807546,15807548,15807550,15807553,15807555,15807556,15807560,15807561,15807562,15807563,15807565,15807567,15807568,15807575,15807585,15807588,15807591,15807593,15807595,15807596,15807597,15807599,15807609,15807663,15807690,15807709,15807728,15807736,15807768,15807769,15807770,15807797,15807837,15807889,15807900,15807901,15807905,15807906,15807908,15807909,15807918,15807919,15807921,15807923,15807933,15807937,15807941,15807958,15807960,15807961,15807962,15807963,15807968,15807983,15807987,15807989,15807993,15808017,15808023,15808024,15808044,15808046,15808048,15808056,15808065,15808175,15808182,15808183,15808189,15808223,15808228,15808283,15808326,15808332,15808334,15808386,15808389,15808473,15808513,15808521,15808523,15808526,15808544,15808546,15808548,15808557,15808560,15808566,15808577,15808582,15808621,15808650,15808662,15808674,15808689,15808710,15808713,15808760,15808770,15808780,15808784,15808815,15808831,15808837,15808838,15808845,15808846,15808849,15808851,15808853,15808868,15808884,15808886,15808888,15808902,15808904,15808905,15808909,15808930,15808948,15808960,15808971,15808980,15809057,15809087,15809100,15809109,15809117,15809124,15809137,15809138,15809143,15809178,15809187,15809222,15809227,15809228,15809229,15809235,15809248,15809263,15809265,15809279,15809286,15809302,15809309,15809320,15809347,15809462,15809509,15809511,15809515,15809517,15809523,15809558,15809585,15809616,15809654,15809663,15809718,15809720,15809722,15809735,15809736,15809737,15809738,15809741,15809772,15809777,15809780,15809800,15809805,15809816,15809817,15809822,15809823,15809824,15809826,15809828,15809830,15809831,15809834,15809835,15809836,15809837,15809838,15809841,15809846,15809847,15809848,15809861,15809862,15809869,15809870,15809871,15809872,15809901,15809902,15809904,15809905,15809906,15809907,15809908,15809909,15809917,15809991,15809999,15810010,15810075,15810105,15810147,15810167,15810203,15810218,15810227,15810347,15810385,15810389,15810390,15810416,15810418,15810432,15810444,15810454,15810457,15810460,15810485,15810563,15810593,15810660,15810711,15810714,15810716,15810722,15810723,15810732,15810764,15810765,15810775,15810781,15810786,15810800,15810807,15810826,15810828,15810829,15810834,15810837,15810839,15810842,15810844,15810845,15810864,15810876,15810878,15810891,15810895,15810898,15810900,15810910,15810921,15810930,15810942,15810943,15810944,15810972,15810978,15810995,15811023,15811026,15811028,15811031,15811032,15811036,15811053,15811127,15811177,15811191,15811200,15811236,15811249,15811261,15811286,15811287,15811290,15811313,15811314,15811315,15811327,15811338,15811343,15811371,15811389,15811415,15811486,15811490,15811565,15811579,15811588,15811589,15811590,15811594,15811613,15811614,15811687,15811690,15811742,15811762,15811773,15811786,15811800,15811842,15811877,15811927,15811947,15811958,15811970,15811978,15811985,15812007,15812009,15812040,15812071,15812073,15812087,15812121,15812184,15812191,15812197,15812198,15812199,15812210,15812230,15812241,15812262,15812272,15812277,15812279,15812295,15812324,15812336,15812338,15812351,15812356,15812365,15812403,15812418,15812422,15812439,15812444,15812457,15812470,15812482,15812497,15812513,15812515,15812516,15812517,15812518,15812534,15812594,15812607,15812611,15812613,15812614,15812616,15812626,15812627,15812628,15812647,15812660,15812706,15812715,15812720,15812746,15812750,15812767,15812769,15812832,15812850,15812877,15812878,15812888,15812893,15812917,15812918,15812920,15812930,15812942,15812945,15812981,15812982,15813022,15813032,15813034,15813035,15813043,15813048,15813063,15813065,15813067,15813095,15813113,15813132,15813140,15813141,15813159,15813163,15813168,15813172,15813192,15813193,15813197,15813283,15813292,15813303,15813327,15813347,15813363,15813412,15813422,15813426,15813427,15813432,15813439,15813444,15813451,15813503,15813504,15813532,15813590,15813630,15813640,15813645,15813655,15813659,15813660,15813673,15813681,15813697,15813710,15813718,15813721,15813741,15813745,15813843,15813844,15813846,15813850,15813862,15813871,15813876,15813877,15813886,15813894,15813911,15813912,15813916,15813917,15813945,15813946,15813949,15813974,15814004,15814022,15814032,15814033,15814035,15814040,15814092,15814116,15814209,15814221,15814236,15814237,15814238,15814241,15814267,15814268,15814275,15814331,15814339,15814405,15814415,15814430,15814465,15814468,15814519,15814536,15814553,15814664,15814675,15814690,15814750,15814757,15814772,15814816,15814834,15814842,15814846,15814923,15814930,15814940,15814998,15815040,15815043,15815050,15815064,15815070,15815080,15815091,15815095,15815097,15815099,15815125,15815131,15815140,15815236,15815241,15815254,15815259,15815271,15815295,15815296,15815316,15815364,15815420,15815428,15815443,15815444,15815460,15815490,15815510,15815530,15815534,15815552,15815560,15815566,15815613,15815615,15815620,15815626,15815627,15815628,15815630,15815640,15815645,15815656,15815660,15815670,15815690,
}

# Gradio介面

In [ ]:
import gradio as gr

def get_value(id, sex, age, p1, p2, geogr, year, act, score, balance):
  # 定義眾數補值
    MODE_AGE = 37.0
    MODE_SCORE = 850
    MODE_BALANCE = 0.0

    # try:
    #     age = int(age)
    # except:
    #     return "請輸入正確的[年齡]數字"
    try:
        customer_id = int(id)
        if customer_id not in VALID_CUSTOMER_IDS:
            return f"CustomerId [{customer_id}] 未出現在訓練資料中，請確認是否輸入正確"
    except:
        return "請輸入正確的 [CustomerId] 數字"
    try:
        score = int(score)
    except:
        return "請輸入正確的[credit_score]數字"
    try:
        balance = float(balance)
    except:
        return "請輸入正確的[balance]數字"


    sex_value = 'female'
    if sex == '男':
        sex_value = 'male'
    elif sex == '女':
        sex_value = 'female'

    geogr_value = 'France'
    if geogr == 'Spain':
        geogr_value = 'Spain'
    elif geogr == 'Germany':
        geogr_value = 'Germany'

    p1_value = 1
    if p1 == '有':
        p1_value = 1

    act_value = 1
    if act == '是':
        act_value = 1
    # Age 補值
    try:
        age = int(age)
        if age < 18 or age > 100:
            age = MODE_AGE
    except:
        age = MODE_AGE
    # CreditScore 補值
    try:
        score = int(score)
        if score < 300 or score > 1000:
            score = MODE_SCORE
    except:
        score = MODE_SCORE

    # Balance 補值
    try:
        balance = float(balance)
        if balance < 0:
            balance = MODE_BALANCE
    except:
        balance = MODE_BALANCE


    data_dcit = {
        #"CustomerId" : int(id),
        "Gender" : sex_value,
        "Age" : int(age),
        "HasCrCard" : p1_value,
        "NumOfProducts" : int(p2),
        "Geography" : geogr_value,
        "Tenure" : int(year),
        "IsActiveMember": act_value,
        "CreditScore": int(score),
        "Balance": float(balance),

    }

    X = ap.transform(data_dcit)
    y_pred = model.predict_proba(X)[:,1]
    v = int(y_pred*100)
    return f"客戶流失率為 {v}%"

with gr.Blocks() as demo:
    gr.Markdown("## 銀行客戶預測\n輸入客戶的資料，預測流失機率")

    with gr.Row():
      with gr.Column():
        id = gr.Textbox(label="CustomerId", value=None)
        sex = gr.Radio(["男", "女"], label="Gender", value="男")
        age = gr.Textbox(label="Age", value=None)
        geogr = gr.Dropdown(["", "France", "Spain", "Germany"],label="Geography", value="France")
        p1 = gr.Radio(choices=["有", "無"], label="HasCrCard", value="有")


      with gr.Column():
        p2 = gr.Dropdown(choices=["", "1", "2", "3", "4"], label="NumOfProducts", value="1")
        score = gr.Textbox(label="CreditScore", value=None)
        act = gr.Radio(choices=["否", "是"], label="IsActiveMember", value="是")
        year = gr.Dropdown(["", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"], label="Tenure", value="0")
        balance = gr.Textbox(label="Balance", value=None)




    eval_button = gr.Button("預估流失率")
    output = gr.Textbox(label="預測結果")

    eval_button.click(fn=get_value,
             inputs=[id, sex, age, p1, p2, geogr, year,
             act, score, balance],
             outputs=output)


# 啟動 Web 應用
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c9dc1d1c586e45b522.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
